In [1]:
import pandas as pd
import numpy as np
import requests

In [6]:
# test pd re.sample
path = "/Users/bytedance/binance_data/csv_data/YFIUSDT-1m-perpetual.csv"
df = pd.read_csv(
        path,
        header=None,
        names=[
            "open_timestamp",
            "open_price",
            "high_price",
            "low_price",
            "close_price",
            "volume",
            "close_timestamp",
            "quote_asset_volume",
            "number_of_trade",
            "tbbav",
            "tbqav",
            "ignore",
        ],
        dtype={
            "open_timestamp": int,
            "open_price": float,
            "high_price": float,
            "low_price": float,
            "close_price": float,
            "volume": float,
            "quote_asset_volume": float,
            "tbbav": float,
            "tbqav": float,
            "close_timestamp": int,
            "number_of_trade": int,
        },
    )
df = df.rename(
        columns={
            "close_price": "Close",
            "open_price": "Open",
            "high_price": "High",
            "low_price": "Low",
            "quote_asset_volume": "Volume",
            "number_of_trade": "Count",
        }
    )

df.drop_duplicates(subset=["open_timestamp"], inplace=True)
df["open_timestamp"] = df["open_timestamp"] // 1_000
df["close_timestamp"] = df["close_timestamp"] // 1_000
df = df.set_index("open_timestamp", drop=True)
df = df.sort_index(ascending=True)
start, end = df.index[0], df.index[-1]
df: pd.DataFrame = df.reindex(pd.Int64Index(np.arange(start, end + 60, 60)))
df["datetime"] = pd.to_datetime(df.index, unit="s")
df = df.set_index("datetime", drop=True)

/var/folders/1l/tb_5qjhd1kg2z9tn_sjtf2jc0000gn/T/ipykernel_82835/1655528444.py:51: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  df: pd.DataFrame = df.reindex(pd.Int64Index(np.arange(start, end + 60, 60)))


In [7]:
sub_df = df.iloc[:300]

In [9]:
sub_df.head(10)

,Open,High,Low,Close,volume,close_timestamp,Volume,Count,tbbav,tbqav,ignore
datetime,,,,,,,,,,,
2020-08-31 07:00:00,32000.0,32449.9,32000.0,32400.0,2.557,1598857259,82525.9560,88,1.081,34872.1355,0
2020-08-31 07:01:00,32400.0,32400.0,32045.5,32300.0,6.123,1598857319,197691.0292,175,2.391,77280.5195,0
2020-08-31 07:02:00,32200.5,32350.0,32101.0,32225.0,6.895,1598857379,222126.8032,171,3.029,97684.9212,0
2020-08-31 07:03:00,32210.0,32210.0,31603.1,31877.9,9.298,1598857439,298029.1068,224,1.998,64168.6287,0
2020-08-31 07:04:00,31940.0,32194.3,31850.0,32164.4,11.515,1598857499,369396.0142,164,8.585,275393.7634,0
2020-08-31 07:05:00,32159.5,32700.0,32125.0,32400.0,18.959,1598857559,613399.6591,284,13.345,431604.5741,0
2020-08-31 07:06:00,32400.0,32745.4,32300.0,32549.5,11.561,1598857619,375787.3718,176,5.655,184119.0564,0
2020-08-31 07:07:00,32600.0,32620.0,32440.0,32561.3,11.861,1598857679,386218.0079,184,3.503,114147.6546,0
2020-08-31 07:08:00,32551.8,32590.0,32300.0,32570.7,12.288,1598857739,398771.7200,190,5.736,186242.5269,0


In [15]:
agg_func = {
        "Close": "last",
        "High": "max",
        "Low": "min",
        "Open": "first",
        "Volume": "sum",
        "Count": "sum",
    }
sub_df_resample = (
        sub_df[["Open", "Close", "High", "Low", "Volume", "Count"]]
        .resample(
            "5min", kind="timestamp", label="left", closed="left"
        )
        .agg(agg_func)
    )

In [16]:
sub_df_resample.head(2)

,Close,High,Low,Open,Volume,Count
datetime,,,,,,
2020-08-31 07:00:00,32164.4,32449.9,31603.1,32000.0,1.169769e+06,822
2020-08-31 07:05:00,33115.0,33168.2,32125.0,32159.5,2.387146e+06,1220


In [2]:
import json
perpetual_info = requests.get("https://fapi.binance.com/fapi/v1/exchangeInfo").json()
spot_info = requests.get("https://api.binance.com/api/v3/exchangeInfo").json()
perpetual_ticker = requests.get("https://fapi.binance.com/fapi/v1/ticker/24hr").json()
# perpetual_info = """{"timezone":"UTC","serverTime":1674504012713,"futuresType":"U_MARGINED","rateLimits":[{"rateLimitType":"REQUEST_WEIGHT","interval":"MINUTE","intervalNum":1,"limit":2400},{"rateLimitType":"ORDERS","interval":"MINUTE","intervalNum":1,"limit":1200},{"rateLimitType":"ORDERS","interval":"SECOND","intervalNum":10,"limit":300}],"exchangeFilters":[],"assets":[{"asset":"USDT","marginAvailable":true,"autoAssetExchange":"-10000"},{"asset":"BTC","marginAvailable":true,"autoAssetExchange":"-0.00100000"},{"asset":"BNB","marginAvailable":true,"autoAssetExchange":"-10"},{"asset":"ETH","marginAvailable":true,"autoAssetExchange":"-5"},{"asset":"XRP","marginAvailable":true,"autoAssetExchange":"0"},{"asset":"ADA","marginAvailable":true,"autoAssetExchange":"0"},{"asset":"DOT","marginAvailable":true,"autoAssetExchange":"0"},{"asset":"SOL","marginAvailable":true,"autoAssetExchange":"0"},{"asset":"BUSD","marginAvailable":true,"autoAssetExchange":"-10000"}],"symbols":[{"symbol":"BTCUSDT","pair":"BTCUSDT","contractType":"PERPETUAL","deliveryDate":4133404800000,"onboardDate":1569398400000,"status":"TRADING","maintMarginPercent":"2.5000","requiredMarginPercent":"5.0000","baseAsset":"BTC","quoteAsset":"USDT","marginAsset":"USDT","pricePrecision":2,"quantityPrecision":3,"baseAssetPrecision":8,"quotePrecision":8,"underlyingType":"COIN","underlyingSubType":["PoW"],"settlePlan":0,"triggerProtect":"0.0500","liquidationFee":"0.012500","marketTakeBound":"0.05","filters":[{"minPrice":"556.80","maxPrice":"4529764","filterType":"PRICE_FILTER","tickSize":"0.10"},{"stepSize":"0.001","filterType":"LOT_SIZE","maxQty":"1000","minQty":"0.001"},{"stepSize":"0.001","filterType":"MARKET_LOT_SIZE","maxQty":"120","minQty":"0.001"},{"limit":200,"filterType":"MAX_NUM_ORDERS"},{"limit":10,"filterType":"MAX_NUM_ALGO_ORDERS"},{"notional":"5","filterType":"MIN_NOTIONAL"},{"multiplierDown":"0.9500","multiplierUp":"1.0500","multiplierDecimal":"4","filterType":"PERCENT_PRICE"}],"orderTypes":["LIMIT","MARKET","STOP","STOP_MARKET","TAKE_PROFIT","TAKE_PROFIT_MARKET","TRAILING_STOP_MARKET"],"timeInForce":["GTC","IOC","FOK","GTX"]},{"symbol":"ETHUSDT","pair":"ETHUSDT","contractType":"PERPETUAL","deliveryDate":4133404800000,"onboardDate":1569398400000,"status":"TRADING","maintMarginPercent":"2.5000","requiredMarginPercent":"5.0000","baseAsset":"ETH","quoteAsset":"USDT","marginAsset":"USDT","pricePrecision":2,"quantityPrecision":3,"baseAssetPrecision":8,"quotePrecision":8,"underlyingType":"COIN","underlyingSubType":["Layer-1"],"settlePlan":0,"triggerProtect":"0.0500","liquidationFee":"0.012500","marketTakeBound":"0.05","filters":[{"minPrice":"39.86","maxPrice":"306177","filterType":"PRICE_FILTER","tickSize":"0.01"},{"stepSize":"0.001","filterType":"LOT_SIZE","maxQty":"10000","minQty":"0.001"},{"stepSize":"0.001","filterType":"MARKET_LOT_SIZE","maxQty":"2000","minQty":"0.001"},{"limit":200,"filterType":"MAX_NUM_ORDERS"},{"limit":10,"filterType":"MAX_NUM_ALGO_ORDERS"},{"notional":"5","filterType":"MIN_NOTIONAL"},{"multiplierDown":"0.9500","multiplierUp":"1.0500","multiplierDecimal":"4","filterType":"PERCENT_PRICE"}],"orderTypes":["LIMIT","MARKET","STOP","STOP_MARKET","TAKE_PROFIT","TAKE_PROFIT_MARKET","TRAILING_STOP_MARKET"],"timeInForce":["GTC","IOC","FOK","GTX"]},{"symbol":"BCHUSDT","pair":"BCHUSDT","contractType":"PERPETUAL","deliveryDate":4133404800000,"onboardDate":1569398400000,"status":"TRADING","maintMarginPercent":"2.5000","requiredMarginPercent":"5.0000","baseAsset":"BCH","quoteAsset":"USDT","marginAsset":"USDT","pricePrecision":2,"quantityPrecision":3,"baseAssetPrecision":8,"quotePrecision":8,"underlyingType":"COIN","underlyingSubType":["PoW"],"settlePlan":0,"triggerProtect":"0.0500","liquidationFee":"0.012500","marketTakeBound":"0.05","filters":[{"minPrice":"13.93","maxPrice":"100000","filterType":"PRICE_FILTER","tickSize":"0.01"},{"stepSize":"0.001","filterType":"LOT_SIZE","maxQty":"10000","minQty":"0.001"},{"stepSize":"0.001","filterType":"MARKET_LOT_SIZE","maxQty":"850","minQty":"0.001"},{"limit":200,"filterType":"MAX_NUM_ORDERS"},{"limit":10,"filterType":"MAX_NUM_ALGO_ORDERS"},{"notional":"5","filterType":"MIN_NOTIONAL"},{"multiplierDown":"0.9500","multiplierUp":"1.0500","multiplierDecimal":"4","filterType":"PERCENT_PRICE"}],"orderTypes":["LIMIT","MARKET","STOP","STOP_MARKET","TAKE_PROFIT","TAKE_PROFIT_MARKET","TRAILING_STOP_MARKET"],"timeInForce":["GTC","IOC","FOK","GTX"]},{"symbol":"XRPUSDT","pair":"XRPUSDT","contractType":"PERPETUAL","deliveryDate":4133404800000,"onboardDate":1569398400000,"status":"TRADING","maintMarginPercent":"2.5000","requiredMarginPercent":"5.0000","baseAsset":"XRP","quoteAsset":"USDT","marginAsset":"USDT","pricePrecision":4,"quantityPrecision":1,"baseAssetPrecision":8,"quotePrecision":8,"underlyingType":"COIN","underlyingSubType":["Payment"],"settlePlan":0,"triggerProtect":"0.0500","liquidationFee":"0.010000","marketTakeBound":"0.05","filters":[{"minPrice":"0.0143","maxPrice":"100000","filterType":"PRICE_FILTER","tickSize":"0.0001"},{"stepSize":"0.1","filterType":"LOT_SIZE","maxQty":"20000000","minQty":"0.1"},{"stepSize":"0.1","filterType":"MARKET_LOT_SIZE","maxQty":"2000000","minQty":"0.1"},{"limit":200,"filterType":"MAX_NUM_ORDERS"},{"limit":10,"filterType":"MAX_NUM_ALGO_ORDERS"},{"notional":"5","filterType":"MIN_NOTIONAL"},{"multiplierDown":"0.9500","multiplierUp":"1.0500","multiplierDecimal":"4","filterType":"PERCENT_PRICE"}],"orderTypes":["LIMIT","MARKET","STOP","STOP_MARKET","TAKE_PROFIT","TAKE_PROFIT_MARKET","TRAILING_STOP_MARKET"],"timeInForce":["GTC","IOC","FOK","GTX"]},{"symbol":"EOSUSDT","pair":"EOSUSDT","contractType":"PERPETUAL","deliveryDate":4133404800000,"onboardDate":1569398400000,"status":"TRADING","maintMarginPercent":"2.5000","requiredMarginPercent":"5.0000","baseAsset":"EOS","quoteAsset":"USDT","marginAsset":"USDT","pricePrecision":3,"quantityPrecision":1,"baseAssetPrecision":8,"quotePrecision":8,"underlyingType":"COIN","underlyingSubType":["Layer-1"],"settlePlan":0,"triggerProtect":"0.1000","liquidationFee":"0.010000","marketTakeBound":"0.10","filters":[{"minPrice":"0.111","maxPrice":"100000","filterType":"PRICE_FILTER","tickSize":"0.001"},{"stepSize":"0.1","filterType":"LOT_SIZE","maxQty":"1000000","minQty":"0.1"},{"stepSize":"0.1","filterType":"MARKET_LOT_SIZE","maxQty":"120000","minQty":"0.1"},{"limit":200,"filterType":"MAX_NUM_ORDERS"},{"limit":10,"filterType":"MAX_NUM_ALGO_ORDERS"},{"notional":"5","filterType":"MIN_NOTIONAL"},{"multiplierDown":"0.9000","multiplierUp":"1.1000","multiplierDecimal":"4","filterType":"PERCENT_PRICE"}],"orderTypes":["LIMIT","MARKET","STOP","STOP_MARKET","TAKE_PROFIT","TAKE_PROFIT_MARKET","TRAILING_STOP_MARKET"],"timeInForce":["GTC","IOC","FOK","GTX"]},{"symbol":"LTCUSDT","pair":"LTCUSDT","contractType":"PERPETUAL","deliveryDate":4133404800000,"onboardDate":1569398400000,"status":"TRADING","maintMarginPercent":"2.5000","requiredMarginPercent":"5.0000","baseAsset":"LTC","quoteAsset":"USDT","marginAsset":"USDT","pricePrecision":2,"quantityPrecision":3,"baseAssetPrecision":8,"quotePrecision":8,"underlyingType":"COIN","underlyingSubType":["PoW"],"settlePlan":0,"triggerProtect":"0.0500","liquidationFee":"0.020000","marketTakeBound":"0.05","filters":[{"minPrice":"3.61","maxPrice":"100000","filterType":"PRICE_FILTER","tickSize":"0.01"},{"stepSize":"0.001","filterType":"LOT_SIZE","maxQty":"10000","minQty":"0.001"},{"stepSize":"0.001","filterType":"MARKET_LOT_SIZE","maxQty":"1500","minQty":"0.001"},{"limit":200,"filterType":"MAX_NUM_ORDERS"},{"limit":10,"filterType":"MAX_NUM_ALGO_ORDERS"},{"notional":"5","filterType":"MIN_NOTIONAL"},{"multiplierDown":"0.9500","multiplierUp":"1.0500","multiplierDecimal":"4","filterType":"PERCENT_PRICE"}],"orderTypes":["LIMIT","MARKET","STOP","STOP_MARKET","TAKE_PROFIT","TAKE_PROFIT_MARKET","TRAILING_STOP_MARKET"],"timeInForce":["GTC","IOC","FOK","GTX"]},{"symbol":"TRXUSDT","pair":"TRXUSDT","contractType":"PERPETUAL","deliveryDate":4133404800000,"onboardDate":1569398400000,"status":"TRADING","maintMarginPercent":"2.5000","requiredMarginPercent":"5.0000","baseAsset":"TRX","quoteAsset":"USDT","marginAsset":"USDT","pricePrecision":5,"quantityPrecision":0,"baseAssetPrecision":8,"quotePrecision":8,"underlyingType":"COIN","underlyingSubType":["Layer-1"],"settlePlan":0,"triggerProtect":"0.0500","liquidationFee":"0.010000","marketTakeBound":"0.05","filters":[{"minPrice":"0.00132","maxPrice":"20000","filterType":"PRICE_FILTER","tickSize":"0.00001"},{"stepSize":"1","filterType":"LOT_SIZE","maxQty":"10000000","minQty":"1"},{"stepSize":"1","filterType":"MARKET_LOT_SIZE","maxQty":"5000000","minQty":"1"},{"limit":200,"filterType":"MAX_NUM_ORDERS"},{"limit":10,"filterType":"MAX_NUM_ALGO_ORDERS"},{"notional":"5","filterType":"MIN_NOTIONAL"},{"multiplierDown":"0.9500","multiplierUp":"1.0500","multiplierDecimal":"4","filterType":"PERCENT_PRICE"}],"orderTypes":["LIMIT","MARKET","STOP","STOP_MARKET","TAKE_PROFIT","TAKE_PROFIT_MARKET","TRAILING_STOP_MARKET"],"timeInForce":["GTC","IOC","FOK","GTX"]},{"symbol":"ETCUSDT","pair":"ETCUSDT","contractType":"PERPETUAL","deliveryDate":4133404800000,"onboardDate":1569398400000,"status":"TRADING","maintMarginPercent":"2.5000","requiredMarginPercent":"5.0000","baseAsset":"ETC","quoteAsset":"USDT","marginAsset":"USDT","pricePrecision":3,"quantityPrecision":2,"baseAssetPrecision":8,"quotePrecision":8,"underlyingType":"COIN","underlyingSubType":["Layer-1"],"settlePlan":0,"triggerProtect":"0.0500","liquidationFee":"0.010000","marketTakeBound":"0.10","filters":[{"minPrice":"1.051","maxPrice":"100000","filterType":"PRICE_FILTER","tickSize":"0.001"},{"stepSize":"0.01","filterType":"LOT_SIZE","maxQty":"100000","minQty":"0.01"},{"stepSize":"0.01","filterType":"MARKET_LOT_SIZE","maxQty":"10000","minQty":"0.01"},{"limit":200,"filterType":"MAX_NUM_ORDERS"},{"limit":10,"filterType":"MAX_NUM_ALGO_ORDERS"},{"notional":"5","filterType":"MIN_NOTIONAL"},{"multiplierDown":"0.9000","multiplierUp":"1.1000","multiplierDecimal":"4","filterType":"PERCENT_PRICE"}],"orderTypes":["LIMIT","MARKET","STOP","STOP_MARKET","TAKE_PROFIT","TAKE_PROFIT_MARKET","TRAILING_STOP_MARKET"],"timeInForce":["GTC","IOC","FOK","GTX"]},{"symbol":"LINKUSDT","pair":"LINKUSDT","contractType":"PERPETUAL","deliveryDate":4133404800000,"onboardDate":1569398400000,"status":"TRADING","maintMarginPercent":"2.5000","requiredMarginPercent":"5.0000","baseAsset":"LINK","quoteAsset":"USDT","marginAsset":"USDT","pricePrecision":3,"quantityPrecision":2,"baseAssetPrecision":8,"quotePrecision":8,"underlyingType":"COIN","underlyingSubType":["Oracle"],"settlePlan":0,"triggerProtect":"0.1000","liquidationFee":"0.010000","marketTakeBound":"0.10","filters":[{"minPrice":"0.464","maxPrice":"100000","filterType":"PRICE_FILTER","tickSize":"0.001"},{"stepSize":"0.01","filterType":"LOT_SIZE","maxQty":"200000","minQty":"0.01"},{"stepSize":"0.01","filterType":"MARKET_LOT_SIZE","maxQty":"20000","minQty":"0.01"},{"limit":200,"filterType":"MAX_NUM_ORDERS"},{"limit":10,"filterType":"MAX_NUM_ALGO_ORDERS"},{"notional":"5","filterType":"MIN_NOTIONAL"},{"multiplierDown":"0.9000","multiplierUp":"1.1000","multiplierDecimal":"4","filterType":"PERCENT_PRICE"}],"orderTypes":["LIMIT","MARKET","STOP","STOP_MARKET","TAKE_PROFIT","TAKE_PROFIT_MARKET","TRAILING_STOP_MARKET"],"timeInForce":["GTC","IOC","FOK","GTX"]},{"symbol":"XLMUSDT","pair":"XLMUSDT","contractType":"PERPETUAL","deliveryDate":4133404800000,"onboardDate":1569398400000,"status":"TRADING","maintMarginPercent":"2.5000","requiredMarginPercent":"5.0000","baseAsset":"XLM","quoteAsset":"USDT","marginAsset":"USDT","pricePrecision":5,"quantityPrecision":0,"baseAssetPrecision":8,"quotePrecision":8,"underlyingType":"COIN","underlyingSubType":["Layer-2"],"settlePlan":0,"triggerProtect":"0.0500","liquidationFee":"0.010000","marketTakeBound":"0.05","filters":[{"minPrice":"0.00648","maxPrice":"20000","filterType":"PRICE_FILTER","tickSize":"0.00001"},{"stepSize":"1","filterType":"LOT_SIZE","maxQty":"10000000","minQty":"1"},{"stepSize":"1","filterType":"MARKET_LOT_SIZE","maxQty":"1000000","minQty":"1"},{"limit":200,"filterType":"MAX_NUM_ORDERS"},{"limit":10,"filterType":"MAX_NUM_ALGO_ORDERS"},{"notional":"5","filterType":"MIN_NOTIONAL"},{"multiplierDown":"0.9500","multiplierUp":"1.0500","multiplierDecimal":"4","filterType":"PERCENT_PRICE"}],"orderTypes":["LIMIT","MARKET","STOP","STOP_MARKET","TAKE_PROFIT","TAKE_PROFIT_MARKET","TRAILING_STOP_MARKET"],"timeInForce":["GTC","IOC","FOK","GTX"]},{"symbol":"ADAUSDT","pair":"ADAUSDT","contractType":"PERPETUAL","deliveryDate":4133404800000,"onboardDate":1569398400000,"status":"TRADING","maintMarginPercent":"2.5000","requiredMarginPercent":"5.0000","baseAsset":"ADA","quoteAsset":"USDT","marginAsset":"USDT","pricePrecision":5,"quantityPrecision":0,"baseAssetPrecision":8,"quotePrecision":8,"underlyingType":"COIN","underlyingSubType":["Layer-1"],"settlePlan":0,"triggerProtect":"0.0500","liquidationFee":"0.010000","marketTakeBound":"0.05","filters":[{"minPrice":"0.01740","maxPrice":"20000","filterType":"PRICE_FILTER","tickSize":"0.00010"},{"stepSize":"1","filterType":"LOT_SIZE","maxQty":"2000000","minQty":"1"},{"stepSize":"1","filterType":"MARKET_LOT_SIZE","maxQty":"300000","minQty":"1"},{"limit":200,"filterType":"MAX_NUM_ORDERS"},{"limit":10,"filterType":"MAX_NUM_ALGO_ORDERS"},{"notional":"5","filterType":"MIN_NOTIONAL"},{"multiplierDown":"0.9500","multiplierUp":"1.0500","multiplierDecimal":"4","filterType":"PERCENT_PRICE"}],"orderTypes":["LIMIT","MARKET","STOP","STOP_MARKET","TAKE_PROFIT","TAKE_PROFIT_MARKET","TRAILING_STOP_MARKET"],"timeInForce":["GTC","IOC","FOK","GTX"]},{"symbol":"XMRUSDT","pair":"XMRUSDT","contractType":"PERPETUAL","deliveryDate":4133404800000,"onboardDate":1569398400000,"status":"TRADING","maintMarginPercent":"2.5000","requiredMarginPercent":"5.0000","baseAsset":"XMR","quoteAsset":"USDT","marginAsset":"USDT","pricePrecision":2,"quantityPrecision":3,"baseAssetPrecision":8,"quotePrecision":8,"underlyingType":"COIN","underlyingSubType":["Privacy"],"settlePlan":0,"triggerProtect":"0.0500","liquidationFee":"0.012500","marketTakeBound":"0.05","filters":[{"minPrice":"4.36","maxPrice":"100000","filterType":"PRICE_FILTER","tickSize":"0.01"},{"stepSize":"0.001","filterType":"LOT_SIZE","maxQty":"10000","minQty":"0.001"},{"stepSize":"0.001","filterType":"MARKET_LOT_SIZE","maxQty":"2000","minQty":"0.001"},{"limit":200,"filterType":"MAX_NUM_ORDERS"},{"limit":10,"filterType":"MAX_NUM_ALGO_ORDERS"},{"notional":"5","filterType":"MIN_NOTIONAL"},{"multiplierDown":"0.9500","multiplierUp":"1.0500","multiplierDecimal":"4","filterType":"PERCENT_PRICE"}],"orderTypes":["LIMIT","MARKET","STOP","STOP_MARKET","TAKE_PROFIT","TAKE_PROFIT_MARKET","TRAILING_STOP_MARKET"],"timeInForce":["GTC","IOC","FOK","GTX"]},{"symbol":"DASHUSDT","pair":"DASHUSDT","contractType":"PERPETUAL","deliveryDate":4133404800000,"onboardDate":1569398400000,"status":"TRADING","maintMarginPercent":"2.5000","requiredMarginPercent":"5.0000","baseAsset":"DASH","quoteAsset":"USDT","marginAsset":"USDT","pricePrecision":2,"quantityPrecision":3,"baseAssetPrecision":8,"quotePrecision":8,"underlyingType":"COIN","underlyingSubType":["Payment"],"settlePlan":0,"triggerProtect":"0.1000","liquidationFee":"0.012500","marketTakeBound":"0.10","filters":[{"minPrice":"3.82","maxPrice":"100000","filterType":"PRICE_FILTER","tickSize":"0.01"},{"stepSize":"0.001","filterType":"LOT_SIZE","maxQty":"10000","minQty":"0.001"},{"stepSize":"0.001","filterType":"MARKET_LOT_SIZE","maxQty":"3000","minQty":"0.001"},{"limit":200,"filterType":"MAX_NUM_ORDERS"},{"limit":10,"filterType":"MAX_NUM_ALGO_ORDERS"},{"notional":"5","filterType":"MIN_NOTIONAL"},{"multiplierDown":"0.9000","multiplierUp":"1.1000","multiplierDecimal":"4","filterType":"PERCENT_PRICE"}],"orderTypes":["LIMIT","MARKET","STOP","STOP_MARKET","TAKE_PROFIT","TAKE_PROFIT_MARKET","TRAILING_STOP_MARKET"],"timeInForce":["GTC","IOC","FOK","GTX"]},{"symbol":"ZECUSDT","pair":"ZECUSDT","contractType":"PERPETUAL","deliveryDate":4133404800000,"onboardDate":1569398400000,"status":"TRADING","maintMarginPercent":"2.5000","requiredMarginPercent":"5.0000","baseAsset":"ZEC","quoteAsset":"USDT","marginAsset":"USDT","pricePrecision":2,"quantityPrecision":3,"baseAssetPrecision":8,"quotePrecision":8,"underlyingType":"COIN","underlyingSubType":["Privacy"],"settlePlan":0,"triggerProtect":"0.1000","liquidationFee":"0.012500","marketTakeBound":"0.10","filters":[{"minPrice":"2.85","maxPrice":"100000","filterType":"PRICE_FILTER","tickSize":"0.01"},{"stepSize":"0.001","filterType":"LOT_SIZE","maxQty":"10000","minQty":"0.001"},{"stepSize":"0.001","filterType":"MARKET_LOT_SIZE","maxQty":"5000","minQty":"0.001"},{"limit":200,"filterType":"MAX_NUM_ORDERS"},{"limit":10,"filterType":"MAX_NUM_ALGO_ORDERS"},{"notional":"5","filterType":"MIN_NOTIONAL"},{"multiplierDown":"0.9000","multiplierUp":"1.1000","multiplierDecimal":"4","filterType":"PERCENT_PRICE"}],"orderTypes":["LIMIT","MARKET","STOP","STOP_MARKET","TAKE_PROFIT","TAKE_PROFIT_MARKET","TRAILING_STOP_MARKET"],"timeInForce":["GTC","IOC","FOK","GTX"]},{"symbol":"XTZUSDT","pair":"XTZUSDT","contractType":"PERPETUAL","deliveryDate":4133404800000,"onboardDate":1569398400000,"status":"TRADING","maintMarginPercent":"2.5000","requiredMarginPercent":"5.0000","baseAsset":"XTZ","quoteAsset":"USDT","marginAsset":"USDT","pricePrecision":3,"quantityPrecision":1,"baseAssetPrecision":8,"quotePrecision":8,"underlyingType":"COIN","underlyingSubType":["Layer-1"],"settlePlan":0,"triggerProtect":"0.1000","liquidationFee":"0.010000","marketTakeBound":"0.10","filters":[{"minPrice":"0.064","maxPrice":"100000","filterType":"PRICE_FILTER","tickSize":"0.001"},{"stepSize":"0.1","filterType":"LOT_SIZE","maxQty":"1000000","minQty":"0.1"},{"stepSize":"0.1","filterType":"MARKET_LOT_SIZE","maxQty":"100000","minQty":"0.1"},{"limit":200,"filterType":"MAX_NUM_ORDERS"},{"limit":10,"filterType":"MAX_NUM_ALGO_ORDERS"},{"notional":"5","filterType":"MIN_NOTIONAL"},{"multiplierDown":"0.9000","multiplierUp":"1.1000","multiplierDecimal":"4","filterType":"PERCENT_PRICE"}],"orderTypes":["LIMIT","MARKET","STOP","STOP_MARKET","TAKE_PROFIT","TAKE_PROFIT_MARKET","TRAILING_STOP_MARKET"],"timeInForce":["GTC","IOC","FOK","GTX"]},{"symbol":"BNBUSDT","pair":"BNBUSDT","contractType":"PERPETUAL","deliveryDate":4133404800000,"onboardDate":1569398400000,"status":"TRADING","maintMarginPercent":"2.5000","requiredMarginPercent":"5.0000","baseAsset":"BNB","quoteAsset":"USDT","marginAsset":"USDT","pricePrecision":3,"quantityPrecision":2,"baseAssetPrecision":8,"quotePrecision":8,"underlyingType":"COIN","underlyingSubType":["CEX"],"settlePlan":0,"triggerProtect":"0.0500","liquidationFee":"0.012500","marketTakeBound":"0.05","filters":[{"minPrice":"6.600","maxPrice":"100000","filterType":"PRICE_FILTER","tickSize":"0.010"},{"stepSize":"0.01","filterType":"LOT_SIZE","maxQty":"100000","minQty":"0.01"},{"stepSize":"0.01","filterType":"MARKET_LOT_SIZE","maxQty":"2000","minQty":"0.01"},{"limit":200,"filterType":"MAX_NUM_ORDERS"},{"limit":10,"filterType":"MAX_NUM_ALGO_ORDERS"},{"notional":"5","filterType":"MIN_NOTIONAL"},{"multiplierDown":"0.9500","multiplierUp":"1.0500","multiplierDecimal":"4","filterType":"PERCENT_PRICE"}],"orderTypes":["LIMIT","MARKET","STOP","STOP_MARKET","TAKE_PROFIT","TAKE_PROFIT_MARKET","TRAILING_STOP_MARKET"],"timeInForce":["GTC","IOC","FOK","GTX"]},{"symbol":"ATOMUSDT","pair":"ATOMUSDT","contractType":"PERPETUAL","deliveryDate":4133404800000,"onboardDate":1569398400000,"status":"TRADING","maintMarginPercent":"2.5000","requiredMarginPercent":"5.0000","baseAsset":"ATOM","quoteAsset":"USDT","marginAsset":"USDT","pricePrecision":3,"quantityPrecision":2,"baseAssetPrecision":8,"quotePrecision":8,"underlyingType":"COIN","underlyingSubType":["Layer-1"],"settlePlan":0,"triggerProtect":"0.1000","liquidationFee":"0.015000","marketTakeBound":"0.10","filters":[{"minPrice":"0.251","maxPrice":"100000","filterType":"PRICE_FILTER","tickSize":"0.001"},{"stepSize":"0.01","filterType":"LOT_SIZE","maxQty":"100000","minQty":"0.01"},{"stepSize":"0.01","filterType":"MARKET_LOT_SIZE","maxQty":"20000","minQty":"0.01"},{"limit":200,"filterType":"MAX_NUM_ORDERS"},{"limit":10,"filterType":"MAX_NUM_ALGO_ORDERS"},{"notional":"5","filterType":"MIN_NOTIONAL"},{"multiplierDown":"0.9000","multiplierUp":"1.1000","multiplierDecimal":"4","filterType":"PERCENT_PRICE"}],"orderTypes":["LIMIT","MARKET","STOP","STOP_MARKET","TAKE_PROFIT","TAKE_PROFIT_MARKET","TRAILING_STOP_MARKET"],"timeInForce":["GTC","IOC","FOK","GTX"]},{"symbol":"ONTUSDT","pair":"ONTUSDT","contractType":"PERPETUAL","deliveryDate":4133404800000,"onboardDate":1569398400000,"status":"TRADING","maintMarginPercent":"2.5000","requiredMarginPercent":"5.0000","baseAsset":"ONT","quoteAsset":"USDT","marginAsset":"USDT","pricePrecision":4,"quantityPrecision":1,"baseAssetPrecision":8,"quotePrecision":8,"underlyingType":"COIN","underlyingSubType":["Storage"],"settlePlan":0,"triggerProtect":"0.1000","liquidationFee":"0.010000","marketTakeBound":"0.10","filters":[{"minPrice":"0.0241","maxPrice":"100000","filterType":"PRICE_FILTER","tickSize":"0.0001"},{"stepSize":"0.1","filterType":"LOT_SIZE","maxQty":"1000000","minQty":"0.1"},{"stepSize":"0.1","filterType":"MARKET_LOT_SIZE","maxQty":"250000","minQty":"0.1"},{"limit":200,"filterType":"MAX_NUM_ORDERS"},{"limit":10,"filterType":"MAX_NUM_ALGO_ORDERS"},{"notional":"5","filterType":"MIN_NOTIONAL"},{"multiplierDown":"0.9000","multiplierUp":"1.1000","multiplierDecimal":"4","filterType":"PERCENT_PRICE"}],"orderTypes":["LIMIT","MARKET","STOP","STOP_MARKET","TAKE_PROFIT","TAKE_PROFIT_MARKET","TRAILING_STOP_MARKET"],"timeInForce":["GTC","IOC","FOK","GTX"]},{"symbol":"IOTAUSDT","pair":"IOTAUSDT","contractType":"PERPETUAL","deliveryDate":4133404800000,"onboardDate":1569398400000,"status":"TRADING","maintMarginPercent":"2.5000","requiredMarginPercent":"5.0000","baseAsset":"IOTA","quoteAsset":"USDT","marginAsset":"USDT","pricePrecision":4,"quantityPrecision":1,"baseAssetPrecision":8,"quotePrecision":8,"underlyingType":"COIN","underlyingSubType":["Layer-1"],"settlePlan":0,"triggerProtect":"0.1000","liquidationFee":"0.010000","marketTakeBound":"0.10","filters":[{"minPrice":"0.0205","maxPrice":"100000","filterType":"PRICE_FILTER","tickSize":"0.0001"},{"stepSize":"0.1","filterType":"LOT_SIZE","maxQty":"1000000","minQty":"0.1"},{"stepSize":"0.1","filterType":"MARKET_LOT_SIZE","maxQty":"50000","minQty":"0.1"},{"limit":200,"filterType":"MAX_NUM_ORDERS"},{"limit":10,"filterType":"MAX_NUM_ALGO_ORDERS"},{"notional":"5","filterType":"MIN_NOTIONAL"},{"multiplierDown":"0.9000","multiplierUp":"1.1000","multiplierDecimal":"4","filterType":"PERCENT_PRICE"}],"orderTypes":["LIMIT","MARKET","STOP","STOP_MARKET","TAKE_PROFIT","TAKE_PROFIT_MARKET","TRAILING_STOP_MARKET"],"timeInForce":["GTC","IOC","FOK","GTX"]},{"symbol":"BATUSDT","pair":"BATUSDT","contractType":"PERPETUAL","deliveryDate":4133404800000,"onboardDate":1569398400000,"status":"TRADING","maintMarginPercent":"2.5000","requiredMarginPercent":"5.0000","baseAsset":"BAT","quoteAsset":"USDT","marginAsset":"USDT","pricePrecision":4,"quantityPrecision":1,"baseAssetPrecision":8,"quotePrecision":8,"underlyingType":"COIN","underlyingSubType":["DeFi"],"settlePlan":0,"triggerProtect":"0.1000","liquidationFee":"0.010000","marketTakeBound":"0.10","filters":[{"minPrice":"0.0134","maxPrice":"100000","filterType":"PRICE_FILTER","tickSize":"0.0001"},{"stepSize":"0.1","filterType":"LOT_SIZE","maxQty":"1000000","minQty":"0.1"},{"stepSize":"0.1","filterType":"MARKET_LOT_SIZE","maxQty":"600000","minQty":"0.1"},{"limit":200,"filterType":"MAX_NUM_ORDERS"},{"limit":10,"filterType":"MAX_NUM_ALGO_ORDERS"},{"notional":"5","filterType":"MIN_NOTIONAL"},{"multiplierDown":"0.9000","multiplierUp":"1.1000","multiplierDecimal":"4","filterType":"PERCENT_PRICE"}],"orderTypes":["LIMIT","MARKET","STOP","STOP_MARKET","TAKE_PROFIT","TAKE_PROFIT_MARKET","TRAILING_STOP_MARKET"],"timeInForce":["GTC","IOC","FOK","GTX"]},{"symbol":"VETUSDT","pair":"VETUSDT","contractType":"PERPETUAL","deliveryDate":4133404800000,"onboardDate":1569398400000,"status":"TRADING","maintMarginPercent":"2.5000","requiredMarginPercent":"5.0000","baseAsset":"VET","quoteAsset":"USDT","marginAsset":"USDT","pricePrecision":6,"quantityPrecision":0,"baseAssetPrecision":8,"quotePrecision":8,"underlyingType":"COIN","underlyingSubType":["Layer-1"],"settlePlan":0,"triggerProtect":"0.0500","liquidationFee":"0.010000","marketTakeBound":"0.05","filters":[{"minPrice":"0.002080","maxPrice":"2000","filterType":"PRICE_FILTER","tickSize":"0.000010"},{"stepSize":"1","filterType":"LOT_SIZE","maxQty":"10000000","minQty":"1"},{"stepSize":"1","filterType":"MARKET_LOT_SIZE","maxQty":"2500000","minQty":"1"},{"limit":200,"filterType":"MAX_NUM_ORDERS"},{"limit":10,"filterType":"MAX_NUM_ALGO_ORDERS"},{"notional":"5","filterType":"MIN_NOTIONAL"},{"multiplierDown":"0.9500","multiplierUp":"1.0500","multiplierDecimal":"4","filterType":"PERCENT_PRICE"}],"orderTypes":["LIMIT","MARKET","STOP","STOP_MARKET","TAKE_PROFIT","TAKE_PROFIT_MARKET","TRAILING_STOP_MARKET"],"timeInForce":["GTC","IOC","FOK","GTX"]},{"symbol":"NEOUSDT","pair":"NEOUSDT","contractType":"PERPETUAL","deliveryDate":4133404800000,"onboardDate":1569398400000,"status":"TRADING","maintMarginPercent":"2.5000","requiredMarginPercent":"5.0000","baseAsset":"NEO","quoteAsset":"USDT","marginAsset":"USDT","pricePrecision":3,"quantityPrecision":2,"baseAssetPrecision":8,"quotePrecision":8,"underlyingType":"COIN","underlyingSubType":["Layer-1"],"settlePlan":0,"triggerProtect":"0.0500","liquidationFee":"0.010000","marketTakeBound":"0.05","filters":[{"minPrice":"1.093","maxPrice":"100000","filterType":"PRICE_FILTER","tickSize":"0.001"},{"stepSize":"0.01","filterType":"LOT_SIZE","maxQty":"100000","minQty":"0.01"},{"stepSize":"0.01","filterType":"MARKET_LOT_SIZE","maxQty":"10000","minQty":"0.01"},{"limit":200,"filterType":"MAX_NUM_ORDERS"},{"limit":10,"filterType":"MAX_NUM_ALGO_ORDERS"},{"notional":"5","filterType":"MIN_NOTIONAL"},{"multiplierDown":"0.9500","multiplierUp":"1.0500","multiplierDecimal":"4","filterType":"PERCENT_PRICE"}],"orderTypes":["LIMIT","MARKET","STOP","STOP_MARKET","TAKE_PROFIT","TAKE_PROFIT_MARKET","TRAILING_STOP_MARKET"],"timeInForce":["GTC","IOC","FOK","GTX"]},{"symbol":"QTUMUSDT","pair":"QTUMUSDT","contractType":"PERPETUAL","deliveryDate":4133404800000,"onboardDate":1569398400000,"status":"TRADING","maintMarginPercent":"2.5000","requiredMarginPercent":"5.0000","baseAsset":"QTUM","quoteAsset":"USDT","marginAsset":"USDT","pricePrecision":3,"quantityPrecision":1,"baseAssetPrecision":8,"quotePrecision":8,"underlyingType":"COIN","underlyingSubType":["Layer-1"],"settlePlan":0,"triggerProtect":"0.1000","liquidationFee":"0.010000","marketTakeBound":"0.10","filters":[{"minPrice":"0.246","maxPrice":"100000","filterType":"PRICE_FILTER","tickSize":"0.001"},{"stepSize":"0.1","filterType":"LOT_SIZE","maxQty":"500000","minQty":"0.1"},{"stepSize":"0.1","filterType":"MARKET_LOT_SIZE","maxQty":"10000","minQty":"0.1"},{"limit":200,"filterType":"MAX_NUM_ORDERS"},{"limit":10,"filterType":"MAX_NUM_ALGO_ORDERS"},{"notional":"5","filterType":"MIN_NOTIONAL"},{"multiplierDown":"0.9000","multiplierUp":"1.1000","multiplierDecimal":"4","filterType":"PERCENT_PRICE"}],"orderTypes":["LIMIT","MARKET","STOP","STOP_MARKET","TAKE_PROFIT","TAKE_PROFIT_MARKET","TRAILING_STOP_MARKET"],"timeInForce":["GTC","IOC","FOK","GTX"]},{"symbol":"IOSTUSDT","pair":"IOSTUSDT","contractType":"PERPETUAL","deliveryDate":4133404800000,"onboardDate":1569398400000,"status":"TRADING","maintMarginPercent":"2.5000","requiredMarginPercent":"5.0000","baseAsset":"IOST","quoteAsset":"USDT","marginAsset":"USDT","pricePrecision":6,"quantityPrecision":0,"baseAssetPrecision":8,"quotePrecision":8,"underlyingType":"COIN","underlyingSubType":["Layer-1"],"settlePlan":0,"triggerProtect":"0.1000","liquidationFee":"0.010000","marketTakeBound":"0.10","filters":[{"minPrice":"0.000587","maxPrice":"2000","filterType":"PRICE_FILTER","tickSize":"0.000001"},{"stepSize":"1","filterType":"LOT_SIZE","maxQty":"10000000","minQty":"1"},{"stepSize":"1","filterType":"MARKET_LOT_SIZE","maxQty":"5000000","minQty":"1"},{"limit":200,"filterType":"MAX_NUM_ORDERS"},{"limit":10,"filterType":"MAX_NUM_ALGO_ORDERS"},{"notional":"5","filterType":"MIN_NOTIONAL"},{"multiplierDown":"0.9000","multiplierUp":"1.1000","multiplierDecimal":"4","filterType":"PERCENT_PRICE"}],"orderTypes":["LIMIT","MARKET","STOP","STOP_MARKET","TAKE_PROFIT","TAKE_PROFIT_MARKET","TRAILING_STOP_MARKET"],"timeInForce":["GTC","IOC","FOK","GTX"]},{"symbol":"THETAUSDT","pair":"THETAUSDT","contractType":"PERPETUAL","deliveryDate":4133404800000,"onboardDate":1569398400000,"status":"TRADING","maintMarginPercent":"2.5000","requiredMarginPercent":"5.0000","baseAsset":"THETA","quoteAsset":"USDT","marginAsset":"USDT","pricePrecision":4,"quantityPrecision":1,"baseAssetPrecision":8,"quotePrecision":8,"underlyingType":"COIN","underlyingSubType":["NFT"],"settlePlan":0,"triggerProtect":"0.1000","liquidationFee":"0.010000","marketTakeBound":"0.10","filters":[{"minPrice":"0.1070","maxPrice":"1000","filterType":"PRICE_FILTER","tickSize":"0.0001"},{"stepSize":"0.1","filterType":"LOT_SIZE","maxQty":"1000000","minQty":"0.1"},{"stepSize":"0.1","filterType":"MARKET_LOT_SIZE","maxQty":"100000","minQty":"0.1"},{"limit":200,"filterType":"MAX_NUM_ORDERS"},{"limit":10,"filterType":"MAX_NUM_ALGO_ORDERS"},{"notional":"5","filterType":"MIN_NOTIONAL"},{"multiplierDown":"0.9000","multiplierUp":"1.1000","multiplierDecimal":"4","filterType":"PERCENT_PRICE"}],"orderTypes":["LIMIT","MARKET","STOP","STOP_MARKET","TAKE_PROFIT","TAKE_PROFIT_MARKET","TRAILING_STOP_MARKET"],"timeInForce":["GTC","IOC","FOK","GTX"]},{"symbol":"ALGOUSDT","pair":"ALGOUSDT","contractType":"PERPETUAL","deliveryDate":4133404800000,"onboardDate":1569398400000,"status":"TRADING","maintMarginPercent":"2.5000","requiredMarginPercent":"5.0000","baseAsset":"ALGO","quoteAsset":"USDT","marginAsset":"USDT","pricePrecision":4,"quantityPrecision":1,"baseAssetPrecision":8,"quotePrecision":8,"underlyingType":"COIN","underlyingSubType":["Layer-1"],"settlePlan":0,"triggerProtect":"0.0500","liquidationFee":"0.010000","marketTakeBound":"0.05","filters":[{"minPrice":"0.0141","maxPrice":"1000","filterType":"PRICE_FILTER","tickSize":"0.0001"},{"stepSize":"0.1","filterType":"LOT_SIZE","maxQty":"1000000","minQty":"0.1"},{"stepSize":"0.1","filterType":"MARKET_LOT_SIZE","maxQty":"500000","minQty":"0.1"},{"limit":200,"filterType":"MAX_NUM_ORDERS"},{"limit":10,"filterType":"MAX_NUM_ALGO_ORDERS"},{"notional":"5","filterType":"MIN_NOTIONAL"},{"multiplierDown":"0.9500","multiplierUp":"1.0500","multiplierDecimal":"4","filterType":"PERCENT_PRICE"}],"orderTypes":["LIMIT","MARKET","STOP","STOP_MARKET","TAKE_PROFIT","TAKE_PROFIT_MARKET","TRAILING_STOP_MARKET"],"timeInForce":["GTC","IOC","FOK","GTX"]},{"symbol":"ZILUSDT","pair":"ZILUSDT","contractType":"PERPETUAL","deliveryDate":4133404800000,"onboardDate":1569398400000,"status":"TRADING","maintMarginPercent":"2.5000","requiredMarginPercent":"5.0000","baseAsset":"ZIL","quoteAsset":"USDT","marginAsset":"USDT","pricePrecision":5,"quantityPrecision":0,"baseAssetPrecision":8,"quotePrecision":8,"underlyingType":"COIN","underlyingSubType":["Layer-1"],"settlePlan":0,"triggerProtect":"0.1000","liquidationFee":"0.010000","marketTakeBound":"0.10","filters":[{"minPrice":"0.00219","maxPrice":"100","filterType":"PRICE_FILTER","tickSize":"0.00001"},{"stepSize":"1","filterType":"LOT_SIZE","maxQty":"10000000","minQty":"1"},{"stepSize":"1","filterType":"MARKET_LOT_SIZE","maxQty":"5000000","minQty":"1"},{"limit":200,"filterType":"MAX_NUM_ORDERS"},{"limit":10,"filterType":"MAX_NUM_ALGO_ORDERS"},{"notional":"5","filterType":"MIN_NOTIONAL"},{"multiplierDown":"0.9000","multiplierUp":"1.1000","multiplierDecimal":"4","filterType":"PERCENT_PRICE"}],"orderTypes":["LIMIT","MARKET","STOP","STOP_MARKET","TAKE_PROFIT","TAKE_PROFIT_MARKET","TRAILING_STOP_MARKET"],"timeInForce":["GTC","IOC","FOK","GTX"]},{"symbol":"KNCUSDT","pair":"KNCUSDT","contractType":"PERPETUAL","deliveryDate":4133404800000,"onboardDate":1569398400000,"status":"TRADING","maintMarginPercent":"2.5000","requiredMarginPercent":"5.0000","baseAsset":"KNC","quoteAsset":"USDT","marginAsset":"USDT","pricePrecision":5,"quantityPrecision":0,"baseAssetPrecision":8,"quotePrecision":8,"underlyingType":"COIN","underlyingSubType":["DeFi"],"settlePlan":0,"triggerProtect":"0.1000","liquidationFee":"0.015000","marketTakeBound":"0.10","filters":[{"minPrice":"0.03200","maxPrice":"181","filterType":"PRICE_FILTER","tickSize":"0.00010"},{"stepSize":"1","filterType":"LOT_SIZE","maxQty":"5000000","minQty":"1"},{"stepSize":"1","filterType":"MARKET_LOT_SIZE","maxQty":"500000","minQty":"1"},{"limit":200,"filterType":"MAX_NUM_ORDERS"},{"limit":10,"filterType":"MAX_NUM_ALGO_ORDERS"},{"notional":"5","filterType":"MIN_NOTIONAL"},{"multiplierDown":"0.9000","multiplierUp":"1.1000","multiplierDecimal":"4","filterType":"PERCENT_PRICE"}],"orderTypes":["LIMIT","MARKET","STOP","STOP_MARKET","TAKE_PROFIT","TAKE_PROFIT_MARKET","TRAILING_STOP_MARKET"],"timeInForce":["GTC","IOC","FOK","GTX"]},{"symbol":"ZRXUSDT","pair":"ZRXUSDT","contractType":"PERPETUAL","deliveryDate":4133404800000,"onboardDate":1569398400000,"status":"TRADING","maintMarginPercent":"2.5000","requiredMarginPercent":"5.0000","baseAsset":"ZRX","quoteAsset":"USDT","marginAsset":"USDT","pricePrecision":4,"quantityPrecision":1,"baseAssetPrecision":8,"quotePrecision":8,"underlyingType":"COIN","underlyingSubType":["DEX"],"settlePlan":0,"triggerProtect":"0.1000","liquidationFee":"0.010000","marketTakeBound":"0.10","filters":[{"minPrice":"0.0179","maxPrice":"1000","filterType":"PRICE_FILTER","tickSize":"0.0001"},{"stepSize":"0.1","filterType":"LOT_SIZE","maxQty":"1000000","minQty":"0.1"},{"stepSize":"0.1","filterType":"MARKET_LOT_SIZE","maxQty":"500000","minQty":"0.1"},{"limit":200,"filterType":"MAX_NUM_ORDERS"},{"limit":10,"filterType":"MAX_NUM_ALGO_ORDERS"},{"notional":"5","filterType":"MIN_NOTIONAL"},{"multiplierDown":"0.9000","multiplierUp":"1.1000","multiplierDecimal":"4","filterType":"PERCENT_PRICE"}],"orderTypes":["LIMIT","MARKET","STOP","STOP_MARKET","TAKE_PROFIT","TAKE_PROFIT_MARKET","TRAILING_STOP_MARKET"],"timeInForce":["GTC","IOC","FOK","GTX"]},{"symbol":"COMPUSDT","pair":"COMPUSDT","contractType":"PERPETUAL","deliveryDate":4133404800000,"onboardDate":1569398400000,"status":"TRADING","maintMarginPercent":"2.5000","requiredMarginPercent":"5.0000","baseAsset":"COMP","quoteAsset":"USDT","marginAsset":"USDT","pricePrecision":2,"quantityPrecision":3,"baseAssetPrecision":8,"quotePrecision":8,"underlyingType":"COIN","underlyingSubType":["DeFi"],"settlePlan":0,"triggerProtect":"0.1000","liquidationFee":"0.012500","marketTakeBound":"0.10","filters":[{"minPrice":"8","maxPrice":"100000","filterType":"PRICE_FILTER","tickSize":"0.01"},{"stepSize":"0.001","filterType":"LOT_SIZE","maxQty":"10000","minQty":"0.001"},{"stepSize":"0.001","filterType":"MARKET_LOT_SIZE","maxQty":"1000","minQty":"0.001"},{"limit":200,"filterType":"MAX_NUM_ORDERS"},{"limit":10,"filterType":"MAX_NUM_ALGO_ORDERS"},{"notional":"5","filterType":"MIN_NOTIONAL"},{"multiplierDown":"0.9000","multiplierUp":"1.1000","multiplierDecimal":"4","filterType":"PERCENT_PRICE"}],"orderTypes":["LIMIT","MARKET","STOP","STOP_MARKET","TAKE_PROFIT","TAKE_PROFIT_MARKET","TRAILING_STOP_MARKET"],"timeInForce":["GTC","IOC","FOK","GTX"]},{"symbol":"OMGUSDT","pair":"OMGUSDT","contractType":"PERPETUAL","deliveryDate":4133404800000,"onboardDate":1569398400000,"status":"TRADING","maintMarginPercent":"2.5000","requiredMarginPercent":"5.0000","baseAsset":"OMG","quoteAsset":"USDT","marginAsset":"USDT","pricePrecision":4,"quantityPrecision":1,"baseAssetPrecision":8,"quotePrecision":8,"underlyingType":"COIN","underlyingSubType":["Layer-2"],"settlePlan":0,"triggerProtect":"0.1000","liquidationFee":"0.010000","marketTakeBound":"0.10","filters":[{"minPrice":"0.1060","maxPrice":"1000","filterType":"PRICE_FILTER","tickSize":"0.0010"},{"stepSize":"0.1","filterType":"LOT_SIZE","maxQty":"1000000","minQty":"0.1"},{"stepSize":"0.1","filterType":"MARKET_LOT_SIZE","maxQty":"10000","minQty":"0.1"},{"limit":200,"filterType":"MAX_NUM_ORDERS"},{"limit":10,"filterType":"MAX_NUM_ALGO_ORDERS"},{"notional":"5","filterType":"MIN_NOTIONAL"},{"multiplierDown":"0.9000","multiplierUp":"1.1000","multiplierDecimal":"4","filterType":"PERCENT_PRICE"}],"orderTypes":["LIMIT","MARKET","STOP","STOP_MARKET","TAKE_PROFIT","TAKE_PROFIT_MARKET","TRAILING_STOP_MARKET"],"timeInForce":["GTC","IOC","FOK","GTX"]},{"symbol":"DOGEUSDT","pair":"DOGEUSDT","contractType":"PERPETUAL","deliveryDate":4133404800000,"onboardDate":1569398400000,"status":"TRADING","maintMarginPercent":"2.5000","requiredMarginPercent":"5.0000","baseAsset":"DOGE","quoteAsset":"USDT","marginAsset":"USDT","pricePrecision":6,"quantityPrecision":0,"baseAssetPrecision":8,"quotePrecision":8,"underlyingType":"COIN","underlyingSubType":["Meme"],"settlePlan":0,"triggerProtect":"0.1000","liquidationFee":"0.010000","marketTakeBound":"0.10","filters":[{"minPrice":"0.002440","maxPrice":"30","filterType":"PRICE_FILTER","tickSize":"0.000010"},{"stepSize":"1","filterType":"LOT_SIZE","maxQty":"50000000","minQty":"1"},{"stepSize":"1","filterType":"MARKET_LOT_SIZE","maxQty":"30000000","minQty":"1"},{"limit":200,"filterType":"MAX_NUM_ORDERS"},{"limit":10,"filterType":"MAX_NUM_ALGO_ORDERS"},{"notional":"5","filterType":"MIN_NOTIONAL"},{"multiplierDown":"0.9000","multiplierUp":"1.1000","multiplierDecimal":"4","filterType":"PERCENT_PRICE"}],"orderTypes":["LIMIT","MARKET","STOP","STOP_MARKET","TAKE_PROFIT","TAKE_PROFIT_MARKET","TRAILING_STOP_MARKET"],"timeInForce":["GTC","IOC","FOK","GTX"]},{"symbol":"SXPUSDT","pair":"SXPUSDT","contractType":"PERPETUAL","deliveryDate":4133404800000,"onboardDate":1569398400000,"status":"TRADING","maintMarginPercent":"2.5000","requiredMarginPercent":"5.0000","baseAsset":"SXP","quoteAsset":"USDT","marginAsset":"USDT","pricePrecision":4,"quantityPrecision":1,"baseAssetPrecision":8,"quotePrecision":8,"underlyingType":"COIN","underlyingSubType":["Payment"],"settlePlan":0,"triggerProtect":"0.1000","liquidationFee":"0.010000","marketTakeBound":"0.10","filters":[{"minPrice":"0.0454","maxPrice":"1000","filterType":"PRICE_FILTER","tickSize":"0.0001"},{"stepSize":"0.1","filterType":"LOT_SIZE","maxQty":"1000000","minQty":"0.1"},{"stepSize":"0.1","filterType":"MARKET_LOT_SIZE","maxQty":"40000","minQty":"0.1"},{"limit":200,"filterType":"MAX_NUM_ORDERS"},{"limit":10,"filterType":"MAX_NUM_ALGO_ORDERS"},{"notional":"5","filterType":"MIN_NOTIONAL"},{"multiplierDown":"0.9000","multiplierUp":"1.1000","multiplierDecimal":"4","filterType":"PERCENT_PRICE"}],"orderTypes":["LIMIT","MARKET","STOP","STOP_MARKET","TAKE_PROFIT","TAKE_PROFIT_MARKET","TRAILING_STOP_MARKET"],"timeInForce":["GTC","IOC","FOK","GTX"]},{"symbol":"KAVAUSDT","pair":"KAVAUSDT","contractType":"PERPETUAL","deliveryDate":4133404800000,"onboardDate":1569398400000,"status":"TRADING","maintMarginPercent":"2.5000","requiredMarginPercent":"5.0000","baseAsset":"KAVA","quoteAsset":"USDT","marginAsset":"USDT","pricePrecision":4,"quantityPrecision":1,"baseAssetPrecision":8,"quotePrecision":8,"underlyingType":"COIN","underlyingSubType":["Layer-1"],"settlePlan":0,"triggerProtect":"0.1000","liquidationFee":"0.010000","marketTakeBound":"0.10","filters":[{"minPrice":"0.0593","maxPrice":"653","filterType":"PRICE_FILTER","tickSize":"0.0001"},{"stepSize":"0.1","filterType":"LOT_SIZE","maxQty":"1000000","minQty":"0.1"},{"stepSize":"0.1","filterType":"MARKET_LOT_SIZE","maxQty":"100000","minQty":"0.1"},{"limit":200,"filterType":"MAX_NUM_ORDERS"},{"limit":10,"filterType":"MAX_NUM_ALGO_ORDERS"},{"notional":"5","filterType":"MIN_NOTIONAL"},{"multiplierDown":"0.9000","multiplierUp":"1.1000","multiplierDecimal":"4","filterType":"PERCENT_PRICE"}],"orderTypes":["LIMIT","MARKET","STOP","STOP_MARKET","TAKE_PROFIT","TAKE_PROFIT_MARKET","TRAILING_STOP_MARKET"],"timeInForce":["GTC","IOC","FOK","GTX"]},{"symbol":"BANDUSDT","pair":"BANDUSDT","contractType":"PERPETUAL","deliveryDate":4133404800000,"onboardDate":1569398400000,"status":"TRADING","maintMarginPercent":"2.5000","requiredMarginPercent":"5.0000","baseAsset":"BAND","quoteAsset":"USDT","marginAsset":"USDT","pricePrecision":4,"quantityPrecision":1,"baseAssetPrecision":8,"quotePrecision":8,"underlyingType":"COIN","underlyingSubType":["Oracle"],"settlePlan":0,"triggerProtect":"0.1000","liquidationFee":"0.020000","marketTakeBound":"0.10","filters":[{"minPrice":"0.1647","maxPrice":"783","filterType":"PRICE_FILTER","tickSize":"0.0001"},{"stepSize":"0.1","filterType":"LOT_SIZE","maxQty":"2000000","minQty":"0.1"},{"stepSize":"0.1","filterType":"MARKET_LOT_SIZE","maxQty":"200000","minQty":"0.1"},{"limit":200,"filterType":"MAX_NUM_ORDERS"},{"limit":10,"filterType":"MAX_NUM_ALGO_ORDERS"},{"notional":"5","filterType":"MIN_NOTIONAL"},{"multiplierDown":"0.9000","multiplierUp":"1.1000","multiplierDecimal":"4","filterType":"PERCENT_PRICE"}],"orderTypes":["LIMIT","MARKET","STOP","STOP_MARKET","TAKE_PROFIT","TAKE_PROFIT_MARKET","TRAILING_STOP_MARKET"],"timeInForce":["GTC","IOC","FOK","GTX"]},{"symbol":"RLCUSDT","pair":"RLCUSDT","contractType":"PERPETUAL","deliveryDate":4133404800000,"onboardDate":1569398400000,"status":"TRADING","maintMarginPercent":"2.5000","requiredMarginPercent":"5.0000","baseAsset":"RLC","quoteAsset":"USDT","marginAsset":"USDT","pricePrecision":4,"quantityPrecision":1,"baseAssetPrecision":8,"quotePrecision":8,"underlyingType":"COIN","underlyingSubType":["Oracle"],"settlePlan":0,"triggerProtect":"0.1000","liquidationFee":"0.020000","marketTakeBound":"0.10","filters":[{"minPrice":"0.1029","maxPrice":"432","filterType":"PRICE_FILTER","tickSize":"0.0001"},{"stepSize":"0.1","filterType":"LOT_SIZE","maxQty":"1000000","minQty":"0.1"},{"stepSize":"0.1","filterType":"MARKET_LOT_SIZE","maxQty":"80000","minQty":"0.1"},{"limit":200,"filterType":"MAX_NUM_ORDERS"},{"limit":10,"filterType":"MAX_NUM_ALGO_ORDERS"},{"notional":"5","filterType":"MIN_NOTIONAL"},{"multiplierDown":"0.9000","multiplierUp":"1.1000","multiplierDecimal":"4","filterType":"PERCENT_PRICE"}],"orderTypes":["LIMIT","MARKET","STOP","STOP_MARKET","TAKE_PROFIT","TAKE_PROFIT_MARKET","TRAILING_STOP_MARKET"],"timeInForce":["GTC","IOC","FOK","GTX"]},{"symbol":"WAVESUSDT","pair":"WAVESUSDT","contractType":"PERPETUAL","deliveryDate":4133404800000,"onboardDate":1569398400000,"status":"TRADING","maintMarginPercent":"2.5000","requiredMarginPercent":"5.0000","baseAsset":"WAVES","quoteAsset":"USDT","marginAsset":"USDT","pricePrecision":4,"quantityPrecision":1,"baseAssetPrecision":8,"quotePrecision":8,"underlyingType":"COIN","underlyingSubType":["Layer-1"],"settlePlan":0,"triggerProtect":"0.1000","liquidationFee":"0.020000","marketTakeBound":"0.10","filters":[{"minPrice":"0.3420","maxPrice":"2249","filterType":"PRICE_FILTER","tickSize":"0.0001"},{"stepSize":"0.1","filterType":"LOT_SIZE","maxQty":"1000000","minQty":"0.1"},{"stepSize":"0.1","filterType":"MARKET_LOT_SIZE","maxQty":"16000","minQty":"0.1"},{"limit":200,"filterType":"MAX_NUM_ORDERS"},{"limit":10,"filterType":"MAX_NUM_ALGO_ORDERS"},{"notional":"5","filterType":"MIN_NOTIONAL"},{"multiplierDown":"0.9000","multiplierUp":"1.1000","multiplierDecimal":"4","filterType":"PERCENT_PRICE"}],"orderTypes":["LIMIT","MARKET","STOP","STOP_MARKET","TAKE_PROFIT","TAKE_PROFIT_MARKET","TRAILING_STOP_MARKET"],"timeInForce":["GTC","IOC","FOK","GTX"]},{"symbol":"MKRUSDT","pair":"MKRUSDT","contractType":"PERPETUAL","deliveryDate":4133404800000,"onboardDate":1569398400000,"status":"TRADING","maintMarginPercent":"2.5000","requiredMarginPercent":"5.0000","baseAsset":"MKR","quoteAsset":"USDT","marginAsset":"USDT","pricePrecision":2,"quantityPrecision":3,"baseAssetPrecision":8,"quotePrecision":8,"underlyingType":"COIN","underlyingSubType":["DeFi"],"settlePlan":0,"triggerProtect":"0.1000","liquidationFee":"0.012500","marketTakeBound":"0.10","filters":[{"minPrice":"50","maxPrice":"351788","filterType":"PRICE_FILTER","tickSize":"0.10"},{"stepSize":"0.001","filterType":"LOT_SIZE","maxQty":"10000","minQty":"0.001"},{"stepSize":"0.001","filterType":"MARKET_LOT_SIZE","maxQty":"100","minQty":"0.001"},{"limit":200,"filterType":"MAX_NUM_ORDERS"},{"limit":10,"filterType":"MAX_NUM_ALGO_ORDERS"},{"notional":"5","filterType":"MIN_NOTIONAL"},{"multiplierDown":"0.9000","multiplierUp":"1.1000","multiplierDecimal":"4","filterType":"PERCENT_PRICE"}],"orderTypes":["LIMIT","MARKET","STOP","STOP_MARKET","TAKE_PROFIT","TAKE_PROFIT_MARKET","TRAILING_STOP_MARKET"],"timeInForce":["GTC","IOC","FOK","GTX"]},{"symbol":"SNXUSDT","pair":"SNXUSDT","contractType":"PERPETUAL","deliveryDate":4133404800000,"onboardDate":1569398400000,"status":"TRADING","maintMarginPercent":"2.5000","requiredMarginPercent":"5.0000","baseAsset":"SNX","quoteAsset":"USDT","marginAsset":"USDT","pricePrecision":3,"quantityPrecision":1,"baseAssetPrecision":8,"quotePrecision":8,"underlyingType":"COIN","underlyingSubType":["DEX"],"settlePlan":0,"triggerProtect":"0.1000","liquidationFee":"0.010000","marketTakeBound":"0.10","filters":[{"minPrice":"0.164","maxPrice":"1144","filterType":"PRICE_FILTER","tickSize":"0.001"},{"stepSize":"0.1","filterType":"LOT_SIZE","maxQty":"1000000","minQty":"0.1"},{"stepSize":"0.1","filterType":"MARKET_LOT_SIZE","maxQty":"8000","minQty":"0.1"},{"limit":200,"filterType":"MAX_NUM_ORDERS"},{"limit":10,"filterType":"MAX_NUM_ALGO_ORDERS"},{"notional":"5","filterType":"MIN_NOTIONAL"},{"multiplierDown":"0.9000","multiplierUp":"1.1000","multiplierDecimal":"4","filterType":"PERCENT_PRICE"}],"orderTypes":["LIMIT","MARKET","STOP","STOP_MARKET","TAKE_PROFIT","TAKE_PROFIT_MARKET","TRAILING_STOP_MARKET"],"timeInForce":["GTC","IOC","FOK","GTX"]},{"symbol":"DOTUSDT","pair":"DOTUSDT","contractType":"PERPETUAL","deliveryDate":4133404800000,"onboardDate":1569398400000,"status":"TRADING","maintMarginPercent":"2.5000","requiredMarginPercent":"5.0000","baseAsset":"DOT","quoteAsset":"USDT","marginAsset":"USDT","pricePrecision":3,"quantityPrecision":1,"baseAssetPrecision":8,"quotePrecision":8,"underlyingType":"COIN","underlyingSubType":["Layer-1"],"settlePlan":0,"triggerProtect":"0.0500","liquidationFee":"0.015000","marketTakeBound":"0.05","filters":[{"minPrice":"0.373","maxPrice":"2398","filterType":"PRICE_FILTER","tickSize":"0.001"},{"stepSize":"0.1","filterType":"LOT_SIZE","maxQty":"1000000","minQty":"0.1"},{"stepSize":"0.1","filterType":"MARKET_LOT_SIZE","maxQty":"50000","minQty":"0.1"},{"limit":200,"filterType":"MAX_NUM_ORDERS"},{"limit":10,"filterType":"MAX_NUM_ALGO_ORDERS"},{"notional":"5","filterType":"MIN_NOTIONAL"},{"multiplierDown":"0.9500","multiplierUp":"1.0500","multiplierDecimal":"4","filterType":"PERCENT_PRICE"}],"orderTypes":["LIMIT","MARKET","STOP","STOP_MARKET","TAKE_PROFIT","TAKE_PROFIT_MARKET","TRAILING_STOP_MARKET"],"timeInForce":["GTC","IOC","FOK","GTX"]},{"symbol":"DEFIUSDT","pair":"DEFIUSDT","contractType":"PERPETUAL","deliveryDate":4133404800000,"onboardDate":1569398400000,"status":"TRADING","maintMarginPercent":"2.5000","requiredMarginPercent":"5.0000","baseAsset":"DEFI","quoteAsset":"USDT","marginAsset":"USDT","pricePrecision":1,"quantityPrecision":3,"baseAssetPrecision":8,"quotePrecision":8,"underlyingType":"INDEX","underlyingSubType":["Index"],"settlePlan":0,"triggerProtect":"0.1000","liquidationFee":"0.012500","marketTakeBound":"0.10","filters":[{"minPrice":"33.8","maxPrice":"230680","filterType":"PRICE_FILTER","tickSize":"0.1"},{"stepSize":"0.001","filterType":"LOT_SIZE","maxQty":"10000","minQty":"0.001"},{"stepSize":"0.001","filterType":"MARKET_LOT_SIZE","maxQty":"200","minQty":"0.001"},{"limit":200,"filterType":"MAX_NUM_ORDERS"},{"limit":10,"filterType":"MAX_NUM_ALGO_ORDERS"},{"notional":"5","filterType":"MIN_NOTIONAL"},{"multiplierDown":"0.9000","multiplierUp":"1.1000","multiplierDecimal":"4","filterType":"PERCENT_PRICE"}],"orderTypes":["LIMIT","MARKET","STOP","STOP_MARKET","TAKE_PROFIT","TAKE_PROFIT_MARKET","TRAILING_STOP_MARKET"],"timeInForce":["GTC","IOC","FOK","GTX"]},{"symbol":"YFIUSDT","pair":"YFIUSDT","contractType":"PERPETUAL","deliveryDate":4133404800000,"onboardDate":1569398400000,"status":"TRADING","maintMarginPercent":"2.5000","requiredMarginPercent":"5.0000","baseAsset":"YFI","quoteAsset":"USDT","marginAsset":"USDT","pricePrecision":1,"quantityPrecision":3,"baseAssetPrecision":8,"quotePrecision":8,"underlyingType":"COIN","underlyingSubType":["DeFi"],"settlePlan":0,"triggerProtect":"0.1000","liquidationFee":"0.015000","marketTakeBound":"0.10","filters":[{"minPrice":"667","maxPrice":"3836373","filterType":"PRICE_FILTER","tickSize":"1"},{"stepSize":"0.001","filterType":"LOT_SIZE","maxQty":"500","minQty":"0.001"},{"stepSize":"0.001","filterType":"MARKET_LOT_SIZE","maxQty":"20","minQty":"0.001"},{"limit":200,"filterType":"MAX_NUM_ORDERS"},{"limit":10,"filterType":"MAX_NUM_ALGO_ORDERS"},{"notional":"5","filterType":"MIN_NOTIONAL"},{"multiplierDown":"0.9000","multiplierUp":"1.1000","multiplierDecimal":"4","filterType":"PERCENT_PRICE"}],"orderTypes":["LIMIT","MARKET","STOP","STOP_MARKET","TAKE_PROFIT","TAKE_PROFIT_MARKET","TRAILING_STOP_MARKET"],"timeInForce":["GTC","IOC","FOK","GTX"]},{"symbol":"BALUSDT","pair":"BALUSDT","contractType":"PERPETUAL","deliveryDate":4133404800000,"onboardDate":1569398400000,"status":"TRADING","maintMarginPercent":"2.5000","requiredMarginPercent":"5.0000","baseAsset":"BAL","quoteAsset":"USDT","marginAsset":"USDT","pricePrecision":3,"quantityPrecision":1,"baseAssetPrecision":8,"quotePrecision":8,"underlyingType":"COIN","underlyingSubType":["DEX"],"settlePlan":0,"triggerProtect":"0.1000","liquidationFee":"0.020000","marketTakeBound":"0.10","filters":[{"minPrice":"0.630","maxPrice":"5000","filterType":"PRICE_FILTER","tickSize":"0.001"},{"stepSize":"0.1","filterType":"LOT_SIZE","maxQty":"1000000","minQty":"0.1"},{"stepSize":"0.1","filterType":"MARKET_LOT_SIZE","maxQty":"20000","minQty":"0.1"},{"limit":200,"filterType":"MAX_NUM_ORDERS"},{"limit":10,"filterType":"MAX_NUM_ALGO_ORDERS"},{"notional":"5","filterType":"MIN_NOTIONAL"},{"multiplierDown":"0.9000","multiplierUp":"1.1000","multiplierDecimal":"4","filterType":"PERCENT_PRICE"}],"orderTypes":["LIMIT","MARKET","STOP","STOP_MARKET","TAKE_PROFIT","TAKE_PROFIT_MARKET","TRAILING_STOP_MARKET"],"timeInForce":["GTC","IOC","FOK","GTX"]},{"symbol":"CRVUSDT","pair":"CRVUSDT","contractType":"PERPETUAL","deliveryDate":4133404800000,"onboardDate":1569398400000,"status":"TRADING","maintMarginPercent":"2.5000","requiredMarginPercent":"5.0000","baseAsset":"CRV","quoteAsset":"USDT","marginAsset":"USDT","pricePrecision":3,"quantityPrecision":1,"baseAssetPrecision":8,"quotePrecision":8,"underlyingType":"COIN","underlyingSubType":["DeFi"],"settlePlan":0,"triggerProtect":"0.1000","liquidationFee":"0.010000","marketTakeBound":"0.10","filters":[{"minPrice":"0.031","maxPrice":"1000","filterType":"PRICE_FILTER","tickSize":"0.001"},{"stepSize":"0.1","filterType":"LOT_SIZE","maxQty":"1000000","minQty":"0.1"},{"stepSize":"0.1","filterType":"MARKET_LOT_SIZE","maxQty":"300000","minQty":"0.1"},{"limit":200,"filterType":"MAX_NUM_ORDERS"},{"limit":10,"filterType":"MAX_NUM_ALGO_ORDERS"},{"notional":"5","filterType":"MIN_NOTIONAL"},{"multiplierDown":"0.9000","multiplierUp":"1.1000","multiplierDecimal":"4","filterType":"PERCENT_PRICE"}],"orderTypes":["LIMIT","MARKET","STOP","STOP_MARKET","TAKE_PROFIT","TAKE_PROFIT_MARKET","TRAILING_STOP_MARKET"],"timeInForce":["GTC","IOC","FOK","GTX"]},{"symbol":"TRBUSDT","pair":"TRBUSDT","contractType":"PERPETUAL","deliveryDate":4133404800000,"onboardDate":1569398400000,"status":"TRADING","maintMarginPercent":"2.5000","requiredMarginPercent":"5.0000","baseAsset":"TRB","quoteAsset":"USDT","marginAsset":"USDT","pricePrecision":3,"quantityPrecision":1,"baseAssetPrecision":8,"quotePrecision":8,"underlyingType":"COIN","underlyingSubType":["Oracle"],"settlePlan":0,"triggerProtect":"0.1000","liquidationFee":"0.010000","marketTakeBound":"0.10","filters":[{"minPrice":"1.230","maxPrice":"5138","filterType":"PRICE_FILTER","tickSize":"0.010"},{"stepSize":"0.1","filterType":"LOT_SIZE","maxQty":"1000000","minQty":"0.1"},{"stepSize":"0.1","filterType":"MARKET_LOT_SIZE","maxQty":"2000","minQty":"0.1"},{"limit":200,"filterType":"MAX_NUM_ORDERS"},{"limit":10,"filterType":"MAX_NUM_ALGO_ORDERS"},{"notional":"5","filterType":"MIN_NOTIONAL"},{"multiplierDown":"0.9000","multiplierUp":"1.1000","multiplierDecimal":"4","filterType":"PERCENT_PRICE"}],"orderTypes":["LIMIT","MARKET","STOP","STOP_MARKET","TAKE_PROFIT","TAKE_PROFIT_MARKET","TRAILING_STOP_MARKET"],"timeInForce":["GTC","IOC","FOK","GTX"]},{"symbol":"RUNEUSDT","pair":"RUNEUSDT","contractType":"PERPETUAL","deliveryDate":4133404800000,"onboardDate":1569398400000,"status":"TRADING","maintMarginPercent":"2.5000","requiredMarginPercent":"5.0000","baseAsset":"RUNE","quoteAsset":"USDT","marginAsset":"USDT","pricePrecision":4,"quantityPrecision":0,"baseAssetPrecision":8,"quotePrecision":8,"underlyingType":"COIN","underlyingSubType":["DEX"],"settlePlan":0,"triggerProtect":"0.1000","liquidationFee":"0.010000","marketTakeBound":"0.10","filters":[{"minPrice":"0.1720","maxPrice":"839","filterType":"PRICE_FILTER","tickSize":"0.0010"},{"stepSize":"1","filterType":"LOT_SIZE","maxQty":"10000000","minQty":"1"},{"stepSize":"1","filterType":"MARKET_LOT_SIZE","maxQty":"50000","minQty":"1"},{"limit":200,"filterType":"MAX_NUM_ORDERS"},{"limit":10,"filterType":"MAX_NUM_ALGO_ORDERS"},{"notional":"5","filterType":"MIN_NOTIONAL"},{"multiplierDown":"0.9000","multiplierUp":"1.1000","multiplierDecimal":"4","filterType":"PERCENT_PRICE"}],"orderTypes":["LIMIT","MARKET","STOP","STOP_MARKET","TAKE_PROFIT","TAKE_PROFIT_MARKET","TRAILING_STOP_MARKET"],"timeInForce":["GTC","IOC","FOK","GTX"]},{"symbol":"SUSHIUSDT","pair":"SUSHIUSDT","contractType":"PERPETUAL","deliveryDate":4133404800000,"onboardDate":1569398400000,"status":"TRADING","maintMarginPercent":"2.5000","requiredMarginPercent":"5.0000","baseAsset":"SUSHI","quoteAsset":"USDT","marginAsset":"USDT","pricePrecision":4,"quantityPrecision":0,"baseAssetPrecision":8,"quotePrecision":8,"underlyingType":"COIN","underlyingSubType":["DEX"],"settlePlan":0,"triggerProtect":"0.1000","liquidationFee":"0.010000","marketTakeBound":"0.10","filters":[{"minPrice":"0.1430","maxPrice":"1301","filterType":"PRICE_FILTER","tickSize":"0.0010"},{"stepSize":"1","filterType":"LOT_SIZE","maxQty":"10000000","minQty":"1"},{"stepSize":"1","filterType":"MARKET_LOT_SIZE","maxQty":"25000","minQty":"1"},{"limit":200,"filterType":"MAX_NUM_ORDERS"},{"limit":10,"filterType":"MAX_NUM_ALGO_ORDERS"},{"notional":"5","filterType":"MIN_NOTIONAL"},{"multiplierDown":"0.9000","multiplierUp":"1.1000","multiplierDecimal":"4","filterType":"PERCENT_PRICE"}],"orderTypes":["LIMIT","MARKET","STOP","STOP_MARKET","TAKE_PROFIT","TAKE_PROFIT_MARKET","TRAILING_STOP_MARKET"],"timeInForce":["GTC","IOC","FOK","GTX"]},{"symbol":"SRMUSDT","pair":"SRMUSDT","contractType":"PERPETUAL","deliveryDate":1668486600000,"onboardDate":1569398400000,"status":"SETTLING","maintMarginPercent":"2.5000","requiredMarginPercent":"5.0000","baseAsset":"SRM","quoteAsset":"USDT","marginAsset":"USDT","pricePrecision":4,"quantityPrecision":0,"baseAssetPrecision":8,"quotePrecision":8,"underlyingType":"COIN","underlyingSubType":["DEX"],"settlePlan":1668486600000,"triggerProtect":"0.1000","liquidationFee":"0.025000","marketTakeBound":"0.10","filters":[{"minPrice":"0.0920","maxPrice":"722","filterType":"PRICE_FILTER","tickSize":"0.0001"},{"stepSize":"1","filterType":"LOT_SIZE","maxQty":"10000000","minQty":"1"},{"stepSize":"1","filterType":"MARKET_LOT_SIZE","maxQty":"50000","minQty":"1"},{"limit":200,"filterType":"MAX_NUM_ORDERS"},{"limit":10,"filterType":"MAX_NUM_ALGO_ORDERS"},{"notional":"5","filterType":"MIN_NOTIONAL"},{"multiplierDown":"0.9000","multiplierUp":"1.1000","multiplierDecimal":"4","filterType":"PERCENT_PRICE"}],"orderTypes":["LIMIT","MARKET","STOP","STOP_MARKET","TAKE_PROFIT","TAKE_PROFIT_MARKET","TRAILING_STOP_MARKET"],"timeInForce":["GTC","IOC","FOK","GTX"]},{"symbol":"EGLDUSDT","pair":"EGLDUSDT","contractType":"PERPETUAL","deliveryDate":4133404800000,"onboardDate":1569398400000,"status":"TRADING","maintMarginPercent":"2.5000","requiredMarginPercent":"5.0000","baseAsset":"EGLD","quoteAsset":"USDT","marginAsset":"USDT","pricePrecision":3,"quantityPrecision":1,"baseAssetPrecision":8,"quotePrecision":8,"underlyingType":"COIN","underlyingSubType":["Layer-1"],"settlePlan":0,"triggerProtect":"0.1000","liquidationFee":"0.012500","marketTakeBound":"0.10","filters":[{"minPrice":"1.770","maxPrice":"14295","filterType":"PRICE_FILTER","tickSize":"0.010"},{"stepSize":"0.1","filterType":"LOT_SIZE","maxQty":"1000000","minQty":"0.1"},{"stepSize":"0.1","filterType":"MARKET_LOT_SIZE","maxQty":"2000","minQty":"0.1"},{"limit":200,"filterType":"MAX_NUM_ORDERS"},{"limit":10,"filterType":"MAX_NUM_ALGO_ORDERS"},{"notional":"5","filterType":"MIN_NOTIONAL"},{"multiplierDown":"0.9000","multiplierUp":"1.1000","multiplierDecimal":"4","filterType":"PERCENT_PRICE"}],"orderTypes":["LIMIT","MARKET","STOP","STOP_MARKET","TAKE_PROFIT","TAKE_PROFIT_MARKET","TRAILING_STOP_MARKET"],"timeInForce":["GTC","IOC","FOK","GTX"]},{"symbol":"SOLUSDT","pair":"SOLUSDT","contractType":"PERPETUAL","deliveryDate":4133404800000,"onboardDate":1569398400000,"status":"TRADING","maintMarginPercent":"2.5000","requiredMarginPercent":"5.0000","baseAsset":"SOL","quoteAsset":"USDT","marginAsset":"USDT","pricePrecision":4,"quantityPrecision":0,"baseAssetPrecision":8,"quotePrecision":8,"underlyingType":"COIN","underlyingSubType":["Layer-1"],"settlePlan":0,"triggerProtect":"0.0500","liquidationFee":"0.020000","marketTakeBound":"0.05","filters":[{"minPrice":"0.4200","maxPrice":"6857","filterType":"PRICE_FILTER","tickSize":"0.0010"},{"stepSize":"1","filterType":"LOT_SIZE","maxQty":"1000000","minQty":"1"},{"stepSize":"1","filterType":"MARKET_LOT_SIZE","maxQty":"1000","minQty":"1"},{"limit":200,"filterType":"MAX_NUM_ORDERS"},{"limit":10,"filterType":"MAX_NUM_ALGO_ORDERS"},{"notional":"5","filterType":"MIN_NOTIONAL"},{"multiplierDown":"0.9500","multiplierUp":"1.0500","multiplierDecimal":"4","filterType":"PERCENT_PRICE"}],"orderTypes":["LIMIT","MARKET","STOP","STOP_MARKET","TAKE_PROFIT","TAKE_PROFIT_MARKET","TRAILING_STOP_MARKET"],"timeInForce":["GTC","IOC","FOK","GTX"]},{"symbol":"ICXUSDT","pair":"ICXUSDT","contractType":"PERPETUAL","deliveryDate":4133404800000,"onboardDate":1569398400000,"status":"TRADING","maintMarginPercent":"2.5000","requiredMarginPercent":"5.0000","baseAsset":"ICX","quoteAsset":"USDT","marginAsset":"USDT","pricePrecision":4,"quantityPrecision":0,"baseAssetPrecision":8,"quotePrecision":8,"underlyingType":"COIN","underlyingSubType":["Layer-1"],"settlePlan":0,"triggerProtect":"0.1000","liquidationFee":"0.010000","marketTakeBound":"0.10","filters":[{"minPrice":"0.0234","maxPrice":"400","filterType":"PRICE_FILTER","tickSize":"0.0001"},{"stepSize":"1","filterType":"LOT_SIZE","maxQty":"10000000","minQty":"1"},{"stepSize":"1","filterType":"MARKET_LOT_SIZE","maxQty":"300000","minQty":"1"},{"limit":200,"filterType":"MAX_NUM_ORDERS"},{"limit":10,"filterType":"MAX_NUM_ALGO_ORDERS"},{"notional":"5","filterType":"MIN_NOTIONAL"},{"multiplierDown":"0.9000","multiplierUp":"1.1000","multiplierDecimal":"4","filterType":"PERCENT_PRICE"}],"orderTypes":["LIMIT","MARKET","STOP","STOP_MARKET","TAKE_PROFIT","TAKE_PROFIT_MARKET","TRAILING_STOP_MARKET"],"timeInForce":["GTC","IOC","FOK","GTX"]},{"symbol":"STORJUSDT","pair":"STORJUSDT","contractType":"PERPETUAL","deliveryDate":4133404800000,"onboardDate":1569398400000,"status":"TRADING","maintMarginPercent":"2.5000","requiredMarginPercent":"5.0000","baseAsset":"STORJ","quoteAsset":"USDT","marginAsset":"USDT","pricePrecision":4,"quantityPrecision":0,"baseAssetPrecision":8,"quotePrecision":8,"underlyingType":"COIN","underlyingSubType":["Storage"],"settlePlan":0,"triggerProtect":"0.1000","liquidationFee":"0.015000","marketTakeBound":"0.10","filters":[{"minPrice":"0.0190","maxPrice":"121","filterType":"PRICE_FILTER","tickSize":"0.0001"},{"stepSize":"1","filterType":"LOT_SIZE","maxQty":"10000000","minQty":"1"},{"stepSize":"1","filterType":"MARKET_LOT_SIZE","maxQty":"20000","minQty":"1"},{"limit":200,"filterType":"MAX_NUM_ORDERS"},{"limit":10,"filterType":"MAX_NUM_ALGO_ORDERS"},{"notional":"5","filterType":"MIN_NOTIONAL"},{"multiplierDown":"0.9000","multiplierUp":"1.1000","multiplierDecimal":"4","filterType":"PERCENT_PRICE"}],"orderTypes":["LIMIT","MARKET","STOP","STOP_MARKET","TAKE_PROFIT","TAKE_PROFIT_MARKET","TRAILING_STOP_MARKET"],"timeInForce":["GTC","IOC","FOK","GTX"]},{"symbol":"BLZUSDT","pair":"BLZUSDT","contractType":"PERPETUAL","deliveryDate":4133404800000,"onboardDate":1569398400000,"status":"TRADING","maintMarginPercent":"2.5000","requiredMarginPercent":"5.0000","baseAsset":"BLZ","quoteAsset":"USDT","marginAsset":"USDT","pricePrecision":5,"quantityPrecision":0,"baseAssetPrecision":8,"quotePrecision":8,"underlyingType":"COIN","underlyingSubType":["Storage"],"settlePlan":0,"triggerProtect":"0.1000","liquidationFee":"0.010000","marketTakeBound":"0.10","filters":[{"minPrice":"0.00360","maxPrice":"22","filterType":"PRICE_FILTER","tickSize":"0.00001"},{"stepSize":"1","filterType":"LOT_SIZE","maxQty":"10000000","minQty":"1"},{"stepSize":"1","filterType":"MARKET_LOT_SIZE","maxQty":"200000","minQty":"1"},{"limit":200,"filterType":"MAX_NUM_ORDERS"},{"limit":10,"filterType":"MAX_NUM_ALGO_ORDERS"},{"notional":"5","filterType":"MIN_NOTIONAL"},{"multiplierDown":"0.9000","multiplierUp":"1.1000","multiplierDecimal":"4","filterType":"PERCENT_PRICE"}],"orderTypes":["LIMIT","MARKET","STOP","STOP_MARKET","TAKE_PROFIT","TAKE_PROFIT_MARKET","TRAILING_STOP_MARKET"],"timeInForce":["GTC","IOC","FOK","GTX"]},{"symbol":"UNIUSDT","pair":"UNIUSDT","contractType":"PERPETUAL","deliveryDate":4133404800000,"onboardDate":1569398400000,"status":"TRADING","maintMarginPercent":"2.5000","requiredMarginPercent":"5.0000","baseAsset":"UNI","quoteAsset":"USDT","marginAsset":"USDT","pricePrecision":4,"quantityPrecision":0,"baseAssetPrecision":8,"quotePrecision":8,"underlyingType":"COIN","underlyingSubType":["DEX"],"settlePlan":0,"triggerProtect":"0.1000","liquidationFee":"0.015000","marketTakeBound":"0.10","filters":[{"minPrice":"0.3730","maxPrice":"2684","filterType":"PRICE_FILTER","tickSize":"0.0010"},{"stepSize":"1","filterType":"LOT_SIZE","maxQty":"10000000","minQty":"1"},{"stepSize":"1","filterType":"MARKET_LOT_SIZE","maxQty":"25000","minQty":"1"},{"limit":200,"filterType":"MAX_NUM_ORDERS"},{"limit":10,"filterType":"MAX_NUM_ALGO_ORDERS"},{"notional":"5","filterType":"MIN_NOTIONAL"},{"multiplierDown":"0.9000","multiplierUp":"1.1000","multiplierDecimal":"4","filterType":"PERCENT_PRICE"}],"orderTypes":["LIMIT","MARKET","STOP","STOP_MARKET","TAKE_PROFIT","TAKE_PROFIT_MARKET","TRAILING_STOP_MARKET"],"timeInForce":["GTC","IOC","FOK","GTX"]},{"symbol":"AVAXUSDT","pair":"AVAXUSDT","contractType":"PERPETUAL","deliveryDate":4133404800000,"onboardDate":1569398400000,"status":"TRADING","maintMarginPercent":"2.5000","requiredMarginPercent":"5.0000","baseAsset":"AVAX","quoteAsset":"USDT","marginAsset":"USDT","pricePrecision":4,"quantityPrecision":0,"baseAssetPrecision":8,"quotePrecision":8,"underlyingType":"COIN","underlyingSubType":["Layer-1"],"settlePlan":0,"triggerProtect":"0.1000","liquidationFee":"0.012500","marketTakeBound":"0.10","filters":[{"minPrice":"0.3500","maxPrice":"2403","filterType":"PRICE_FILTER","tickSize":"0.0010"},{"stepSize":"1","filterType":"LOT_SIZE","maxQty":"100000","minQty":"1"},{"stepSize":"1","filterType":"MARKET_LOT_SIZE","maxQty":"5000","minQty":"1"},{"limit":200,"filterType":"MAX_NUM_ORDERS"},{"limit":10,"filterType":"MAX_NUM_ALGO_ORDERS"},{"notional":"5","filterType":"MIN_NOTIONAL"},{"multiplierDown":"0.9000","multiplierUp":"1.1000","multiplierDecimal":"4","filterType":"PERCENT_PRICE"}],"orderTypes":["LIMIT","MARKET","STOP","STOP_MARKET","TAKE_PROFIT","TAKE_PROFIT_MARKET","TRAILING_STOP_MARKET"],"timeInForce":["GTC","IOC","FOK","GTX"]},{"symbol":"FTMUSDT","pair":"FTMUSDT","contractType":"PERPETUAL","deliveryDate":4133404800000,"onboardDate":1569398400000,"status":"TRADING","maintMarginPercent":"2.5000","requiredMarginPercent":"5.0000","baseAsset":"FTM","quoteAsset":"USDT","marginAsset":"USDT","pricePrecision":6,"quantityPrecision":0,"baseAssetPrecision":8,"quotePrecision":8,"underlyingType":"COIN","underlyingSubType":["Layer-1"],"settlePlan":0,"triggerProtect":"0.1000","liquidationFee":"0.010000","marketTakeBound":"0.10","filters":[{"minPrice":"0.007700","maxPrice":"41","filterType":"PRICE_FILTER","tickSize":"0.000100"},{"stepSize":"1","filterType":"LOT_SIZE","maxQty":"10000000","minQty":"1"},{"stepSize":"1","filterType":"MARKET_LOT_SIZE","maxQty":"250000","minQty":"1"},{"limit":200,"filterType":"MAX_NUM_ORDERS"},{"limit":10,"filterType":"MAX_NUM_ALGO_ORDERS"},{"notional":"5","filterType":"MIN_NOTIONAL"},{"multiplierDown":"0.9000","multiplierUp":"1.1000","multiplierDecimal":"4","filterType":"PERCENT_PRICE"}],"orderTypes":["LIMIT","MARKET","STOP","STOP_MARKET","TAKE_PROFIT","TAKE_PROFIT_MARKET","TRAILING_STOP_MARKET"],"timeInForce":["GTC","IOC","FOK","GTX"]},{"symbol":"HNTUSDT","pair":"HNTUSDT","contractType":"PERPETUAL","deliveryDate":4133404800000,"onboardDate":1569398400000,"status":"TRADING","maintMarginPercent":"2.5000","requiredMarginPercent":"5.0000","baseAsset":"HNT","quoteAsset":"USDT","marginAsset":"USDT","pricePrecision":4,"quantityPrecision":0,"baseAssetPrecision":8,"quotePrecision":8,"underlyingType":"COIN","underlyingSubType":["Infrastructure"],"settlePlan":0,"triggerProtect":"0.1000","liquidationFee":"0.020000","marketTakeBound":"0.10","filters":[{"minPrice":"0.1660","maxPrice":"1929","filterType":"PRICE_FILTER","tickSize":"0.0010"},{"stepSize":"1","filterType":"LOT_SIZE","maxQty":"10000000","minQty":"1"},{"stepSize":"1","filterType":"MARKET_LOT_SIZE","maxQty":"20000","minQty":"1"},{"limit":200,"filterType":"MAX_NUM_ORDERS"},{"limit":10,"filterType":"MAX_NUM_ALGO_ORDERS"},{"notional":"5","filterType":"MIN_NOTIONAL"},{"multiplierDown":"0.9000","multiplierUp":"1.1000","multiplierDecimal":"4","filterType":"PERCENT_PRICE"}],"orderTypes":["LIMIT","MARKET","STOP","STOP_MARKET","TAKE_PROFIT","TAKE_PROFIT_MARKET","TRAILING_STOP_MARKET"],"timeInForce":["GTC","IOC","FOK","GTX"]},{"symbol":"ENJUSDT","pair":"ENJUSDT","contractType":"PERPETUAL","deliveryDate":4133404800000,"onboardDate":1569398400000,"status":"TRADING","maintMarginPercent":"2.5000","requiredMarginPercent":"5.0000","baseAsset":"ENJ","quoteAsset":"USDT","marginAsset":"USDT","pricePrecision":5,"quantityPrecision":0,"baseAssetPrecision":8,"quotePrecision":8,"underlyingType":"COIN","underlyingSubType":["NFT"],"settlePlan":0,"triggerProtect":"0.1000","liquidationFee":"0.010000","marketTakeBound":"0.10","filters":[{"minPrice":"0.02250","maxPrice":"153","filterType":"PRICE_FILTER","tickSize":"0.00010"},{"stepSize":"1","filterType":"LOT_SIZE","maxQty":"10000000","minQty":"1"},{"stepSize":"1","filterType":"MARKET_LOT_SIZE","maxQty":"100000","minQty":"1"},{"limit":200,"filterType":"MAX_NUM_ORDERS"},{"limit":10,"filterType":"MAX_NUM_ALGO_ORDERS"},{"notional":"5","filterType":"MIN_NOTIONAL"},{"multiplierDown":"0.9000","multiplierUp":"1.1000","multiplierDecimal":"4","filterType":"PERCENT_PRICE"}],"orderTypes":["LIMIT","MARKET","STOP","STOP_MARKET","TAKE_PROFIT","TAKE_PROFIT_MARKET","TRAILING_STOP_MARKET"],"timeInForce":["GTC","IOC","FOK","GTX"]},{"symbol":"FLMUSDT","pair":"FLMUSDT","contractType":"PERPETUAL","deliveryDate":4133404800000,"onboardDate":1569398400000,"status":"TRADING","maintMarginPercent":"2.5000","requiredMarginPercent":"5.0000","baseAsset":"FLM","quoteAsset":"USDT","marginAsset":"USDT","pricePrecision":4,"quantityPrecision":0,"baseAssetPrecision":8,"quotePrecision":8,"underlyingType":"COIN","underlyingSubType":["DeFi"],"settlePlan":0,"triggerProtect":"0.1000","liquidationFee":"0.010000","marketTakeBound":"0.10","filters":[{"minPrice":"0.0096","maxPrice":"1000","filterType":"PRICE_FILTER","tickSize":"0.0001"},{"stepSize":"1","filterType":"LOT_SIZE","maxQty":"1000000","minQty":"1"},{"stepSize":"1","filterType":"MARKET_LOT_SIZE","maxQty":"100000","minQty":"1"},{"limit":200,"filterType":"MAX_NUM_ORDERS"},{"limit":10,"filterType":"MAX_NUM_ALGO_ORDERS"},{"notional":"5","filterType":"MIN_NOTIONAL"},{"multiplierDown":"0.9000","multiplierUp":"1.1000","multiplierDecimal":"4","filterType":"PERCENT_PRICE"}],"orderTypes":["LIMIT","MARKET","STOP","STOP_MARKET","TAKE_PROFIT","TAKE_PROFIT_MARKET","TRAILING_STOP_MARKET"],"timeInForce":["GTC","IOC","FOK","GTX"]},{"symbol":"TOMOUSDT","pair":"TOMOUSDT","contractType":"PERPETUAL","deliveryDate":4133404800000,"onboardDate":1569398400000,"status":"TRADING","maintMarginPercent":"2.5000","requiredMarginPercent":"5.0000","baseAsset":"TOMO","quoteAsset":"USDT","marginAsset":"USDT","pricePrecision":4,"quantityPrecision":0,"baseAssetPrecision":8,"quotePrecision":8,"underlyingType":"COIN","underlyingSubType":["Layer-1"],"settlePlan":0,"triggerProtect":"0.1000","liquidationFee":"0.015000","marketTakeBound":"0.10","filters":[{"minPrice":"0.0250","maxPrice":"1000","filterType":"PRICE_FILTER","tickSize":"0.0001"},{"stepSize":"1","filterType":"LOT_SIZE","maxQty":"10000000","minQty":"1"},{"stepSize":"1","filterType":"MARKET_LOT_SIZE","maxQty":"500000","minQty":"1"},{"limit":200,"filterType":"MAX_NUM_ORDERS"},{"limit":10,"filterType":"MAX_NUM_ALGO_ORDERS"},{"notional":"5","filterType":"MIN_NOTIONAL"},{"multiplierDown":"0.9000","multiplierUp":"1.1000","multiplierDecimal":"4","filterType":"PERCENT_PRICE"}],"orderTypes":["LIMIT","MARKET","STOP","STOP_MARKET","TAKE_PROFIT","TAKE_PROFIT_MARKET","TRAILING_STOP_MARKET"],"timeInForce":["GTC","IOC","FOK","GTX"]},{"symbol":"RENUSDT","pair":"RENUSDT","contractType":"PERPETUAL","deliveryDate":4133404800000,"onboardDate":1569398400000,"status":"TRADING","maintMarginPercent":"2.5000","requiredMarginPercent":"5.0000","baseAsset":"REN","quoteAsset":"USDT","marginAsset":"USDT","pricePrecision":5,"quantityPrecision":0,"baseAssetPrecision":8,"quotePrecision":8,"underlyingType":"COIN","underlyingSubType":["DeFi"],"settlePlan":0,"triggerProtect":"0.1000","liquidationFee":"0.010000","marketTakeBound":"0.10","filters":[{"minPrice":"0.00880","maxPrice":"100","filterType":"PRICE_FILTER","tickSize":"0.00001"},{"stepSize":"1","filterType":"LOT_SIZE","maxQty":"10000000","minQty":"1"},{"stepSize":"1","filterType":"MARKET_LOT_SIZE","maxQty":"1000000","minQty":"1"},{"limit":200,"filterType":"MAX_NUM_ORDERS"},{"limit":10,"filterType":"MAX_NUM_ALGO_ORDERS"},{"notional":"5","filterType":"MIN_NOTIONAL"},{"multiplierDown":"0.9000","multiplierUp":"1.1000","multiplierDecimal":"4","filterType":"PERCENT_PRICE"}],"orderTypes":["LIMIT","MARKET","STOP","STOP_MARKET","TAKE_PROFIT","TAKE_PROFIT_MARKET","TRAILING_STOP_MARKET"],"timeInForce":["GTC","IOC","FOK","GTX"]},{"symbol":"KSMUSDT","pair":"KSMUSDT","contractType":"PERPETUAL","deliveryDate":4133404800000,"onboardDate":1569398400000,"status":"TRADING","maintMarginPercent":"2.5000","requiredMarginPercent":"5.0000","baseAsset":"KSM","quoteAsset":"USDT","marginAsset":"USDT","pricePrecision":3,"quantityPrecision":1,"baseAssetPrecision":8,"quotePrecision":8,"underlyingType":"COIN","underlyingSubType":["Layer-1"],"settlePlan":0,"triggerProtect":"0.1000","liquidationFee":"0.012500","marketTakeBound":"0.10","filters":[{"minPrice":"4.060","maxPrice":"28450","filterType":"PRICE_FILTER","tickSize":"0.010"},{"stepSize":"0.1","filterType":"LOT_SIZE","maxQty":"1000000","minQty":"0.1"},{"stepSize":"0.1","filterType":"MARKET_LOT_SIZE","maxQty":"1000","minQty":"0.1"},{"limit":200,"filterType":"MAX_NUM_ORDERS"},{"limit":10,"filterType":"MAX_NUM_ALGO_ORDERS"},{"notional":"5","filterType":"MIN_NOTIONAL"},{"multiplierDown":"0.9000","multiplierUp":"1.1000","multiplierDecimal":"4","filterType":"PERCENT_PRICE"}],"orderTypes":["LIMIT","MARKET","STOP","STOP_MARKET","TAKE_PROFIT","TAKE_PROFIT_MARKET","TRAILING_STOP_MARKET"],"timeInForce":["GTC","IOC","FOK","GTX"]},{"symbol":"NEARUSDT","pair":"NEARUSDT","contractType":"PERPETUAL","deliveryDate":4133404800000,"onboardDate":1569398400000,"status":"TRADING","maintMarginPercent":"2.5000","requiredMarginPercent":"5.0000","baseAsset":"NEAR","quoteAsset":"USDT","marginAsset":"USDT","pricePrecision":4,"quantityPrecision":0,"baseAssetPrecision":8,"quotePrecision":8,"underlyingType":"COIN","underlyingSubType":["Layer-2"],"settlePlan":0,"triggerProtect":"0.1000","liquidationFee":"0.010000","marketTakeBound":"0.10","filters":[{"minPrice":"0.0480","maxPrice":"1000","filterType":"PRICE_FILTER","tickSize":"0.0010"},{"stepSize":"1","filterType":"LOT_SIZE","maxQty":"10000000","minQty":"1"},{"stepSize":"1","filterType":"MARKET_LOT_SIZE","maxQty":"24000","minQty":"1"},{"limit":200,"filterType":"MAX_NUM_ORDERS"},{"limit":10,"filterType":"MAX_NUM_ALGO_ORDERS"},{"notional":"5","filterType":"MIN_NOTIONAL"},{"multiplierDown":"0.9000","multiplierUp":"1.1000","multiplierDecimal":"4","filterType":"PERCENT_PRICE"}],"orderTypes":["LIMIT","MARKET","STOP","STOP_MARKET","TAKE_PROFIT","TAKE_PROFIT_MARKET","TRAILING_STOP_MARKET"],"timeInForce":["GTC","IOC","FOK","GTX"]},{"symbol":"AAVEUSDT","pair":"AAVEUSDT","contractType":"PERPETUAL","deliveryDate":4133404800000,"onboardDate":1569398400000,"status":"TRADING","maintMarginPercent":"2.5000","requiredMarginPercent":"5.0000","baseAsset":"AAVE","quoteAsset":"USDT","marginAsset":"USDT","pricePrecision":3,"quantityPrecision":1,"baseAssetPrecision":8,"quotePrecision":8,"underlyingType":"COIN","underlyingSubType":["DeFi"],"settlePlan":0,"triggerProtect":"0.1000","liquidationFee":"0.012500","marketTakeBound":"0.10","filters":[{"minPrice":"4.340","maxPrice":"38702","filterType":"PRICE_FILTER","tickSize":"0.010"},{"stepSize":"0.1","filterType":"LOT_SIZE","maxQty":"1000000","minQty":"0.1"},{"stepSize":"0.1","filterType":"MARKET_LOT_SIZE","maxQty":"1250","minQty":"0.1"},{"limit":200,"filterType":"MAX_NUM_ORDERS"},{"limit":10,"filterType":"MAX_NUM_ALGO_ORDERS"},{"notional":"5","filterType":"MIN_NOTIONAL"},{"multiplierDown":"0.9000","multiplierUp":"1.1000","multiplierDecimal":"4","filterType":"PERCENT_PRICE"}],"orderTypes":["LIMIT","MARKET","STOP","STOP_MARKET","TAKE_PROFIT","TAKE_PROFIT_MARKET","TRAILING_STOP_MARKET"],"timeInForce":["GTC","IOC","FOK","GTX"]},{"symbol":"FILUSDT","pair":"FILUSDT","contractType":"PERPETUAL","deliveryDate":4133404800000,"onboardDate":1569398400000,"status":"TRADING","maintMarginPercent":"2.5000","requiredMarginPercent":"5.0000","baseAsset":"FIL","quoteAsset":"USDT","marginAsset":"USDT","pricePrecision":3,"quantityPrecision":1,"baseAssetPrecision":8,"quotePrecision":8,"underlyingType":"COIN","underlyingSubType":["Storage"],"settlePlan":0,"triggerProtect":"0.1000","liquidationFee":"0.015000","marketTakeBound":"0.10","filters":[{"minPrice":"1.381","maxPrice":"10000","filterType":"PRICE_FILTER","tickSize":"0.001"},{"stepSize":"0.1","filterType":"LOT_SIZE","maxQty":"10000000","minQty":"0.1"},{"stepSize":"0.1","filterType":"MARKET_LOT_SIZE","maxQty":"7500","minQty":"0.1"},{"limit":200,"filterType":"MAX_NUM_ORDERS"},{"limit":10,"filterType":"MAX_NUM_ALGO_ORDERS"},{"notional":"5","filterType":"MIN_NOTIONAL"},{"multiplierDown":"0.9000","multiplierUp":"1.1000","multiplierDecimal":"4","filterType":"PERCENT_PRICE"}],"orderTypes":["LIMIT","MARKET","STOP","STOP_MARKET","TAKE_PROFIT","TAKE_PROFIT_MARKET","TRAILING_STOP_MARKET"],"timeInForce":["GTC","IOC","FOK","GTX"]},{"symbol":"RSRUSDT","pair":"RSRUSDT","contractType":"PERPETUAL","deliveryDate":4133404800000,"onboardDate":1569398400000,"status":"TRADING","maintMarginPercent":"2.5000","requiredMarginPercent":"5.0000","baseAsset":"RSR","quoteAsset":"USDT","marginAsset":"USDT","pricePrecision":6,"quantityPrecision":0,"baseAssetPrecision":8,"quotePrecision":8,"underlyingType":"COIN","underlyingSubType":["DeFi"],"settlePlan":0,"triggerProtect":"0.1000","liquidationFee":"0.010000","marketTakeBound":"0.10","filters":[{"minPrice":"0.000778","maxPrice":"10","filterType":"PRICE_FILTER","tickSize":"0.000001"},{"stepSize":"1","filterType":"LOT_SIZE","maxQty":"10000000","minQty":"1"},{"stepSize":"1","filterType":"MARKET_LOT_SIZE","maxQty":"10000000","minQty":"1"},{"limit":200,"filterType":"MAX_NUM_ORDERS"},{"limit":10,"filterType":"MAX_NUM_ALGO_ORDERS"},{"notional":"5","filterType":"MIN_NOTIONAL"},{"multiplierDown":"0.9000","multiplierUp":"1.1000","multiplierDecimal":"4","filterType":"PERCENT_PRICE"}],"orderTypes":["LIMIT","MARKET","STOP","STOP_MARKET","TAKE_PROFIT","TAKE_PROFIT_MARKET","TRAILING_STOP_MARKET"],"timeInForce":["GTC","IOC","FOK","GTX"]},{"symbol":"LRCUSDT","pair":"LRCUSDT","contractType":"PERPETUAL","deliveryDate":4133404800000,"onboardDate":1569398400000,"status":"TRADING","maintMarginPercent":"2.5000","requiredMarginPercent":"5.0000","baseAsset":"LRC","quoteAsset":"USDT","marginAsset":"USDT","pricePrecision":5,"quantityPrecision":0,"baseAssetPrecision":8,"quotePrecision":8,"underlyingType":"COIN","underlyingSubType":["DEX"],"settlePlan":0,"triggerProtect":"0.1000","liquidationFee":"0.010000","marketTakeBound":"0.10","filters":[{"minPrice":"0.00520","maxPrice":"100","filterType":"PRICE_FILTER","tickSize":"0.00010"},{"stepSize":"1","filterType":"LOT_SIZE","maxQty":"10000000","minQty":"1"},{"stepSize":"1","filterType":"MARKET_LOT_SIZE","maxQty":"500000","minQty":"1"},{"limit":200,"filterType":"MAX_NUM_ORDERS"},{"limit":10,"filterType":"MAX_NUM_ALGO_ORDERS"},{"notional":"5","filterType":"MIN_NOTIONAL"},{"multiplierDown":"0.9000","multiplierUp":"1.1000","multiplierDecimal":"4","filterType":"PERCENT_PRICE"}],"orderTypes":["LIMIT","MARKET","STOP","STOP_MARKET","TAKE_PROFIT","TAKE_PROFIT_MARKET","TRAILING_STOP_MARKET"],"timeInForce":["GTC","IOC","FOK","GTX"]},{"symbol":"MATICUSDT","pair":"MATICUSDT","contractType":"PERPETUAL","deliveryDate":4133404800000,"onboardDate":1569398400000,"status":"TRADING","maintMarginPercent":"2.5000","requiredMarginPercent":"5.0000","baseAsset":"MATIC","quoteAsset":"USDT","marginAsset":"USDT","pricePrecision":5,"quantityPrecision":0,"baseAssetPrecision":8,"quotePrecision":8,"underlyingType":"COIN","underlyingSubType":["Layer-2"],"settlePlan":0,"triggerProtect":"0.1000","liquidationFee":"0.010000","marketTakeBound":"0.10","filters":[{"minPrice":"0.00960","maxPrice":"137","filterType":"PRICE_FILTER","tickSize":"0.00010"},{"stepSize":"1","filterType":"LOT_SIZE","maxQty":"10000000","minQty":"1"},{"stepSize":"1","filterType":"MARKET_LOT_SIZE","maxQty":"500000","minQty":"1"},{"limit":200,"filterType":"MAX_NUM_ORDERS"},{"limit":10,"filterType":"MAX_NUM_ALGO_ORDERS"},{"notional":"5","filterType":"MIN_NOTIONAL"},{"multiplierDown":"0.9000","multiplierUp":"1.1000","multiplierDecimal":"4","filterType":"PERCENT_PRICE"}],"orderTypes":["LIMIT","MARKET","STOP","STOP_MARKET","TAKE_PROFIT","TAKE_PROFIT_MARKET","TRAILING_STOP_MARKET"],"timeInForce":["GTC","IOC","FOK","GTX"]},{"symbol":"OCEANUSDT","pair":"OCEANUSDT","contractType":"PERPETUAL","deliveryDate":4133404800000,"onboardDate":1569398400000,"status":"TRADING","maintMarginPercent":"2.5000","requiredMarginPercent":"5.0000","baseAsset":"OCEAN","quoteAsset":"USDT","marginAsset":"USDT","pricePrecision":5,"quantityPrecision":0,"baseAssetPrecision":8,"quotePrecision":8,"underlyingType":"COIN","underlyingSubType":["Storage"],"settlePlan":0,"triggerProtect":"0.1000","liquidationFee":"0.015000","marketTakeBound":"0.10","filters":[{"minPrice":"0.01379","maxPrice":"100","filterType":"PRICE_FILTER","tickSize":"0.00001"},{"stepSize":"1","filterType":"LOT_SIZE","maxQty":"10000000","minQty":"1"},{"stepSize":"1","filterType":"MARKET_LOT_SIZE","maxQty":"1000000","minQty":"1"},{"limit":200,"filterType":"MAX_NUM_ORDERS"},{"limit":10,"filterType":"MAX_NUM_ALGO_ORDERS"},{"notional":"5","filterType":"MIN_NOTIONAL"},{"multiplierDown":"0.9000","multiplierUp":"1.1000","multiplierDecimal":"4","filterType":"PERCENT_PRICE"}],"orderTypes":["LIMIT","MARKET","STOP","STOP_MARKET","TAKE_PROFIT","TAKE_PROFIT_MARKET","TRAILING_STOP_MARKET"],"timeInForce":["GTC","IOC","FOK","GTX"]},{"symbol":"CVCUSDT","pair":"CVCUSDT","contractType":"PERPETUAL","deliveryDate":1669712400000,"onboardDate":1569398400000,"status":"SETTLING","maintMarginPercent":"2.5000","requiredMarginPercent":"5.0000","baseAsset":"CVC","quoteAsset":"USDT","marginAsset":"USDT","pricePrecision":5,"quantityPrecision":0,"baseAssetPrecision":8,"quotePrecision":8,"underlyingType":"COIN","underlyingSubType":["Infrastructure"],"settlePlan":1669712400000,"triggerProtect":"0.1000","liquidationFee":"0.025000","marketTakeBound":"0.10","filters":[{"minPrice":"0.00517","maxPrice":"100","filterType":"PRICE_FILTER","tickSize":"0.00001"},{"stepSize":"1","filterType":"LOT_SIZE","maxQty":"10000000","minQty":"1"},{"stepSize":"1","filterType":"MARKET_LOT_SIZE","maxQty":"800000","minQty":"1"},{"limit":200,"filterType":"MAX_NUM_ORDERS"},{"limit":10,"filterType":"MAX_NUM_ALGO_ORDERS"},{"notional":"5","filterType":"MIN_NOTIONAL"},{"multiplierDown":"0.9000","multiplierUp":"1.1000","multiplierDecimal":"4","filterType":"PERCENT_PRICE"}],"orderTypes":["LIMIT","MARKET","STOP","STOP_MARKET","TAKE_PROFIT","TAKE_PROFIT_MARKET","TRAILING_STOP_MARKET"],"timeInForce":["GTC","IOC","FOK","GTX"]},{"symbol":"BELUSDT","pair":"BELUSDT","contractType":"PERPETUAL","deliveryDate":4133404800000,"onboardDate":1569398400000,"status":"TRADING","maintMarginPercent":"2.5000","requiredMarginPercent":"5.0000","baseAsset":"BEL","quoteAsset":"USDT","marginAsset":"USDT","pricePrecision":5,"quantityPrecision":0,"baseAssetPrecision":8,"quotePrecision":8,"underlyingType":"COIN","underlyingSubType":["DeFi"],"settlePlan":0,"triggerProtect":"0.1000","liquidationFee":"0.015000","marketTakeBound":"0.10","filters":[{"minPrice":"0.03610","maxPrice":"204","filterType":"PRICE_FILTER","tickSize":"0.00010"},{"stepSize":"1","filterType":"LOT_SIZE","maxQty":"10000000","minQty":"1"},{"stepSize":"1","filterType":"MARKET_LOT_SIZE","maxQty":"150000","minQty":"1"},{"limit":200,"filterType":"MAX_NUM_ORDERS"},{"limit":10,"filterType":"MAX_NUM_ALGO_ORDERS"},{"notional":"5","filterType":"MIN_NOTIONAL"},{"multiplierDown":"0.9000","multiplierUp":"1.1000","multiplierDecimal":"4","filterType":"PERCENT_PRICE"}],"orderTypes":["LIMIT","MARKET","STOP","STOP_MARKET","TAKE_PROFIT","TAKE_PROFIT_MARKET","TRAILING_STOP_MARKET"],"timeInForce":["GTC","IOC","FOK","GTX"]},{"symbol":"CTKUSDT","pair":"CTKUSDT","contractType":"PERPETUAL","deliveryDate":4133404800000,"onboardDate":1569398400000,"status":"TRADING","maintMarginPercent":"2.5000","requiredMarginPercent":"5.0000","baseAsset":"CTK","quoteAsset":"USDT","marginAsset":"USDT","pricePrecision":5,"quantityPrecision":0,"baseAssetPrecision":8,"quotePrecision":8,"underlyingType":"COIN","underlyingSubType":["Infrastructure"],"settlePlan":0,"triggerProtect":"0.1000","liquidationFee":"0.015000","marketTakeBound":"0.10","filters":[{"minPrice":"0.02600","maxPrice":"180","filterType":"PRICE_FILTER","tickSize":"0.00010"},{"stepSize":"1","filterType":"LOT_SIZE","maxQty":"10000000","minQty":"1"},{"stepSize":"1","filterType":"MARKET_LOT_SIZE","maxQty":"800000","minQty":"1"},{"limit":200,"filterType":"MAX_NUM_ORDERS"},{"limit":10,"filterType":"MAX_NUM_ALGO_ORDERS"},{"notional":"5","filterType":"MIN_NOTIONAL"},{"multiplierDown":"0.9000","multiplierUp":"1.1000","multiplierDecimal":"4","filterType":"PERCENT_PRICE"}],"orderTypes":["LIMIT","MARKET","STOP","STOP_MARKET","TAKE_PROFIT","TAKE_PROFIT_MARKET","TRAILING_STOP_MARKET"],"timeInForce":["GTC","IOC","FOK","GTX"]},{"symbol":"AXSUSDT","pair":"AXSUSDT","contractType":"PERPETUAL","deliveryDate":4133404800000,"onboardDate":1569398400000,"status":"TRADING","maintMarginPercent":"2.5000","requiredMarginPercent":"5.0000","baseAsset":"AXS","quoteAsset":"USDT","marginAsset":"USDT","pricePrecision":5,"quantityPrecision":0,"baseAssetPrecision":8,"quotePrecision":8,"underlyingType":"COIN","underlyingSubType":["Metaverse"],"settlePlan":0,"triggerProtect":"0.1000","liquidationFee":"0.020000","marketTakeBound":"0.10","filters":[{"minPrice":"0.08000","maxPrice":"6989","filterType":"PRICE_FILTER","tickSize":"0.00100"},{"stepSize":"1","filterType":"LOT_SIZE","maxQty":"10000000","minQty":"1"},{"stepSize":"1","filterType":"MARKET_LOT_SIZE","maxQty":"8000","minQty":"1"},{"limit":200,"filterType":"MAX_NUM_ORDERS"},{"limit":10,"filterType":"MAX_NUM_ALGO_ORDERS"},{"notional":"5","filterType":"MIN_NOTIONAL"},{"multiplierDown":"0.9000","multiplierUp":"1.1000","multiplierDecimal":"4","filterType":"PERCENT_PRICE"}],"orderTypes":["LIMIT","MARKET","STOP","STOP_MARKET","TAKE_PROFIT","TAKE_PROFIT_MARKET","TRAILING_STOP_MARKET"],"timeInForce":["GTC","IOC","FOK","GTX"]},{"symbol":"ALPHAUSDT","pair":"ALPHAUSDT","contractType":"PERPETUAL","deliveryDate":4133404800000,"onboardDate":1569398400000,"status":"TRADING","maintMarginPercent":"2.5000","requiredMarginPercent":"5.0000","baseAsset":"ALPHA","quoteAsset":"USDT","marginAsset":"USDT","pricePrecision":5,"quantityPrecision":0,"baseAssetPrecision":8,"quotePrecision":8,"underlyingType":"COIN","underlyingSubType":["DeFi"],"settlePlan":0,"triggerProtect":"0.1000","liquidationFee":"0.015000","marketTakeBound":"0.10","filters":[{"minPrice":"0.01760","maxPrice":"100","filterType":"PRICE_FILTER","tickSize":"0.00001"},{"stepSize":"1","filterType":"LOT_SIZE","maxQty":"5000000","minQty":"1"},{"stepSize":"1","filterType":"MARKET_LOT_SIZE","maxQty":"500000","minQty":"1"},{"limit":200,"filterType":"MAX_NUM_ORDERS"},{"limit":10,"filterType":"MAX_NUM_ALGO_ORDERS"},{"notional":"5","filterType":"MIN_NOTIONAL"},{"multiplierDown":"0.9000","multiplierUp":"1.1000","multiplierDecimal":"4","filterType":"PERCENT_PRICE"}],"orderTypes":["LIMIT","MARKET","STOP","STOP_MARKET","TAKE_PROFIT","TAKE_PROFIT_MARKET","TRAILING_STOP_MARKET"],"timeInForce":["GTC","IOC","FOK","GTX"]},{"symbol":"ZENUSDT","pair":"ZENUSDT","contractType":"PERPETUAL","deliveryDate":4133404800000,"onboardDate":1569398400000,"status":"TRADING","maintMarginPercent":"2.5000","requiredMarginPercent":"5.0000","baseAsset":"ZEN","quoteAsset":"USDT","marginAsset":"USDT","pricePrecision":3,"quantityPrecision":1,"baseAssetPrecision":8,"quotePrecision":8,"underlyingType":"COIN","underlyingSubType":["Privacy"],"settlePlan":0,"triggerProtect":"0.1000","liquidationFee":"0.010000","marketTakeBound":"0.10","filters":[{"minPrice":"1.437","maxPrice":"10000","filterType":"PRICE_FILTER","tickSize":"0.001"},{"stepSize":"0.1","filterType":"LOT_SIZE","maxQty":"1000000","minQty":"0.1"},{"stepSize":"0.1","filterType":"MARKET_LOT_SIZE","maxQty":"4000","minQty":"0.1"},{"limit":200,"filterType":"MAX_NUM_ORDERS"},{"limit":10,"filterType":"MAX_NUM_ALGO_ORDERS"},{"notional":"5","filterType":"MIN_NOTIONAL"},{"multiplierDown":"0.9000","multiplierUp":"1.1000","multiplierDecimal":"4","filterType":"PERCENT_PRICE"}],"orderTypes":["LIMIT","MARKET","STOP","STOP_MARKET","TAKE_PROFIT","TAKE_PROFIT_MARKET","TRAILING_STOP_MARKET"],"timeInForce":["GTC","IOC","FOK","GTX"]},{"symbol":"SKLUSDT","pair":"SKLUSDT","contractType":"PERPETUAL","deliveryDate":4133404800000,"onboardDate":1598252400000,"status":"TRADING","maintMarginPercent":"2.5000","requiredMarginPercent":"5.0000","baseAsset":"SKL","quoteAsset":"USDT","marginAsset":"USDT","pricePrecision":5,"quantityPrecision":0,"baseAssetPrecision":8,"quotePrecision":8,"underlyingType":"COIN","underlyingSubType":["Layer-2"],"settlePlan":0,"triggerProtect":"0.1000","liquidationFee":"0.010000","marketTakeBound":"0.10","filters":[{"minPrice":"0.00544","maxPrice":"100","filterType":"PRICE_FILTER","tickSize":"0.00001"},{"stepSize":"1","filterType":"LOT_SIZE","maxQty":"10000000","minQty":"1"},{"stepSize":"1","filterType":"MARKET_LOT_SIZE","maxQty":"1000000","minQty":"1"},{"limit":200,"filterType":"MAX_NUM_ORDERS"},{"limit":10,"filterType":"MAX_NUM_ALGO_ORDERS"},{"notional":"5","filterType":"MIN_NOTIONAL"},{"multiplierDown":"0.9000","multiplierUp":"1.1000","multiplierDecimal":"4","filterType":"PERCENT_PRICE"}],"orderTypes":["LIMIT","MARKET","STOP","STOP_MARKET","TAKE_PROFIT","TAKE_PROFIT_MARKET","TRAILING_STOP_MARKET"],"timeInForce":["GTC","IOC","FOK","GTX"]},{"symbol":"GRTUSDT","pair":"GRTUSDT","contractType":"PERPETUAL","deliveryDate":4133404800000,"onboardDate":1608274800000,"status":"TRADING","maintMarginPercent":"2.5000","requiredMarginPercent":"5.0000","baseAsset":"GRT","quoteAsset":"USDT","marginAsset":"USDT","pricePrecision":5,"quantityPrecision":0,"baseAssetPrecision":8,"quotePrecision":8,"underlyingType":"COIN","underlyingSubType":["Oracle"],"settlePlan":0,"triggerProtect":"0.1000","liquidationFee":"0.010000","marketTakeBound":"0.10","filters":[{"minPrice":"0.01398","maxPrice":"100","filterType":"PRICE_FILTER","tickSize":"0.00001"},{"stepSize":"1","filterType":"LOT_SIZE","maxQty":"1000000","minQty":"1"},{"stepSize":"1","filterType":"MARKET_LOT_SIZE","maxQty":"150000","minQty":"1"},{"limit":200,"filterType":"MAX_NUM_ORDERS"},{"limit":10,"filterType":"MAX_NUM_ALGO_ORDERS"},{"notional":"5","filterType":"MIN_NOTIONAL"},{"multiplierDown":"0.9000","multiplierUp":"1.1000","multiplierDecimal":"4","filterType":"PERCENT_PRICE"}],"orderTypes":["LIMIT","MARKET","STOP","STOP_MARKET","TAKE_PROFIT","TAKE_PROFIT_MARKET","TRAILING_STOP_MARKET"],"timeInForce":["GTC","IOC","FOK","GTX"]},{"symbol":"1INCHUSDT","pair":"1INCHUSDT","contractType":"PERPETUAL","deliveryDate":4133404800000,"onboardDate":1608879600000,"status":"TRADING","maintMarginPercent":"2.5000","requiredMarginPercent":"5.0000","baseAsset":"1INCH","quoteAsset":"USDT","marginAsset":"USDT","pricePrecision":4,"quantityPrecision":0,"baseAssetPrecision":8,"quotePrecision":8,"underlyingType":"COIN","underlyingSubType":["DEX"],"settlePlan":0,"triggerProtect":"0.1000","liquidationFee":"0.010000","marketTakeBound":"0.10","filters":[{"minPrice":"0.0613","maxPrice":"100000","filterType":"PRICE_FILTER","tickSize":"0.0001"},{"stepSize":"1","filterType":"LOT_SIZE","maxQty":"1000000","minQty":"1"},{"stepSize":"1","filterType":"MARKET_LOT_SIZE","maxQty":"100000","minQty":"1"},{"limit":200,"filterType":"MAX_NUM_ORDERS"},{"limit":10,"filterType":"MAX_NUM_ALGO_ORDERS"},{"notional":"5","filterType":"MIN_NOTIONAL"},{"multiplierDown":"0.9000","multiplierUp":"1.1000","multiplierDecimal":"4","filterType":"PERCENT_PRICE"}],"orderTypes":["LIMIT","MARKET","STOP","STOP_MARKET","TAKE_PROFIT","TAKE_PROFIT_MARKET","TRAILING_STOP_MARKET"],"timeInForce":["GTC","IOC","FOK","GTX"]},{"symbol":"BTCBUSD","pair":"BTCBUSD","contractType":"PERPETUAL","deliveryDate":4133404800000,"onboardDate":1610352000000,"status":"TRADING","maintMarginPercent":"2.5000","requiredMarginPercent":"5.0000","baseAsset":"BTC","quoteAsset":"BUSD","marginAsset":"BUSD","pricePrecision":1,"quantityPrecision":3,"baseAssetPrecision":8,"quotePrecision":8,"underlyingType":"COIN","underlyingSubType":["BUSD"],"settlePlan":0,"triggerProtect":"0.0500","liquidationFee":"0.012500","marketTakeBound":"0.05","filters":[{"minPrice":"557.6","maxPrice":"4529890","filterType":"PRICE_FILTER","tickSize":"0.1"},{"stepSize":"0.001","filterType":"LOT_SIZE","maxQty":"500","minQty":"0.001"},{"stepSize":"0.001","filterType":"MARKET_LOT_SIZE","maxQty":"50","minQty":"0.001"},{"limit":200,"filterType":"MAX_NUM_ORDERS"},{"limit":10,"filterType":"MAX_NUM_ALGO_ORDERS"},{"notional":"5","filterType":"MIN_NOTIONAL"},{"multiplierDown":"0.9500","multiplierUp":"1.0500","multiplierDecimal":"4","filterType":"PERCENT_PRICE"}],"orderTypes":["LIMIT","MARKET","STOP","STOP_MARKET","TAKE_PROFIT","TAKE_PROFIT_MARKET","TRAILING_STOP_MARKET"],"timeInForce":["GTC","IOC","FOK","GTX"]},{"symbol":"CHZUSDT","pair":"CHZUSDT","contractType":"PERPETUAL","deliveryDate":4133404800000,"onboardDate":1611212400000,"status":"TRADING","maintMarginPercent":"2.5000","requiredMarginPercent":"5.0000","baseAsset":"CHZ","quoteAsset":"USDT","marginAsset":"USDT","pricePrecision":5,"quantityPrecision":0,"baseAssetPrecision":8,"quotePrecision":8,"underlyingType":"COIN","underlyingSubType":["NFT"],"settlePlan":0,"triggerProtect":"0.1000","liquidationFee":"0.015000","marketTakeBound":"0.10","filters":[{"minPrice":"0.00447","maxPrice":"100","filterType":"PRICE_FILTER","tickSize":"0.00001"},{"stepSize":"1","filterType":"LOT_SIZE","maxQty":"10000000","minQty":"1"},{"stepSize":"1","filterType":"MARKET_LOT_SIZE","maxQty":"850000","minQty":"1"},{"limit":200,"filterType":"MAX_NUM_ORDERS"},{"limit":10,"filterType":"MAX_NUM_ALGO_ORDERS"},{"notional":"5","filterType":"MIN_NOTIONAL"},{"multiplierDown":"0.9000","multiplierUp":"1.1000","multiplierDecimal":"4","filterType":"PERCENT_PRICE"}],"orderTypes":["LIMIT","MARKET","STOP","STOP_MARKET","TAKE_PROFIT","TAKE_PROFIT_MARKET","TRAILING_STOP_MARKET"],"timeInForce":["GTC","IOC","FOK","GTX"]},{"symbol":"SANDUSDT","pair":"SANDUSDT","contractType":"PERPETUAL","deliveryDate":4133404800000,"onboardDate":1610953200000,"status":"TRADING","maintMarginPercent":"2.5000","requiredMarginPercent":"5.0000","baseAsset":"SAND","quoteAsset":"USDT","marginAsset":"USDT","pricePrecision":5,"quantityPrecision":0,"baseAssetPrecision":8,"quotePrecision":8,"underlyingType":"COIN","underlyingSubType":["Metaverse"],"settlePlan":0,"triggerProtect":"0.1000","liquidationFee":"0.015000","marketTakeBound":"0.10","filters":[{"minPrice":"0.00500","maxPrice":"20000","filterType":"PRICE_FILTER","tickSize":"0.00010"},{"stepSize":"1","filterType":"LOT_SIZE","maxQty":"10000000","minQty":"1"},{"stepSize":"1","filterType":"MARKET_LOT_SIZE","maxQty":"500000","minQty":"1"},{"limit":200,"filterType":"MAX_NUM_ORDERS"},{"limit":10,"filterType":"MAX_NUM_ALGO_ORDERS"},{"notional":"5","filterType":"MIN_NOTIONAL"},{"multiplierDown":"0.9000","multiplierUp":"1.1000","multiplierDecimal":"4","filterType":"PERCENT_PRICE"}],"orderTypes":["LIMIT","MARKET","STOP","STOP_MARKET","TAKE_PROFIT","TAKE_PROFIT_MARKET","TRAILING_STOP_MARKET"],"timeInForce":["GTC","IOC","FOK","GTX"]},{"symbol":"ANKRUSDT","pair":"ANKRUSDT","contractType":"PERPETUAL","deliveryDate":4133404800000,"onboardDate":1611558000000,"status":"TRADING","maintMarginPercent":"2.5000","requiredMarginPercent":"5.0000","baseAsset":"ANKR","quoteAsset":"USDT","marginAsset":"USDT","pricePrecision":6,"quantityPrecision":0,"baseAssetPrecision":8,"quotePrecision":8,"underlyingType":"COIN","underlyingSubType":["Infrastructure"],"settlePlan":0,"triggerProtect":"0.1000","liquidationFee":"0.010000","marketTakeBound":"0.10","filters":[{"minPrice":"0.001510","maxPrice":"2000","filterType":"PRICE_FILTER","tickSize":"0.000010"},{"stepSize":"1","filterType":"LOT_SIZE","maxQty":"60000000","minQty":"1"},{"stepSize":"1","filterType":"MARKET_LOT_SIZE","maxQty":"10000000","minQty":"1"},{"limit":200,"filterType":"MAX_NUM_ORDERS"},{"limit":10,"filterType":"MAX_NUM_ALGO_ORDERS"},{"notional":"5","filterType":"MIN_NOTIONAL"},{"multiplierDown":"0.9000","multiplierUp":"1.1000","multiplierDecimal":"4","filterType":"PERCENT_PRICE"}],"orderTypes":["LIMIT","MARKET","STOP","STOP_MARKET","TAKE_PROFIT","TAKE_PROFIT_MARKET","TRAILING_STOP_MARKET"],"timeInForce":["GTC","IOC","FOK","GTX"]},{"symbol":"BTSUSDT","pair":"BTSUSDT","contractType":"PERPETUAL","deliveryDate":1660813200000,"onboardDate":1612076400000,"status":"SETTLING","maintMarginPercent":"2.5000","requiredMarginPercent":"5.0000","baseAsset":"BTS","quoteAsset":"USDT","marginAsset":"USDT","pricePrecision":5,"quantityPrecision":0,"baseAssetPrecision":8,"quotePrecision":8,"underlyingType":"COIN","underlyingSubType":["DEX"],"settlePlan":0,"triggerProtect":"0.0500","liquidationFee":"0.025000","marketTakeBound":"0.10","filters":[{"minPrice":"0.00114","maxPrice":"20000","filterType":"PRICE_FILTER","tickSize":"0.00001"},{"stepSize":"1","filterType":"LOT_SIZE","maxQty":"10000000","minQty":"1"},{"stepSize":"1","filterType":"MARKET_LOT_SIZE","maxQty":"3000000","minQty":"1"},{"limit":200,"filterType":"MAX_NUM_ORDERS"},{"limit":10,"filterType":"MAX_NUM_ALGO_ORDERS"},{"notional":"5","filterType":"MIN_NOTIONAL"},{"multiplierDown":"0.9000","multiplierUp":"1.1000","multiplierDecimal":"4","filterType":"PERCENT_PRICE"}],"orderTypes":["LIMIT","MARKET","STOP","STOP_MARKET","TAKE_PROFIT","TAKE_PROFIT_MARKET","TRAILING_STOP_MARKET"],"timeInForce":["GTC","IOC","FOK","GTX"]},{"symbol":"LITUSDT","pair":"LITUSDT","contractType":"PERPETUAL","deliveryDate":4133404800000,"onboardDate":1613545200000,"status":"TRADING","maintMarginPercent":"2.5000","requiredMarginPercent":"5.0000","baseAsset":"LIT","quoteAsset":"USDT","marginAsset":"USDT","pricePrecision":3,"quantityPrecision":1,"baseAssetPrecision":8,"quotePrecision":8,"underlyingType":"COIN","underlyingSubType":["Privacy"],"settlePlan":0,"triggerProtect":"0.1000","liquidationFee":"0.015000","marketTakeBound":"0.10","filters":[{"minPrice":"0.077","maxPrice":"100000","filterType":"PRICE_FILTER","tickSize":"0.001"},{"stepSize":"0.1","filterType":"LOT_SIZE","maxQty":"5000000","minQty":"0.1"},{"stepSize":"0.1","filterType":"MARKET_LOT_SIZE","maxQty":"1000000","minQty":"0.1"},{"limit":200,"filterType":"MAX_NUM_ORDERS"},{"limit":10,"filterType":"MAX_NUM_ALGO_ORDERS"},{"notional":"5","filterType":"MIN_NOTIONAL"},{"multiplierDown":"0.9000","multiplierUp":"1.1000","multiplierDecimal":"4","filterType":"PERCENT_PRICE"}],"orderTypes":["LIMIT","MARKET","STOP","STOP_MARKET","TAKE_PROFIT","TAKE_PROFIT_MARKET","TRAILING_STOP_MARKET"],"timeInForce":["GTC","IOC","FOK","GTX"]},{"symbol":"UNFIUSDT","pair":"UNFIUSDT","contractType":"PERPETUAL","deliveryDate":4133404800000,"onboardDate":1612854000000,"status":"TRADING","maintMarginPercent":"2.5000","requiredMarginPercent":"5.0000","baseAsset":"UNFI","quoteAsset":"USDT","marginAsset":"USDT","pricePrecision":3,"quantityPrecision":1,"baseAssetPrecision":8,"quotePrecision":8,"underlyingType":"COIN","underlyingSubType":["DeFi"],"settlePlan":0,"triggerProtect":"0.1000","liquidationFee":"0.010000","marketTakeBound":"0.10","filters":[{"minPrice":"0.222","maxPrice":"10000","filterType":"PRICE_FILTER","tickSize":"0.001"},{"stepSize":"0.1","filterType":"LOT_SIZE","maxQty":"100000","minQty":"0.1"},{"stepSize":"0.1","filterType":"MARKET_LOT_SIZE","maxQty":"8000","minQty":"0.1"},{"limit":200,"filterType":"MAX_NUM_ORDERS"},{"limit":10,"filterType":"MAX_NUM_ALGO_ORDERS"},{"notional":"5","filterType":"MIN_NOTIONAL"},{"multiplierDown":"0.9000","multiplierUp":"1.1000","multiplierDecimal":"4","filterType":"PERCENT_PRICE"}],"orderTypes":["LIMIT","MARKET","STOP","STOP_MARKET","TAKE_PROFIT","TAKE_PROFIT_MARKET","TRAILING_STOP_MARKET"],"timeInForce":["GTC","IOC","FOK","GTX"]},{"symbol":"REEFUSDT","pair":"REEFUSDT","contractType":"PERPETUAL","deliveryDate":4133404800000,"onboardDate":1611558000000,"status":"TRADING","maintMarginPercent":"2.5000","requiredMarginPercent":"5.0000","baseAsset":"REEF","quoteAsset":"USDT","marginAsset":"USDT","pricePrecision":6,"quantityPrecision":0,"baseAssetPrecision":8,"quotePrecision":8,"underlyingType":"COIN","underlyingSubType":["DeFi"],"settlePlan":0,"triggerProtect":"0.1200","liquidationFee":"0.010000","marketTakeBound":"0.12","filters":[{"minPrice":"0.000516","maxPrice":"2000","filterType":"PRICE_FILTER","tickSize":"0.000001"},{"stepSize":"1","filterType":"LOT_SIZE","maxQty":"200000000","minQty":"1"},{"stepSize":"1","filterType":"MARKET_LOT_SIZE","maxQty":"100000000","minQty":"1"},{"limit":200,"filterType":"MAX_NUM_ORDERS"},{"limit":10,"filterType":"MAX_NUM_ALGO_ORDERS"},{"notional":"5","filterType":"MIN_NOTIONAL"},{"multiplierDown":"0.8800","multiplierUp":"1.1200","multiplierDecimal":"4","filterType":"PERCENT_PRICE"}],"orderTypes":["LIMIT","MARKET","STOP","STOP_MARKET","TAKE_PROFIT","TAKE_PROFIT_MARKET","TRAILING_STOP_MARKET"],"timeInForce":["GTC","IOC","FOK","GTX"]},{"symbol":"RVNUSDT","pair":"RVNUSDT","contractType":"PERPETUAL","deliveryDate":4133404800000,"onboardDate":1614063600000,"status":"TRADING","maintMarginPercent":"2.5000","requiredMarginPercent":"5.0000","baseAsset":"RVN","quoteAsset":"USDT","marginAsset":"USDT","pricePrecision":5,"quantityPrecision":0,"baseAssetPrecision":8,"quotePrecision":8,"underlyingType":"COIN","underlyingSubType":["PoW"],"settlePlan":0,"triggerProtect":"0.1000","liquidationFee":"0.010000","marketTakeBound":"0.10","filters":[{"minPrice":"0.00154","maxPrice":"20000","filterType":"PRICE_FILTER","tickSize":"0.00001"},{"stepSize":"1","filterType":"LOT_SIZE","maxQty":"10000000","minQty":"1"},{"stepSize":"1","filterType":"MARKET_LOT_SIZE","maxQty":"3000000","minQty":"1"},{"limit":200,"filterType":"MAX_NUM_ORDERS"},{"limit":10,"filterType":"MAX_NUM_ALGO_ORDERS"},{"notional":"5","filterType":"MIN_NOTIONAL"},{"multiplierDown":"0.9000","multiplierUp":"1.1000","multiplierDecimal":"4","filterType":"PERCENT_PRICE"}],"orderTypes":["LIMIT","MARKET","STOP","STOP_MARKET","TAKE_PROFIT","TAKE_PROFIT_MARKET","TRAILING_STOP_MARKET"],"timeInForce":["GTC","IOC","FOK","GTX"]},{"symbol":"SFPUSDT","pair":"SFPUSDT","contractType":"PERPETUAL","deliveryDate":4133404800000,"onboardDate":1614150000000,"status":"TRADING","maintMarginPercent":"2.5000","requiredMarginPercent":"5.0000","baseAsset":"SFP","quoteAsset":"USDT","marginAsset":"USDT","pricePrecision":4,"quantityPrecision":0,"baseAssetPrecision":8,"quotePrecision":8,"underlyingType":"COIN","underlyingSubType":["Wallet"],"settlePlan":0,"triggerProtect":"0.1000","liquidationFee":"0.015000","marketTakeBound":"0.10","filters":[{"minPrice":"0.0223","maxPrice":"100000","filterType":"PRICE_FILTER","tickSize":"0.0001"},{"stepSize":"1","filterType":"LOT_SIZE","maxQty":"5000000","minQty":"1"},{"stepSize":"1","filterType":"MARKET_LOT_SIZE","maxQty":"300000","minQty":"1"},{"limit":200,"filterType":"MAX_NUM_ORDERS"},{"limit":10,"filterType":"MAX_NUM_ALGO_ORDERS"},{"notional":"5","filterType":"MIN_NOTIONAL"},{"multiplierDown":"0.9000","multiplierUp":"1.1000","multiplierDecimal":"4","filterType":"PERCENT_PRICE"}],"orderTypes":["LIMIT","MARKET","STOP","STOP_MARKET","TAKE_PROFIT","TAKE_PROFIT_MARKET","TRAILING_STOP_MARKET"],"timeInForce":["GTC","IOC","FOK","GTX"]},{"symbol":"XEMUSDT","pair":"XEMUSDT","contractType":"PERPETUAL","deliveryDate":4133404800000,"onboardDate":1614668400000,"status":"TRADING","maintMarginPercent":"2.5000","requiredMarginPercent":"5.0000","baseAsset":"XEM","quoteAsset":"USDT","marginAsset":"USDT","pricePrecision":4,"quantityPrecision":0,"baseAssetPrecision":8,"quotePrecision":8,"underlyingType":"COIN","underlyingSubType":["Layer-1"],"settlePlan":0,"triggerProtect":"0.1000","liquidationFee":"0.010000","marketTakeBound":"0.10","filters":[{"minPrice":"0.0035","maxPrice":"100000","filterType":"PRICE_FILTER","tickSize":"0.0001"},{"stepSize":"1","filterType":"LOT_SIZE","maxQty":"10000000","minQty":"1"},{"stepSize":"1","filterType":"MARKET_LOT_SIZE","maxQty":"2000000","minQty":"1"},{"limit":200,"filterType":"MAX_NUM_ORDERS"},{"limit":10,"filterType":"MAX_NUM_ALGO_ORDERS"},{"notional":"5","filterType":"MIN_NOTIONAL"},{"multiplierDown":"0.9000","multiplierUp":"1.1000","multiplierDecimal":"4","filterType":"PERCENT_PRICE"}],"orderTypes":["LIMIT","MARKET","STOP","STOP_MARKET","TAKE_PROFIT","TAKE_PROFIT_MARKET","TRAILING_STOP_MARKET"],"timeInForce":["GTC","IOC","FOK","GTX"]},{"symbol":"BTCSTUSDT","pair":"BTCSTUSDT","contractType":"","deliveryDate":4133404800000,"onboardDate":1614754800000,"status":"PENDING_TRADING","maintMarginPercent":"2.5000","requiredMarginPercent":"5.0000","baseAsset":"BTCST","quoteAsset":"USDT","marginAsset":"USDT","pricePrecision":3,"quantityPrecision":1,"baseAssetPrecision":8,"quotePrecision":8,"underlyingType":"COIN","underlyingSubType":["DEFI"],"settlePlan":0,"triggerProtect":"0.1500","liquidationFee":"0.040000","marketTakeBound":"0.30","filters":[{"minPrice":"0.668","maxPrice":"100000","filterType":"PRICE_FILTER","tickSize":"0.001"},{"stepSize":"0.1","filterType":"LOT_SIZE","maxQty":"1000000","minQty":"0.1"},{"stepSize":"0.1","filterType":"MARKET_LOT_SIZE","maxQty":"1000","minQty":"0.1"},{"limit":200,"filterType":"MAX_NUM_ORDERS"},{"limit":10,"filterType":"MAX_NUM_ALGO_ORDERS"},{"notional":"5","filterType":"MIN_NOTIONAL"},{"multiplierDown":"8500","multiplierUp":"11500","multiplierDecimal":"4","filterType":"PERCENT_PRICE"}],"orderTypes":["LIMIT","MARKET","STOP","STOP_MARKET","TAKE_PROFIT","TAKE_PROFIT_MARKET","TRAILING_STOP_MARKET"],"timeInForce":["GTC","IOC","FOK","GTX"]},{"symbol":"COTIUSDT","pair":"COTIUSDT","contractType":"PERPETUAL","deliveryDate":4133404800000,"onboardDate":1615273200000,"status":"TRADING","maintMarginPercent":"2.5000","requiredMarginPercent":"5.0000","baseAsset":"COTI","quoteAsset":"USDT","marginAsset":"USDT","pricePrecision":5,"quantityPrecision":0,"baseAssetPrecision":8,"quotePrecision":8,"underlyingType":"COIN","underlyingSubType":["Payment"],"settlePlan":0,"triggerProtect":"0.1000","liquidationFee":"0.010000","marketTakeBound":"0.10","filters":[{"minPrice":"0.00328","maxPrice":"20000","filterType":"PRICE_FILTER","tickSize":"0.00001"},{"stepSize":"1","filterType":"LOT_SIZE","maxQty":"5000000","minQty":"1"},{"stepSize":"1","filterType":"MARKET_LOT_SIZE","maxQty":"800000","minQty":"1"},{"limit":200,"filterType":"MAX_NUM_ORDERS"},{"limit":10,"filterType":"MAX_NUM_ALGO_ORDERS"},{"notional":"5","filterType":"MIN_NOTIONAL"},{"multiplierDown":"0.9000","multiplierUp":"1.1000","multiplierDecimal":"4","filterType":"PERCENT_PRICE"}],"orderTypes":["LIMIT","MARKET","STOP","STOP_MARKET","TAKE_PROFIT","TAKE_PROFIT_MARKET","TRAILING_STOP_MARKET"],"timeInForce":["GTC","IOC","FOK","GTX"]},{"symbol":"CHRUSDT","pair":"CHRUSDT","contractType":"PERPETUAL","deliveryDate":4133404800000,"onboardDate":1615532400000,"status":"TRADING","maintMarginPercent":"2.5000","requiredMarginPercent":"5.0000","baseAsset":"CHR","quoteAsset":"USDT","marginAsset":"USDT","pricePrecision":4,"quantityPrecision":0,"baseAssetPrecision":8,"quotePrecision":8,"underlyingType":"COIN","underlyingSubType":["Layer-2"],"settlePlan":0,"triggerProtect":"0.1000","liquidationFee":"0.010000","marketTakeBound":"0.10","filters":[{"minPrice":"0.0031","maxPrice":"100000","filterType":"PRICE_FILTER","tickSize":"0.0001"},{"stepSize":"1","filterType":"LOT_SIZE","maxQty":"1000000","minQty":"1"},{"stepSize":"1","filterType":"MARKET_LOT_SIZE","maxQty":"250000","minQty":"1"},{"limit":200,"filterType":"MAX_NUM_ORDERS"},{"limit":10,"filterType":"MAX_NUM_ALGO_ORDERS"},{"notional":"5","filterType":"MIN_NOTIONAL"},{"multiplierDown":"0.9000","multiplierUp":"1.1000","multiplierDecimal":"4","filterType":"PERCENT_PRICE"}],"orderTypes":["LIMIT","MARKET","STOP","STOP_MARKET","TAKE_PROFIT","TAKE_PROFIT_MARKET","TRAILING_STOP_MARKET"],"timeInForce":["GTC","IOC","FOK","GTX"]},{"symbol":"MANAUSDT","pair":"MANAUSDT","contractType":"PERPETUAL","deliveryDate":4133404800000,"onboardDate":1615705200000,"status":"TRADING","maintMarginPercent":"2.5000","requiredMarginPercent":"5.0000","baseAsset":"MANA","quoteAsset":"USDT","marginAsset":"USDT","pricePrecision":4,"quantityPrecision":0,"baseAssetPrecision":8,"quotePrecision":8,"underlyingType":"COIN","underlyingSubType":["Metaverse"],"settlePlan":0,"triggerProtect":"0.1000","liquidationFee":"0.010000","marketTakeBound":"0.10","filters":[{"minPrice":"0.0136","maxPrice":"100000","filterType":"PRICE_FILTER","tickSize":"0.0001"},{"stepSize":"1","filterType":"LOT_SIZE","maxQty":"1000000","minQty":"1"},{"stepSize":"1","filterType":"MARKET_LOT_SIZE","maxQty":"100000","minQty":"1"},{"limit":200,"filterType":"MAX_NUM_ORDERS"},{"limit":10,"filterType":"MAX_NUM_ALGO_ORDERS"},{"notional":"5","filterType":"MIN_NOTIONAL"},{"multiplierDown":"0.9000","multiplierUp":"1.1000","multiplierDecimal":"4","filterType":"PERCENT_PRICE"}],"orderTypes":["LIMIT","MARKET","STOP","STOP_MARKET","TAKE_PROFIT","TAKE_PROFIT_MARKET","TRAILING_STOP_MARKET"],"timeInForce":["GTC","IOC","FOK","GTX"]},{"symbol":"ALICEUSDT","pair":"ALICEUSDT","contractType":"PERPETUAL","deliveryDate":4133404800000,"onboardDate":1615791600000,"status":"TRADING","maintMarginPercent":"2.5000","requiredMarginPercent":"5.0000","baseAsset":"ALICE","quoteAsset":"USDT","marginAsset":"USDT","pricePrecision":3,"quantityPrecision":1,"baseAssetPrecision":8,"quotePrecision":8,"underlyingType":"COIN","underlyingSubType":["Metaverse"],"settlePlan":0,"triggerProtect":"0.1000","liquidationFee":"0.010000","marketTakeBound":"0.10","filters":[{"minPrice":"0.120","maxPrice":"100000","filterType":"PRICE_FILTER","tickSize":"0.001"},{"stepSize":"0.1","filterType":"LOT_SIZE","maxQty":"50000","minQty":"0.1"},{"stepSize":"0.1","filterType":"MARKET_LOT_SIZE","maxQty":"12000","minQty":"0.1"},{"limit":200,"filterType":"MAX_NUM_ORDERS"},{"limit":10,"filterType":"MAX_NUM_ALGO_ORDERS"},{"notional":"5","filterType":"MIN_NOTIONAL"},{"multiplierDown":"0.9000","multiplierUp":"1.1000","multiplierDecimal":"4","filterType":"PERCENT_PRICE"}],"orderTypes":["LIMIT","MARKET","STOP","STOP_MARKET","TAKE_PROFIT","TAKE_PROFIT_MARKET","TRAILING_STOP_MARKET"],"timeInForce":["GTC","IOC","FOK","GTX"]},{"symbol":"HBARUSDT","pair":"HBARUSDT","contractType":"PERPETUAL","deliveryDate":4133404800000,"onboardDate":1615964400000,"status":"TRADING","maintMarginPercent":"2.5000","requiredMarginPercent":"5.0000","baseAsset":"HBAR","quoteAsset":"USDT","marginAsset":"USDT","pricePrecision":5,"quantityPrecision":0,"baseAssetPrecision":8,"quotePrecision":8,"underlyingType":"COIN","underlyingSubType":["Layer-1"],"settlePlan":0,"triggerProtect":"0.1000","liquidationFee":"0.010000","marketTakeBound":"0.10","filters":[{"minPrice":"0.00279","maxPrice":"20000","filterType":"PRICE_FILTER","tickSize":"0.00001"},{"stepSize":"1","filterType":"LOT_SIZE","maxQty":"10000000","minQty":"1"},{"stepSize":"1","filterType":"MARKET_LOT_SIZE","maxQty":"300000","minQty":"1"},{"limit":200,"filterType":"MAX_NUM_ORDERS"},{"limit":10,"filterType":"MAX_NUM_ALGO_ORDERS"},{"notional":"5","filterType":"MIN_NOTIONAL"},{"multiplierDown":"0.9000","multiplierUp":"1.1000","multiplierDecimal":"4","filterType":"PERCENT_PRICE"}],"orderTypes":["LIMIT","MARKET","STOP","STOP_MARKET","TAKE_PROFIT","TAKE_PROFIT_MARKET","TRAILING_STOP_MARKET"],"timeInForce":["GTC","IOC","FOK","GTX"]},{"symbol":"ONEUSDT","pair":"ONEUSDT","contractType":"PERPETUAL","deliveryDate":4133404800000,"onboardDate":1615964400000,"status":"TRADING","maintMarginPercent":"2.5000","requiredMarginPercent":"5.0000","baseAsset":"ONE","quoteAsset":"USDT","marginAsset":"USDT","pricePrecision":5,"quantityPrecision":0,"baseAssetPrecision":8,"quotePrecision":8,"underlyingType":"COIN","underlyingSubType":["Layer-2"],"settlePlan":0,"triggerProtect":"0.1000","liquidationFee":"0.010000","marketTakeBound":"0.10","filters":[{"minPrice":"0.00124","maxPrice":"20000","filterType":"PRICE_FILTER","tickSize":"0.00001"},{"stepSize":"1","filterType":"LOT_SIZE","maxQty":"10000000","minQty":"1"},{"stepSize":"1","filterType":"MARKET_LOT_SIZE","maxQty":"500000","minQty":"1"},{"limit":200,"filterType":"MAX_NUM_ORDERS"},{"limit":10,"filterType":"MAX_NUM_ALGO_ORDERS"},{"notional":"5","filterType":"MIN_NOTIONAL"},{"multiplierDown":"0.9000","multiplierUp":"1.1000","multiplierDecimal":"4","filterType":"PERCENT_PRICE"}],"orderTypes":["LIMIT","MARKET","STOP","STOP_MARKET","TAKE_PROFIT","TAKE_PROFIT_MARKET","TRAILING_STOP_MARKET"],"timeInForce":["GTC","IOC","FOK","GTX"]},{"symbol":"LINAUSDT","pair":"LINAUSDT","contractType":"PERPETUAL","deliveryDate":4133404800000,"onboardDate":1615964400000,"status":"TRADING","maintMarginPercent":"2.5000","requiredMarginPercent":"5.0000","baseAsset":"LINA","quoteAsset":"USDT","marginAsset":"USDT","pricePrecision":5,"quantityPrecision":0,"baseAssetPrecision":8,"quotePrecision":8,"underlyingType":"COIN","underlyingSubType":["DeFi"],"settlePlan":0,"triggerProtect":"0.1000","liquidationFee":"0.010000","marketTakeBound":"0.10","filters":[{"minPrice":"0.00087","maxPrice":"20000","filterType":"PRICE_FILTER","tickSize":"0.00001"},{"stepSize":"1","filterType":"LOT_SIZE","maxQty":"10000000","minQty":"1"},{"stepSize":"1","filterType":"MARKET_LOT_SIZE","maxQty":"2500000","minQty":"1"},{"limit":200,"filterType":"MAX_NUM_ORDERS"},{"limit":10,"filterType":"MAX_NUM_ALGO_ORDERS"},{"notional":"5","filterType":"MIN_NOTIONAL"},{"multiplierDown":"0.9000","multiplierUp":"1.1000","multiplierDecimal":"4","filterType":"PERCENT_PRICE"}],"orderTypes":["LIMIT","MARKET","STOP","STOP_MARKET","TAKE_PROFIT","TAKE_PROFIT_MARKET","TRAILING_STOP_MARKET"],"timeInForce":["GTC","IOC","FOK","GTX"]},{"symbol":"STMXUSDT","pair":"STMXUSDT","contractType":"PERPETUAL","deliveryDate":4133404800000,"onboardDate":1615964400000,"status":"TRADING","maintMarginPercent":"2.5000","requiredMarginPercent":"5.0000","baseAsset":"STMX","quoteAsset":"USDT","marginAsset":"USDT","pricePrecision":5,"quantityPrecision":0,"baseAssetPrecision":8,"quotePrecision":8,"underlyingType":"COIN","underlyingSubType":["Payment"],"settlePlan":0,"triggerProtect":"0.1000","liquidationFee":"0.010000","marketTakeBound":"0.10","filters":[{"minPrice":"0.00048","maxPrice":"20000","filterType":"PRICE_FILTER","tickSize":"0.00001"},{"stepSize":"1","filterType":"LOT_SIZE","maxQty":"10000000","minQty":"1"},{"stepSize":"1","filterType":"MARKET_LOT_SIZE","maxQty":"5000000","minQty":"1"},{"limit":200,"filterType":"MAX_NUM_ORDERS"},{"limit":10,"filterType":"MAX_NUM_ALGO_ORDERS"},{"notional":"5","filterType":"MIN_NOTIONAL"},{"multiplierDown":"0.9000","multiplierUp":"1.1000","multiplierDecimal":"4","filterType":"PERCENT_PRICE"}],"orderTypes":["LIMIT","MARKET","STOP","STOP_MARKET","TAKE_PROFIT","TAKE_PROFIT_MARKET","TRAILING_STOP_MARKET"],"timeInForce":["GTC","IOC","FOK","GTX"]},{"symbol":"DENTUSDT","pair":"DENTUSDT","contractType":"PERPETUAL","deliveryDate":4133404800000,"onboardDate":1616482800000,"status":"TRADING","maintMarginPercent":"2.5000","requiredMarginPercent":"5.0000","baseAsset":"DENT","quoteAsset":"USDT","marginAsset":"USDT","pricePrecision":6,"quantityPrecision":0,"baseAssetPrecision":8,"quotePrecision":8,"underlyingType":"COIN","underlyingSubType":["Infrastructure"],"settlePlan":0,"triggerProtect":"0.1000","liquidationFee":"0.010000","marketTakeBound":"0.10","filters":[{"minPrice":"0.000070","maxPrice":"2000","filterType":"PRICE_FILTER","tickSize":"0.000001"},{"stepSize":"1","filterType":"LOT_SIZE","maxQty":"10000000","minQty":"1"},{"stepSize":"1","filterType":"MARKET_LOT_SIZE","maxQty":"2000000","minQty":"1"},{"limit":200,"filterType":"MAX_NUM_ORDERS"},{"limit":10,"filterType":"MAX_NUM_ALGO_ORDERS"},{"notional":"5","filterType":"MIN_NOTIONAL"},{"multiplierDown":"0.9000","multiplierUp":"1.1000","multiplierDecimal":"4","filterType":"PERCENT_PRICE"}],"orderTypes":["LIMIT","MARKET","STOP","STOP_MARKET","TAKE_PROFIT","TAKE_PROFIT_MARKET","TRAILING_STOP_MARKET"],"timeInForce":["GTC","IOC","FOK","GTX"]},{"symbol":"CELRUSDT","pair":"CELRUSDT","contractType":"PERPETUAL","deliveryDate":4133404800000,"onboardDate":1617001200000,"status":"TRADING","maintMarginPercent":"2.5000","requiredMarginPercent":"5.0000","baseAsset":"CELR","quoteAsset":"USDT","marginAsset":"USDT","pricePrecision":5,"quantityPrecision":0,"baseAssetPrecision":8,"quotePrecision":8,"underlyingType":"COIN","underlyingSubType":["Layer-2"],"settlePlan":0,"triggerProtect":"0.1000","liquidationFee":"0.010000","marketTakeBound":"0.10","filters":[{"minPrice":"0.00050","maxPrice":"20000","filterType":"PRICE_FILTER","tickSize":"0.00001"},{"stepSize":"1","filterType":"LOT_SIZE","maxQty":"10000000","minQty":"1"},{"stepSize":"1","filterType":"MARKET_LOT_SIZE","maxQty":"500000","minQty":"1"},{"limit":200,"filterType":"MAX_NUM_ORDERS"},{"limit":10,"filterType":"MAX_NUM_ALGO_ORDERS"},{"notional":"5","filterType":"MIN_NOTIONAL"},{"multiplierDown":"0.9000","multiplierUp":"1.1000","multiplierDecimal":"4","filterType":"PERCENT_PRICE"}],"orderTypes":["LIMIT","MARKET","STOP","STOP_MARKET","TAKE_PROFIT","TAKE_PROFIT_MARKET","TRAILING_STOP_MARKET"],"timeInForce":["GTC","IOC","FOK","GTX"]},{"symbol":"HOTUSDT","pair":"HOTUSDT","contractType":"PERPETUAL","deliveryDate":4133404800000,"onboardDate":1617087600000,"status":"TRADING","maintMarginPercent":"2.5000","requiredMarginPercent":"5.0000","baseAsset":"HOT","quoteAsset":"USDT","marginAsset":"USDT","pricePrecision":6,"quantityPrecision":0,"baseAssetPrecision":8,"quotePrecision":8,"underlyingType":"COIN","underlyingSubType":["Storage"],"settlePlan":0,"triggerProtect":"0.1000","liquidationFee":"0.010000","marketTakeBound":"0.10","filters":[{"minPrice":"0.000129","maxPrice":"2000","filterType":"PRICE_FILTER","tickSize":"0.000001"},{"stepSize":"1","filterType":"LOT_SIZE","maxQty":"20000000","minQty":"1"},{"stepSize":"1","filterType":"MARKET_LOT_SIZE","maxQty":"10000000","minQty":"1"},{"limit":200,"filterType":"MAX_NUM_ORDERS"},{"limit":10,"filterType":"MAX_NUM_ALGO_ORDERS"},{"notional":"5","filterType":"MIN_NOTIONAL"},{"multiplierDown":"0.9000","multiplierUp":"1.1000","multiplierDecimal":"4","filterType":"PERCENT_PRICE"}],"orderTypes":["LIMIT","MARKET","STOP","STOP_MARKET","TAKE_PROFIT","TAKE_PROFIT_MARKET","TRAILING_STOP_MARKET"],"timeInForce":["GTC","IOC","FOK","GTX"]},{"symbol":"MTLUSDT","pair":"MTLUSDT","contractType":"PERPETUAL","deliveryDate":4133404800000,"onboardDate":1617174000000,"status":"TRADING","maintMarginPercent":"2.5000","requiredMarginPercent":"5.0000","baseAsset":"MTL","quoteAsset":"USDT","marginAsset":"USDT","pricePrecision":4,"quantityPrecision":0,"baseAssetPrecision":8,"quotePrecision":8,"underlyingType":"COIN","underlyingSubType":["Wallet"],"settlePlan":0,"triggerProtect":"0.1000","liquidationFee":"0.020000","marketTakeBound":"0.10","filters":[{"minPrice":"0.0390","maxPrice":"100000","filterType":"PRICE_FILTER","tickSize":"0.0001"},{"stepSize":"1","filterType":"LOT_SIZE","maxQty":"1000000","minQty":"1"},{"stepSize":"1","filterType":"MARKET_LOT_SIZE","maxQty":"300000","minQty":"1"},{"limit":200,"filterType":"MAX_NUM_ORDERS"},{"limit":10,"filterType":"MAX_NUM_ALGO_ORDERS"},{"notional":"5","filterType":"MIN_NOTIONAL"},{"multiplierDown":"0.9000","multiplierUp":"1.1000","multiplierDecimal":"4","filterType":"PERCENT_PRICE"}],"orderTypes":["LIMIT","MARKET","STOP","STOP_MARKET","TAKE_PROFIT","TAKE_PROFIT_MARKET","TRAILING_STOP_MARKET"],"timeInForce":["GTC","IOC","FOK","GTX"]},{"symbol":"OGNUSDT","pair":"OGNUSDT","contractType":"PERPETUAL","deliveryDate":4133404800000,"onboardDate":1617174000000,"status":"TRADING","maintMarginPercent":"2.5000","requiredMarginPercent":"5.0000","baseAsset":"OGN","quoteAsset":"USDT","marginAsset":"USDT","pricePrecision":4,"quantityPrecision":0,"baseAssetPrecision":8,"quotePrecision":8,"underlyingType":"COIN","underlyingSubType":["NFT"],"settlePlan":0,"triggerProtect":"0.1000","liquidationFee":"0.010000","marketTakeBound":"0.10","filters":[{"minPrice":"0.0137","maxPrice":"100000","filterType":"PRICE_FILTER","tickSize":"0.0001"},{"stepSize":"1","filterType":"LOT_SIZE","maxQty":"1000000","minQty":"1"},{"stepSize":"1","filterType":"MARKET_LOT_SIZE","maxQty":"100000","minQty":"1"},{"limit":200,"filterType":"MAX_NUM_ORDERS"},{"limit":10,"filterType":"MAX_NUM_ALGO_ORDERS"},{"notional":"5","filterType":"MIN_NOTIONAL"},{"multiplierDown":"0.9000","multiplierUp":"1.1000","multiplierDecimal":"4","filterType":"PERCENT_PRICE"}],"orderTypes":["LIMIT","MARKET","STOP","STOP_MARKET","TAKE_PROFIT","TAKE_PROFIT_MARKET","TRAILING_STOP_MARKET"],"timeInForce":["GTC","IOC","FOK","GTX"]},{"symbol":"NKNUSDT","pair":"NKNUSDT","contractType":"PERPETUAL","deliveryDate":4133404800000,"onboardDate":1617865200000,"status":"TRADING","maintMarginPercent":"2.5000","requiredMarginPercent":"5.0000","baseAsset":"NKN","quoteAsset":"USDT","marginAsset":"USDT","pricePrecision":5,"quantityPrecision":0,"baseAssetPrecision":8,"quotePrecision":8,"underlyingType":"COIN","underlyingSubType":["Infrastructure"],"settlePlan":0,"triggerProtect":"0.1000","liquidationFee":"0.010000","marketTakeBound":"0.10","filters":[{"minPrice":"0.00602","maxPrice":"20000","filterType":"PRICE_FILTER","tickSize":"0.00001"},{"stepSize":"1","filterType":"LOT_SIZE","maxQty":"10000000","minQty":"1"},{"stepSize":"1","filterType":"MARKET_LOT_SIZE","maxQty":"500000","minQty":"1"},{"limit":200,"filterType":"MAX_NUM_ORDERS"},{"limit":10,"filterType":"MAX_NUM_ALGO_ORDERS"},{"notional":"5","filterType":"MIN_NOTIONAL"},{"multiplierDown":"0.9000","multiplierUp":"1.1000","multiplierDecimal":"4","filterType":"PERCENT_PRICE"}],"orderTypes":["LIMIT","MARKET","STOP","STOP_MARKET","TAKE_PROFIT","TAKE_PROFIT_MARKET","TRAILING_STOP_MARKET"],"timeInForce":["GTC","IOC","FOK","GTX"]},{"symbol":"SCUSDT","pair":"SCUSDT","contractType":"PERPETUAL","deliveryDate":1655456400000,"onboardDate":1618210800000,"status":"SETTLING","maintMarginPercent":"2.5000","requiredMarginPercent":"5.0000","baseAsset":"SC","quoteAsset":"USDT","marginAsset":"USDT","pricePrecision":6,"quantityPrecision":0,"baseAssetPrecision":8,"quotePrecision":8,"underlyingType":"COIN","underlyingSubType":["Storage"],"settlePlan":0,"triggerProtect":"0.0500","liquidationFee":"0.010000","marketTakeBound":"0.10","filters":[{"minPrice":"0.000368","maxPrice":"2000","filterType":"PRICE_FILTER","tickSize":"0.000001"},{"stepSize":"1","filterType":"LOT_SIZE","maxQty":"10000000","minQty":"1"},{"stepSize":"1","filterType":"MARKET_LOT_SIZE","maxQty":"5000000","minQty":"1"},{"limit":200,"filterType":"MAX_NUM_ORDERS"},{"limit":10,"filterType":"MAX_NUM_ALGO_ORDERS"},{"notional":"5","filterType":"MIN_NOTIONAL"},{"multiplierDown":"0.9000","multiplierUp":"1.1000","multiplierDecimal":"4","filterType":"PERCENT_PRICE"}],"orderTypes":["LIMIT","MARKET","STOP","STOP_MARKET","TAKE_PROFIT","TAKE_PROFIT_MARKET","TRAILING_STOP_MARKET"],"timeInForce":["GTC","IOC","FOK","GTX"]},{"symbol":"DGBUSDT","pair":"DGBUSDT","contractType":"PERPETUAL","deliveryDate":4133404800000,"onboardDate":1618815600000,"status":"TRADING","maintMarginPercent":"2.5000","requiredMarginPercent":"5.0000","baseAsset":"DGB","quoteAsset":"USDT","marginAsset":"USDT","pricePrecision":5,"quantityPrecision":0,"baseAssetPrecision":8,"quotePrecision":8,"underlyingType":"COIN","underlyingSubType":["Payment"],"settlePlan":0,"triggerProtect":"0.1000","liquidationFee":"0.010000","marketTakeBound":"0.10","filters":[{"minPrice":"0.00131","maxPrice":"20000","filterType":"PRICE_FILTER","tickSize":"0.00001"},{"stepSize":"1","filterType":"LOT_SIZE","maxQty":"10000000","minQty":"1"},{"stepSize":"1","filterType":"MARKET_LOT_SIZE","maxQty":"500000","minQty":"1"},{"limit":200,"filterType":"MAX_NUM_ORDERS"},{"limit":10,"filterType":"MAX_NUM_ALGO_ORDERS"},{"notional":"5","filterType":"MIN_NOTIONAL"},{"multiplierDown":"0.9000","multiplierUp":"1.1000","multiplierDecimal":"4","filterType":"PERCENT_PRICE"}],"orderTypes":["LIMIT","MARKET","STOP","STOP_MARKET","TAKE_PROFIT","TAKE_PROFIT_MARKET","TRAILING_STOP_MARKET"],"timeInForce":["GTC","IOC","FOK","GTX"]},{"symbol":"1000SHIBUSDT","pair":"1000SHIBUSDT","contractType":"PERPETUAL","deliveryDate":4133404800000,"onboardDate":1620630000000,"status":"TRADING","maintMarginPercent":"2.5000","requiredMarginPercent":"5.0000","baseAsset":"1000SHIB","quoteAsset":"USDT","marginAsset":"USDT","pricePrecision":6,"quantityPrecision":0,"baseAssetPrecision":8,"quotePrecision":8,"underlyingType":"COIN","underlyingSubType":["Meme"],"settlePlan":0,"triggerProtect":"0.1000","liquidationFee":"0.015000","marketTakeBound":"0.10","filters":[{"minPrice":"0.000157","maxPrice":"2000","filterType":"PRICE_FILTER","tickSize":"0.000001"},{"stepSize":"1","filterType":"LOT_SIZE","maxQty":"100000000","minQty":"1"},{"stepSize":"1","filterType":"MARKET_LOT_SIZE","maxQty":"20000000","minQty":"1"},{"limit":200,"filterType":"MAX_NUM_ORDERS"},{"limit":10,"filterType":"MAX_NUM_ALGO_ORDERS"},{"notional":"5","filterType":"MIN_NOTIONAL"},{"multiplierDown":"0.9000","multiplierUp":"1.1000","multiplierDecimal":"4","filterType":"PERCENT_PRICE"}],"orderTypes":["LIMIT","MARKET","STOP","STOP_MARKET","TAKE_PROFIT","TAKE_PROFIT_MARKET","TRAILING_STOP_MARKET"],"timeInForce":["GTC","IOC","FOK","GTX"]},{"symbol":"BAKEUSDT","pair":"BAKEUSDT","contractType":"PERPETUAL","deliveryDate":4133404800000,"onboardDate":1621321200000,"status":"TRADING","maintMarginPercent":"2.5000","requiredMarginPercent":"5.0000","baseAsset":"BAKE","quoteAsset":"USDT","marginAsset":"USDT","pricePrecision":4,"quantityPrecision":0,"baseAssetPrecision":8,"quotePrecision":8,"underlyingType":"COIN","underlyingSubType":["NFT"],"settlePlan":0,"triggerProtect":"0.1000","liquidationFee":"0.010000","marketTakeBound":"0.10","filters":[{"minPrice":"0.0001","maxPrice":"100000","filterType":"PRICE_FILTER","tickSize":"0.0001"},{"stepSize":"1","filterType":"LOT_SIZE","maxQty":"500000","minQty":"1"},{"stepSize":"1","filterType":"MARKET_LOT_SIZE","maxQty":"30000","minQty":"1"},{"limit":200,"filterType":"MAX_NUM_ORDERS"},{"limit":10,"filterType":"MAX_NUM_ALGO_ORDERS"},{"notional":"5","filterType":"MIN_NOTIONAL"},{"multiplierDown":"0.9000","multiplierUp":"1.1000","multiplierDecimal":"4","filterType":"PERCENT_PRICE"}],"orderTypes":["LIMIT","MARKET","STOP","STOP_MARKET","TAKE_PROFIT","TAKE_PROFIT_MARKET","TRAILING_STOP_MARKET"],"timeInForce":["GTC","IOC","FOK","GTX"]},{"symbol":"GTCUSDT","pair":"GTCUSDT","contractType":"PERPETUAL","deliveryDate":4133404800000,"onboardDate":1615791600000,"status":"TRADING","maintMarginPercent":"2.5000","requiredMarginPercent":"5.0000","baseAsset":"GTC","quoteAsset":"USDT","marginAsset":"USDT","pricePrecision":3,"quantityPrecision":1,"baseAssetPrecision":8,"quotePrecision":8,"underlyingType":"COIN","underlyingSubType":["Infrastructure"],"settlePlan":0,"triggerProtect":"0.1000","liquidationFee":"0.010000","marketTakeBound":"0.10","filters":[{"minPrice":"0.200","maxPrice":"100000","filterType":"PRICE_FILTER","tickSize":"0.001"},{"stepSize":"0.1","filterType":"LOT_SIZE","maxQty":"100000","minQty":"0.1"},{"stepSize":"0.1","filterType":"MARKET_LOT_SIZE","maxQty":"5000","minQty":"0.1"},{"limit":200,"filterType":"MAX_NUM_ORDERS"},{"limit":10,"filterType":"MAX_NUM_ALGO_ORDERS"},{"notional":"5","filterType":"MIN_NOTIONAL"},{"multiplierDown":"0.9000","multiplierUp":"1.1000","multiplierDecimal":"4","filterType":"PERCENT_PRICE"}],"orderTypes":["LIMIT","MARKET","STOP","STOP_MARKET","TAKE_PROFIT","TAKE_PROFIT_MARKET","TRAILING_STOP_MARKET"],"timeInForce":["GTC","IOC","FOK","GTX"]},{"symbol":"ETHBUSD","pair":"ETHBUSD","contractType":"PERPETUAL","deliveryDate":4133404800000,"onboardDate":1623744000000,"status":"TRADING","maintMarginPercent":"2.5000","requiredMarginPercent":"5.0000","baseAsset":"ETH","quoteAsset":"BUSD","marginAsset":"BUSD","pricePrecision":2,"quantityPrecision":3,"baseAssetPrecision":8,"quotePrecision":8,"underlyingType":"COIN","underlyingSubType":["BUSD"],"settlePlan":0,"triggerProtect":"0.0500","liquidationFee":"0.012500","marketTakeBound":"0.05","filters":[{"minPrice":"41.10","maxPrice":"306468","filterType":"PRICE_FILTER","tickSize":"0.01"},{"stepSize":"0.001","filterType":"LOT_SIZE","maxQty":"10000","minQty":"0.001"},{"stepSize":"0.001","filterType":"MARKET_LOT_SIZE","maxQty":"1000","minQty":"0.001"},{"limit":200,"filterType":"MAX_NUM_ORDERS"},{"limit":10,"filterType":"MAX_NUM_ALGO_ORDERS"},{"notional":"5","filterType":"MIN_NOTIONAL"},{"multiplierDown":"0.9500","multiplierUp":"1.0500","multiplierDecimal":"4","filterType":"PERCENT_PRICE"}],"orderTypes":["LIMIT","MARKET","STOP","STOP_MARKET","TAKE_PROFIT","TAKE_PROFIT_MARKET","TRAILING_STOP_MARKET"],"timeInForce":["GTC","IOC","FOK","GTX"]},{"symbol":"BTCDOMUSDT","pair":"BTCDOMUSDT","contractType":"PERPETUAL","deliveryDate":4133404800000,"onboardDate":1623913200000,"status":"TRADING","maintMarginPercent":"2.5000","requiredMarginPercent":"5.0000","baseAsset":"BTCDOM","quoteAsset":"USDT","marginAsset":"USDT","pricePrecision":1,"quantityPrecision":3,"baseAssetPrecision":8,"quotePrecision":8,"underlyingType":"INDEX","underlyingSubType":["Index"],"settlePlan":0,"triggerProtect":"0.1000","liquidationFee":"0.012500","marketTakeBound":"0.10","filters":[{"minPrice":"10","maxPrice":"100000","filterType":"PRICE_FILTER","tickSize":"0.1"},{"stepSize":"0.001","filterType":"LOT_SIZE","maxQty":"10000","minQty":"0.001"},{"stepSize":"0.001","filterType":"MARKET_LOT_SIZE","maxQty":"100","minQty":"0.001"},{"limit":200,"filterType":"MAX_NUM_ORDERS"},{"limit":10,"filterType":"MAX_NUM_ALGO_ORDERS"},{"notional":"5","filterType":"MIN_NOTIONAL"},{"multiplierDown":"0.9000","multiplierUp":"1.1000","multiplierDecimal":"4","filterType":"PERCENT_PRICE"}],"orderTypes":["LIMIT","MARKET","STOP","STOP_MARKET","TAKE_PROFIT","TAKE_PROFIT_MARKET","TRAILING_STOP_MARKET"],"timeInForce":["GTC","IOC","FOK","GTX"]},{"symbol":"TLMUSDT","pair":"TLMUSDT","contractType":"PERPETUAL","deliveryDate":1654765200000,"onboardDate":1626332400000,"status":"SETTLING","maintMarginPercent":"2.5000","requiredMarginPercent":"5.0000","baseAsset":"TLM","quoteAsset":"USDT","marginAsset":"USDT","pricePrecision":4,"quantityPrecision":0,"baseAssetPrecision":8,"quotePrecision":8,"underlyingType":"COIN","underlyingSubType":["Metaverse"],"settlePlan":0,"triggerProtect":"0.0500","liquidationFee":"0.025000","marketTakeBound":"0.05","filters":[{"minPrice":"0.0040","maxPrice":"100000","filterType":"PRICE_FILTER","tickSize":"0.0001"},{"stepSize":"1","filterType":"LOT_SIZE","maxQty":"1000000","minQty":"1"},{"stepSize":"1","filterType":"MARKET_LOT_SIZE","maxQty":"625000","minQty":"1"},{"limit":200,"filterType":"MAX_NUM_ORDERS"},{"limit":10,"filterType":"MAX_NUM_ALGO_ORDERS"},{"notional":"5","filterType":"MIN_NOTIONAL"},{"multiplierDown":"0.9500","multiplierUp":"1.0500","multiplierDecimal":"4","filterType":"PERCENT_PRICE"}],"orderTypes":["LIMIT","MARKET","STOP","STOP_MARKET","TAKE_PROFIT","TAKE_PROFIT_MARKET","TRAILING_STOP_MARKET"],"timeInForce":["GTC","IOC","FOK","GTX"]},{"symbol":"BNBBUSD","pair":"BNBBUSD","contractType":"PERPETUAL","deliveryDate":4133404800000,"onboardDate":1627372800000,"status":"TRADING","maintMarginPercent":"2.5000","requiredMarginPercent":"5.0000","baseAsset":"BNB","quoteAsset":"BUSD","marginAsset":"BUSD","pricePrecision":3,"quantityPrecision":2,"baseAssetPrecision":8,"quotePrecision":8,"underlyingType":"COIN","underlyingSubType":["BUSD"],"settlePlan":0,"triggerProtect":"0.0500","liquidationFee":"0.012500","marketTakeBound":"0.05","filters":[{"minPrice":"6.600","maxPrice":"100000","filterType":"PRICE_FILTER","tickSize":"0.010"},{"stepSize":"0.01","filterType":"LOT_SIZE","maxQty":"10000","minQty":"0.01"},{"stepSize":"0.01","filterType":"MARKET_LOT_SIZE","maxQty":"1000","minQty":"0.01"},{"limit":200,"filterType":"MAX_NUM_ORDERS"},{"limit":10,"filterType":"MAX_NUM_ALGO_ORDERS"},{"notional":"5","filterType":"MIN_NOTIONAL"},{"multiplierDown":"0.9500","multiplierUp":"1.0500","multiplierDecimal":"4","filterType":"PERCENT_PRICE"}],"orderTypes":["LIMIT","MARKET","STOP","STOP_MARKET","TAKE_PROFIT","TAKE_PROFIT_MARKET","TRAILING_STOP_MARKET"],"timeInForce":["GTC","IOC","FOK","GTX"]},{"symbol":"ADABUSD","pair":"ADABUSD","contractType":"PERPETUAL","deliveryDate":4133404800000,"onboardDate":1627372800000,"status":"TRADING","maintMarginPercent":"2.5000","requiredMarginPercent":"5.0000","baseAsset":"ADA","quoteAsset":"BUSD","marginAsset":"BUSD","pricePrecision":5,"quantityPrecision":0,"baseAssetPrecision":8,"quotePrecision":8,"underlyingType":"COIN","underlyingSubType":["BUSD"],"settlePlan":0,"triggerProtect":"0.0500","liquidationFee":"0.015000","marketTakeBound":"0.05","filters":[{"minPrice":"0.01740","maxPrice":"20000","filterType":"PRICE_FILTER","tickSize":"0.00010"},{"stepSize":"1","filterType":"LOT_SIZE","maxQty":"10000000","minQty":"1"},{"stepSize":"1","filterType":"MARKET_LOT_SIZE","maxQty":"25000","minQty":"1"},{"limit":200,"filterType":"MAX_NUM_ORDERS"},{"limit":10,"filterType":"MAX_NUM_ALGO_ORDERS"},{"notional":"5","filterType":"MIN_NOTIONAL"},{"multiplierDown":"0.9500","multiplierUp":"1.0500","multiplierDecimal":"4","filterType":"PERCENT_PRICE"}],"orderTypes":["LIMIT","MARKET","STOP","STOP_MARKET","TAKE_PROFIT","TAKE_PROFIT_MARKET","TRAILING_STOP_MARKET"],"timeInForce":["GTC","IOC","FOK","GTX"]},{"symbol":"XRPBUSD","pair":"XRPBUSD","contractType":"PERPETUAL","deliveryDate":4133404800000,"onboardDate":1627372800000,"status":"TRADING","maintMarginPercent":"2.5000","requiredMarginPercent":"5.0000","baseAsset":"XRP","quoteAsset":"BUSD","marginAsset":"BUSD","pricePrecision":4,"quantityPrecision":1,"baseAssetPrecision":8,"quotePrecision":8,"underlyingType":"COIN","underlyingSubType":["BUSD"],"settlePlan":0,"triggerProtect":"0.0500","liquidationFee":"0.010000","marketTakeBound":"0.05","filters":[{"minPrice":"0.0143","maxPrice":"100000","filterType":"PRICE_FILTER","tickSize":"0.0001"},{"stepSize":"0.1","filterType":"LOT_SIZE","maxQty":"4000000","minQty":"0.1"},{"stepSize":"0.1","filterType":"MARKET_LOT_SIZE","maxQty":"800000","minQty":"0.1"},{"limit":200,"filterType":"MAX_NUM_ORDERS"},{"limit":10,"filterType":"MAX_NUM_ALGO_ORDERS"},{"notional":"5","filterType":"MIN_NOTIONAL"},{"multiplierDown":"0.9500","multiplierUp":"1.0500","multiplierDecimal":"4","filterType":"PERCENT_PRICE"}],"orderTypes":["LIMIT","MARKET","STOP","STOP_MARKET","TAKE_PROFIT","TAKE_PROFIT_MARKET","TRAILING_STOP_MARKET"],"timeInForce":["GTC","IOC","FOK","GTX"]},{"symbol":"IOTXUSDT","pair":"IOTXUSDT","contractType":"PERPETUAL","deliveryDate":4133404800000,"onboardDate":1628665200000,"status":"TRADING","maintMarginPercent":"2.5000","requiredMarginPercent":"5.0000","baseAsset":"IOTX","quoteAsset":"USDT","marginAsset":"USDT","pricePrecision":5,"quantityPrecision":0,"baseAssetPrecision":8,"quotePrecision":8,"underlyingType":"COIN","underlyingSubType":["Layer-1"],"settlePlan":0,"triggerProtect":"0.1000","liquidationFee":"0.010000","marketTakeBound":"0.10","filters":[{"minPrice":"0.00050","maxPrice":"20000","filterType":"PRICE_FILTER","tickSize":"0.00001"},{"stepSize":"1","filterType":"LOT_SIZE","maxQty":"10000000","minQty":"1"},{"stepSize":"1","filterType":"MARKET_LOT_SIZE","maxQty":"500000","minQty":"1"},{"limit":200,"filterType":"MAX_NUM_ORDERS"},{"limit":10,"filterType":"MAX_NUM_ALGO_ORDERS"},{"notional":"5","filterType":"MIN_NOTIONAL"},{"multiplierDown":"0.9000","multiplierUp":"1.1000","multiplierDecimal":"4","filterType":"PERCENT_PRICE"}],"orderTypes":["LIMIT","MARKET","STOP","STOP_MARKET","TAKE_PROFIT","TAKE_PROFIT_MARKET","TRAILING_STOP_MARKET"],"timeInForce":["GTC","IOC","FOK","GTX"]},{"symbol":"DOGEBUSD","pair":"DOGEBUSD","contractType":"PERPETUAL","deliveryDate":4133404800000,"onboardDate":1627372800000,"status":"TRADING","maintMarginPercent":"2.5000","requiredMarginPercent":"5.0000","baseAsset":"DOGE","quoteAsset":"BUSD","marginAsset":"BUSD","pricePrecision":6,"quantityPrecision":0,"baseAssetPrecision":8,"quotePrecision":8,"underlyingType":"COIN","underlyingSubType":["BUSD"],"settlePlan":0,"triggerProtect":"0.1000","liquidationFee":"0.010000","marketTakeBound":"0.10","filters":[{"minPrice":"0.002440","maxPrice":"2000","filterType":"PRICE_FILTER","tickSize":"0.000010"},{"stepSize":"1","filterType":"LOT_SIZE","maxQty":"10000000","minQty":"1"},{"stepSize":"1","filterType":"MARKET_LOT_SIZE","maxQty":"5000000","minQty":"1"},{"limit":200,"filterType":"MAX_NUM_ORDERS"},{"limit":10,"filterType":"MAX_NUM_ALGO_ORDERS"},{"notional":"5","filterType":"MIN_NOTIONAL"},{"multiplierDown":"0.9000","multiplierUp":"1.1000","multiplierDecimal":"4","filterType":"PERCENT_PRICE"}],"orderTypes":["LIMIT","MARKET","STOP","STOP_MARKET","TAKE_PROFIT","TAKE_PROFIT_MARKET","TRAILING_STOP_MARKET"],"timeInForce":["GTC","IOC","FOK","GTX"]},{"symbol":"AUDIOUSDT","pair":"AUDIOUSDT","contractType":"PERPETUAL","deliveryDate":4133404800000,"onboardDate":1629270000000,"status":"TRADING","maintMarginPercent":"2.5000","requiredMarginPercent":"5.0000","baseAsset":"AUDIO","quoteAsset":"USDT","marginAsset":"USDT","pricePrecision":4,"quantityPrecision":0,"baseAssetPrecision":8,"quotePrecision":8,"underlyingType":"COIN","underlyingSubType":["NFT"],"settlePlan":0,"triggerProtect":"0.1000","liquidationFee":"0.010000","marketTakeBound":"0.10","filters":[{"minPrice":"0.0040","maxPrice":"100000","filterType":"PRICE_FILTER","tickSize":"0.0001"},{"stepSize":"1","filterType":"LOT_SIZE","maxQty":"1000000","minQty":"1"},{"stepSize":"1","filterType":"MARKET_LOT_SIZE","maxQty":"18000","minQty":"1"},{"limit":200,"filterType":"MAX_NUM_ORDERS"},{"limit":10,"filterType":"MAX_NUM_ALGO_ORDERS"},{"notional":"5","filterType":"MIN_NOTIONAL"},{"multiplierDown":"0.9000","multiplierUp":"1.1000","multiplierDecimal":"4","filterType":"PERCENT_PRICE"}],"orderTypes":["LIMIT","MARKET","STOP","STOP_MARKET","TAKE_PROFIT","TAKE_PROFIT_MARKET","TRAILING_STOP_MARKET"],"timeInForce":["GTC","IOC","FOK","GTX"]},{"symbol":"RAYUSDT","pair":"RAYUSDT","contractType":"PERPETUAL","deliveryDate":1668484800000,"onboardDate":1629270000000,"status":"SETTLING","maintMarginPercent":"2.5000","requiredMarginPercent":"5.0000","baseAsset":"RAY","quoteAsset":"USDT","marginAsset":"USDT","pricePrecision":3,"quantityPrecision":1,"baseAssetPrecision":8,"quotePrecision":8,"underlyingType":"COIN","underlyingSubType":["DEX"],"settlePlan":1668484800000,"triggerProtect":"0.1000","liquidationFee":"0.025000","marketTakeBound":"0.10","filters":[{"minPrice":"0.010","maxPrice":"100000","filterType":"PRICE_FILTER","tickSize":"0.001"},{"stepSize":"0.1","filterType":"LOT_SIZE","maxQty":"500000","minQty":"0.1"},{"stepSize":"0.1","filterType":"MARKET_LOT_SIZE","maxQty":"5000","minQty":"0.1"},{"limit":200,"filterType":"MAX_NUM_ORDERS"},{"limit":10,"filterType":"MAX_NUM_ALGO_ORDERS"},{"notional":"5","filterType":"MIN_NOTIONAL"},{"multiplierDown":"0.9000","multiplierUp":"1.1000","multiplierDecimal":"4","filterType":"PERCENT_PRICE"}],"orderTypes":["LIMIT","MARKET","STOP","STOP_MARKET","TAKE_PROFIT","TAKE_PROFIT_MARKET","TRAILING_STOP_MARKET"],"timeInForce":["GTC","IOC","FOK","GTX"]},{"symbol":"C98USDT","pair":"C98USDT","contractType":"PERPETUAL","deliveryDate":4133404800000,"onboardDate":1629702000000,"status":"TRADING","maintMarginPercent":"2.5000","requiredMarginPercent":"5.0000","baseAsset":"C98","quoteAsset":"USDT","marginAsset":"USDT","pricePrecision":4,"quantityPrecision":0,"baseAssetPrecision":8,"quotePrecision":8,"underlyingType":"COIN","underlyingSubType":["Wallet"],"settlePlan":0,"triggerProtect":"0.1000","liquidationFee":"0.010000","marketTakeBound":"0.10","filters":[{"minPrice":"0.0010","maxPrice":"100000","filterType":"PRICE_FILTER","tickSize":"0.0001"},{"stepSize":"1","filterType":"LOT_SIZE","maxQty":"250000","minQty":"1"},{"stepSize":"1","filterType":"MARKET_LOT_SIZE","maxQty":"30000","minQty":"1"},{"limit":200,"filterType":"MAX_NUM_ORDERS"},{"limit":10,"filterType":"MAX_NUM_ALGO_ORDERS"},{"notional":"5","filterType":"MIN_NOTIONAL"},{"multiplierDown":"0.9000","multiplierUp":"1.1000","multiplierDecimal":"4","filterType":"PERCENT_PRICE"}],"orderTypes":["LIMIT","MARKET","STOP","STOP_MARKET","TAKE_PROFIT","TAKE_PROFIT_MARKET","TRAILING_STOP_MARKET"],"timeInForce":["GTC","IOC","FOK","GTX"]},{"symbol":"MASKUSDT","pair":"MASKUSDT","contractType":"PERPETUAL","deliveryDate":4133404800000,"onboardDate":1626418800000,"status":"TRADING","maintMarginPercent":"2.5000","requiredMarginPercent":"5.0000","baseAsset":"MASK","quoteAsset":"USDT","marginAsset":"USDT","pricePrecision":4,"quantityPrecision":0,"baseAssetPrecision":8,"quotePrecision":8,"underlyingType":"COIN","underlyingSubType":["Privacy"],"settlePlan":0,"triggerProtect":"0.1000","liquidationFee":"0.020000","marketTakeBound":"0.10","filters":[{"minPrice":"0.0040","maxPrice":"100000","filterType":"PRICE_FILTER","tickSize":"0.0010"},{"stepSize":"1","filterType":"LOT_SIZE","maxQty":"5000000","minQty":"1"},{"stepSize":"1","filterType":"MARKET_LOT_SIZE","maxQty":"40000","minQty":"1"},{"limit":200,"filterType":"MAX_NUM_ORDERS"},{"limit":10,"filterType":"MAX_NUM_ALGO_ORDERS"},{"notional":"5","filterType":"MIN_NOTIONAL"},{"multiplierDown":"0.9000","multiplierUp":"1.1000","multiplierDecimal":"4","filterType":"PERCENT_PRICE"}],"orderTypes":["LIMIT","MARKET","STOP","STOP_MARKET","TAKE_PROFIT","TAKE_PROFIT_MARKET","TRAILING_STOP_MARKET"],"timeInForce":["GTC","IOC","FOK","GTX"]},{"symbol":"ATAUSDT","pair":"ATAUSDT","contractType":"PERPETUAL","deliveryDate":4133404800000,"onboardDate":1630306800000,"status":"TRADING","maintMarginPercent":"2.5000","requiredMarginPercent":"5.0000","baseAsset":"ATA","quoteAsset":"USDT","marginAsset":"USDT","pricePrecision":4,"quantityPrecision":0,"baseAssetPrecision":8,"quotePrecision":8,"underlyingType":"COIN","underlyingSubType":["Privacy"],"settlePlan":0,"triggerProtect":"0.1200","liquidationFee":"0.010000","marketTakeBound":"0.12","filters":[{"minPrice":"0.0040","maxPrice":"100000","filterType":"PRICE_FILTER","tickSize":"0.0001"},{"stepSize":"1","filterType":"LOT_SIZE","maxQty":"10000000","minQty":"1"},{"stepSize":"1","filterType":"MARKET_LOT_SIZE","maxQty":"1000000","minQty":"1"},{"limit":200,"filterType":"MAX_NUM_ORDERS"},{"limit":10,"filterType":"MAX_NUM_ALGO_ORDERS"},{"notional":"5","filterType":"MIN_NOTIONAL"},{"multiplierDown":"0.8800","multiplierUp":"1.1200","multiplierDecimal":"4","filterType":"PERCENT_PRICE"}],"orderTypes":["LIMIT","MARKET","STOP","STOP_MARKET","TAKE_PROFIT","TAKE_PROFIT_MARKET","TRAILING_STOP_MARKET"],"timeInForce":["GTC","IOC","FOK","GTX"]},{"symbol":"SOLBUSD","pair":"SOLBUSD","contractType":"PERPETUAL","deliveryDate":4133404800000,"onboardDate":1630566000000,"status":"TRADING","maintMarginPercent":"2.5000","requiredMarginPercent":"5.0000","baseAsset":"SOL","quoteAsset":"BUSD","marginAsset":"BUSD","pricePrecision":4,"quantityPrecision":0,"baseAssetPrecision":8,"quotePrecision":8,"underlyingType":"COIN","underlyingSubType":["BUSD"],"settlePlan":0,"triggerProtect":"0.0500","liquidationFee":"0.020000","marketTakeBound":"0.05","filters":[{"minPrice":"0.0400","maxPrice":"100000","filterType":"PRICE_FILTER","tickSize":"0.0010"},{"stepSize":"1","filterType":"LOT_SIZE","maxQty":"1000000","minQty":"1"},{"stepSize":"1","filterType":"MARKET_LOT_SIZE","maxQty":"1000","minQty":"1"},{"limit":200,"filterType":"MAX_NUM_ORDERS"},{"limit":10,"filterType":"MAX_NUM_ALGO_ORDERS"},{"notional":"5","filterType":"MIN_NOTIONAL"},{"multiplierDown":"0.9500","multiplierUp":"1.0500","multiplierDecimal":"4","filterType":"PERCENT_PRICE"}],"orderTypes":["LIMIT","MARKET","STOP","STOP_MARKET","TAKE_PROFIT","TAKE_PROFIT_MARKET","TRAILING_STOP_MARKET"],"timeInForce":["GTC","IOC","FOK","GTX"]},{"symbol":"FTTBUSD","pair":"FTTBUSD","contractType":"PERPETUAL","deliveryDate":1668400200000,"onboardDate":1630911600000,"status":"SETTLING","maintMarginPercent":"2.5000","requiredMarginPercent":"5.0000","baseAsset":"FTT","quoteAsset":"BUSD","marginAsset":"BUSD","pricePrecision":3,"quantityPrecision":1,"baseAssetPrecision":8,"quotePrecision":8,"underlyingType":"COIN","underlyingSubType":["BUSD"],"settlePlan":0,"triggerProtect":"0.0500","liquidationFee":"0.025000","marketTakeBound":"0.05","filters":[{"minPrice":"0.010","maxPrice":"100000","filterType":"PRICE_FILTER","tickSize":"0.001"},{"stepSize":"0.1","filterType":"LOT_SIZE","maxQty":"1000000","minQty":"0.1"},{"stepSize":"0.1","filterType":"MARKET_LOT_SIZE","maxQty":"1000","minQty":"0.1"},{"limit":200,"filterType":"MAX_NUM_ORDERS"},{"limit":10,"filterType":"MAX_NUM_ALGO_ORDERS"},{"notional":"5","filterType":"MIN_NOTIONAL"},{"multiplierDown":"0.9500","multiplierUp":"1.0500","multiplierDecimal":"4","filterType":"PERCENT_PRICE"}],"orderTypes":["LIMIT","MARKET","STOP","STOP_MARKET","TAKE_PROFIT","TAKE_PROFIT_MARKET","TRAILING_STOP_MARKET"],"timeInForce":["GTC","IOC","FOK","GTX"]},{"symbol":"DYDXUSDT","pair":"DYDXUSDT","contractType":"PERPETUAL","deliveryDate":4133404800000,"onboardDate":1631170800000,"status":"TRADING","maintMarginPercent":"2.5000","requiredMarginPercent":"5.0000","baseAsset":"DYDX","quoteAsset":"USDT","marginAsset":"USDT","pricePrecision":3,"quantityPrecision":1,"baseAssetPrecision":8,"quotePrecision":8,"underlyingType":"COIN","underlyingSubType":["DEX"],"settlePlan":0,"triggerProtect":"0.1000","liquidationFee":"0.015000","marketTakeBound":"0.10","filters":[{"minPrice":"0.010","maxPrice":"100000","filterType":"PRICE_FILTER","tickSize":"0.001"},{"stepSize":"0.1","filterType":"LOT_SIZE","maxQty":"1000000","minQty":"0.1"},{"stepSize":"0.1","filterType":"MARKET_LOT_SIZE","maxQty":"16800","minQty":"0.1"},{"limit":200,"filterType":"MAX_NUM_ORDERS"},{"limit":10,"filterType":"MAX_NUM_ALGO_ORDERS"},{"notional":"5","filterType":"MIN_NOTIONAL"},{"multiplierDown":"0.9000","multiplierUp":"1.1000","multiplierDecimal":"4","filterType":"PERCENT_PRICE"}],"orderTypes":["LIMIT","MARKET","STOP","STOP_MARKET","TAKE_PROFIT","TAKE_PROFIT_MARKET","TRAILING_STOP_MARKET"],"timeInForce":["GTC","IOC","FOK","GTX"]},{"symbol":"1000XECUSDT","pair":"1000XECUSDT","contractType":"PERPETUAL","deliveryDate":4133404800000,"onboardDate":1631775600000,"status":"TRADING","maintMarginPercent":"2.5000","requiredMarginPercent":"5.0000","baseAsset":"1000XEC","quoteAsset":"USDT","marginAsset":"USDT","pricePrecision":5,"quantityPrecision":0,"baseAssetPrecision":8,"quotePrecision":8,"underlyingType":"COIN","underlyingSubType":["PoW"],"settlePlan":0,"triggerProtect":"0.1500","liquidationFee":"0.010000","marketTakeBound":"0.15","filters":[{"minPrice":"0.00050","maxPrice":"20000","filterType":"PRICE_FILTER","tickSize":"0.00001"},{"stepSize":"1","filterType":"LOT_SIZE","maxQty":"50000000","minQty":"1"},{"stepSize":"1","filterType":"MARKET_LOT_SIZE","maxQty":"5000000","minQty":"1"},{"limit":200,"filterType":"MAX_NUM_ORDERS"},{"limit":10,"filterType":"MAX_NUM_ALGO_ORDERS"},{"notional":"5","filterType":"MIN_NOTIONAL"},{"multiplierDown":"0.8500","multiplierUp":"1.1500","multiplierDecimal":"4","filterType":"PERCENT_PRICE"}],"orderTypes":["LIMIT","MARKET","STOP","STOP_MARKET","TAKE_PROFIT","TAKE_PROFIT_MARKET","TRAILING_STOP_MARKET"],"timeInForce":["GTC","IOC","FOK","GTX"]},{"symbol":"GALAUSDT","pair":"GALAUSDT","contractType":"PERPETUAL","deliveryDate":4133404800000,"onboardDate":1631862000000,"status":"TRADING","maintMarginPercent":"2.5000","requiredMarginPercent":"5.0000","baseAsset":"GALA","quoteAsset":"USDT","marginAsset":"USDT","pricePrecision":5,"quantityPrecision":0,"baseAssetPrecision":8,"quotePrecision":8,"underlyingType":"COIN","underlyingSubType":["Metaverse"],"settlePlan":0,"triggerProtect":"0.1000","liquidationFee":"0.010000","marketTakeBound":"0.10","filters":[{"minPrice":"0.00050","maxPrice":"20000","filterType":"PRICE_FILTER","tickSize":"0.00001"},{"stepSize":"1","filterType":"LOT_SIZE","maxQty":"10000000","minQty":"1"},{"stepSize":"1","filterType":"MARKET_LOT_SIZE","maxQty":"1000000","minQty":"1"},{"limit":200,"filterType":"MAX_NUM_ORDERS"},{"limit":10,"filterType":"MAX_NUM_ALGO_ORDERS"},{"notional":"5","filterType":"MIN_NOTIONAL"},{"multiplierDown":"0.9000","multiplierUp":"1.1000","multiplierDecimal":"4","filterType":"PERCENT_PRICE"}],"orderTypes":["LIMIT","MARKET","STOP","STOP_MARKET","TAKE_PROFIT","TAKE_PROFIT_MARKET","TRAILING_STOP_MARKET"],"timeInForce":["GTC","IOC","FOK","GTX"]},{"symbol":"CELOUSDT","pair":"CELOUSDT","contractType":"PERPETUAL","deliveryDate":4133404800000,"onboardDate":1632639600000,"status":"TRADING","maintMarginPercent":"2.5000","requiredMarginPercent":"5.0000","baseAsset":"CELO","quoteAsset":"USDT","marginAsset":"USDT","pricePrecision":3,"quantityPrecision":1,"baseAssetPrecision":8,"quotePrecision":8,"underlyingType":"COIN","underlyingSubType":["Layer-1"],"settlePlan":0,"triggerProtect":"0.0500","liquidationFee":"0.015000","marketTakeBound":"0.05","filters":[{"minPrice":"0.010","maxPrice":"100000","filterType":"PRICE_FILTER","tickSize":"0.001"},{"stepSize":"0.1","filterType":"LOT_SIZE","maxQty":"1000000","minQty":"0.1"},{"stepSize":"0.1","filterType":"MARKET_LOT_SIZE","maxQty":"100000","minQty":"0.1"},{"limit":200,"filterType":"MAX_NUM_ORDERS"},{"limit":10,"filterType":"MAX_NUM_ALGO_ORDERS"},{"notional":"5","filterType":"MIN_NOTIONAL"},{"multiplierDown":"0.9500","multiplierUp":"1.0500","multiplierDecimal":"4","filterType":"PERCENT_PRICE"}],"orderTypes":["LIMIT","MARKET","STOP","STOP_MARKET","TAKE_PROFIT","TAKE_PROFIT_MARKET","TRAILING_STOP_MARKET"],"timeInForce":["GTC","IOC","FOK","GTX"]},{"symbol":"ARUSDT","pair":"ARUSDT","contractType":"PERPETUAL","deliveryDate":4133404800000,"onboardDate":1632812400000,"status":"TRADING","maintMarginPercent":"2.5000","requiredMarginPercent":"5.0000","baseAsset":"AR","quoteAsset":"USDT","marginAsset":"USDT","pricePrecision":3,"quantityPrecision":1,"baseAssetPrecision":8,"quotePrecision":8,"underlyingType":"COIN","underlyingSubType":["Storage"],"settlePlan":0,"triggerProtect":"0.1000","liquidationFee":"0.012500","marketTakeBound":"0.10","filters":[{"minPrice":"0.010","maxPrice":"100000","filterType":"PRICE_FILTER","tickSize":"0.001"},{"stepSize":"0.1","filterType":"LOT_SIZE","maxQty":"1000000","minQty":"0.1"},{"stepSize":"0.1","filterType":"MARKET_LOT_SIZE","maxQty":"1500","minQty":"0.1"},{"limit":200,"filterType":"MAX_NUM_ORDERS"},{"limit":10,"filterType":"MAX_NUM_ALGO_ORDERS"},{"notional":"5","filterType":"MIN_NOTIONAL"},{"multiplierDown":"0.9000","multiplierUp":"1.1000","multiplierDecimal":"4","filterType":"PERCENT_PRICE"}],"orderTypes":["LIMIT","MARKET","STOP","STOP_MARKET","TAKE_PROFIT","TAKE_PROFIT_MARKET","TRAILING_STOP_MARKET"],"timeInForce":["GTC","IOC","FOK","GTX"]},{"symbol":"KLAYUSDT","pair":"KLAYUSDT","contractType":"PERPETUAL","deliveryDate":4133404800000,"onboardDate":1633935600000,"status":"TRADING","maintMarginPercent":"2.5000","requiredMarginPercent":"5.0000","baseAsset":"KLAY","quoteAsset":"USDT","marginAsset":"USDT","pricePrecision":4,"quantityPrecision":1,"baseAssetPrecision":8,"quotePrecision":8,"underlyingType":"COIN","underlyingSubType":["Layer-1"],"settlePlan":0,"triggerProtect":"0.0500","liquidationFee":"0.015000","marketTakeBound":"0.05","filters":[{"minPrice":"0.0010","maxPrice":"100000","filterType":"PRICE_FILTER","tickSize":"0.0001"},{"stepSize":"0.1","filterType":"LOT_SIZE","maxQty":"5000000","minQty":"0.1"},{"stepSize":"0.1","filterType":"MARKET_LOT_SIZE","maxQty":"700000","minQty":"0.1"},{"limit":200,"filterType":"MAX_NUM_ORDERS"},{"limit":10,"filterType":"MAX_NUM_ALGO_ORDERS"},{"notional":"5","filterType":"MIN_NOTIONAL"},{"multiplierDown":"0.9500","multiplierUp":"1.0500","multiplierDecimal":"4","filterType":"PERCENT_PRICE"}],"orderTypes":["LIMIT","MARKET","STOP","STOP_MARKET","TAKE_PROFIT","TAKE_PROFIT_MARKET","TRAILING_STOP_MARKET"],"timeInForce":["GTC","IOC","FOK","GTX"]},{"symbol":"ARPAUSDT","pair":"ARPAUSDT","contractType":"PERPETUAL","deliveryDate":4133404800000,"onboardDate":1634540400000,"status":"TRADING","maintMarginPercent":"2.5000","requiredMarginPercent":"5.0000","baseAsset":"ARPA","quoteAsset":"USDT","marginAsset":"USDT","pricePrecision":5,"quantityPrecision":0,"baseAssetPrecision":8,"quotePrecision":8,"underlyingType":"COIN","underlyingSubType":["Privacy"],"settlePlan":0,"triggerProtect":"0.1000","liquidationFee":"0.010000","marketTakeBound":"0.10","filters":[{"minPrice":"0.00050","maxPrice":"20000","filterType":"PRICE_FILTER","tickSize":"0.00001"},{"stepSize":"1","filterType":"LOT_SIZE","maxQty":"10000000","minQty":"1"},{"stepSize":"1","filterType":"MARKET_LOT_SIZE","maxQty":"5000000","minQty":"1"},{"limit":200,"filterType":"MAX_NUM_ORDERS"},{"limit":10,"filterType":"MAX_NUM_ALGO_ORDERS"},{"notional":"5","filterType":"MIN_NOTIONAL"},{"multiplierDown":"0.9000","multiplierUp":"1.1000","multiplierDecimal":"4","filterType":"PERCENT_PRICE"}],"orderTypes":["LIMIT","MARKET","STOP","STOP_MARKET","TAKE_PROFIT","TAKE_PROFIT_MARKET","TRAILING_STOP_MARKET"],"timeInForce":["GTC","IOC","FOK","GTX"]},{"symbol":"CTSIUSDT","pair":"CTSIUSDT","contractType":"PERPETUAL","deliveryDate":4133404800000,"onboardDate":1635145200000,"status":"TRADING","maintMarginPercent":"2.5000","requiredMarginPercent":"5.0000","baseAsset":"CTSI","quoteAsset":"USDT","marginAsset":"USDT","pricePrecision":4,"quantityPrecision":0,"baseAssetPrecision":8,"quotePrecision":8,"underlyingType":"COIN","underlyingSubType":["Layer-2"],"settlePlan":0,"triggerProtect":"0.1000","liquidationFee":"0.015000","marketTakeBound":"0.10","filters":[{"minPrice":"0.0010","maxPrice":"100000","filterType":"PRICE_FILTER","tickSize":"0.0001"},{"stepSize":"1","filterType":"LOT_SIZE","maxQty":"1000000","minQty":"1"},{"stepSize":"1","filterType":"MARKET_LOT_SIZE","maxQty":"700000","minQty":"1"},{"limit":200,"filterType":"MAX_NUM_ORDERS"},{"limit":10,"filterType":"MAX_NUM_ALGO_ORDERS"},{"notional":"5","filterType":"MIN_NOTIONAL"},{"multiplierDown":"0.9000","multiplierUp":"1.1000","multiplierDecimal":"4","filterType":"PERCENT_PRICE"}],"orderTypes":["LIMIT","MARKET","STOP","STOP_MARKET","TAKE_PROFIT","TAKE_PROFIT_MARKET","TRAILING_STOP_MARKET"],"timeInForce":["GTC","IOC","FOK","GTX"]},{"symbol":"LPTUSDT","pair":"LPTUSDT","contractType":"PERPETUAL","deliveryDate":4133404800000,"onboardDate":1636527600000,"status":"TRADING","maintMarginPercent":"2.5000","requiredMarginPercent":"5.0000","baseAsset":"LPT","quoteAsset":"USDT","marginAsset":"USDT","pricePrecision":3,"quantityPrecision":1,"baseAssetPrecision":8,"quotePrecision":8,"underlyingType":"COIN","underlyingSubType":["Infrastructure"],"settlePlan":0,"triggerProtect":"0.1000","liquidationFee":"0.010000","marketTakeBound":"0.10","filters":[{"minPrice":"0.010","maxPrice":"100000","filterType":"PRICE_FILTER","tickSize":"0.001"},{"stepSize":"0.1","filterType":"LOT_SIZE","maxQty":"1000000","minQty":"0.1"},{"stepSize":"0.1","filterType":"MARKET_LOT_SIZE","maxQty":"1000","minQty":"0.1"},{"limit":200,"filterType":"MAX_NUM_ORDERS"},{"limit":10,"filterType":"MAX_NUM_ALGO_ORDERS"},{"notional":"5","filterType":"MIN_NOTIONAL"},{"multiplierDown":"0.9000","multiplierUp":"1.1000","multiplierDecimal":"4","filterType":"PERCENT_PRICE"}],"orderTypes":["LIMIT","MARKET","STOP","STOP_MARKET","TAKE_PROFIT","TAKE_PROFIT_MARKET","TRAILING_STOP_MARKET"],"timeInForce":["GTC","IOC","FOK","GTX"]},{"symbol":"ENSUSDT","pair":"ENSUSDT","contractType":"PERPETUAL","deliveryDate":4133404800000,"onboardDate":1638169200000,"status":"TRADING","maintMarginPercent":"2.5000","requiredMarginPercent":"5.0000","baseAsset":"ENS","quoteAsset":"USDT","marginAsset":"USDT","pricePrecision":3,"quantityPrecision":1,"baseAssetPrecision":8,"quotePrecision":8,"underlyingType":"COIN","underlyingSubType":["DAO"],"settlePlan":0,"triggerProtect":"0.1000","liquidationFee":"0.015000","marketTakeBound":"0.10","filters":[{"minPrice":"0.010","maxPrice":"100000","filterType":"PRICE_FILTER","tickSize":"0.001"},{"stepSize":"0.1","filterType":"LOT_SIZE","maxQty":"1000000","minQty":"0.1"},{"stepSize":"0.1","filterType":"MARKET_LOT_SIZE","maxQty":"1000","minQty":"0.1"},{"limit":200,"filterType":"MAX_NUM_ORDERS"},{"limit":10,"filterType":"MAX_NUM_ALGO_ORDERS"},{"notional":"5","filterType":"MIN_NOTIONAL"},{"multiplierDown":"0.9000","multiplierUp":"1.1000","multiplierDecimal":"4","filterType":"PERCENT_PRICE"}],"orderTypes":["LIMIT","MARKET","STOP","STOP_MARKET","TAKE_PROFIT","TAKE_PROFIT_MARKET","TRAILING_STOP_MARKET"],"timeInForce":["GTC","IOC","FOK","GTX"]},{"symbol":"PEOPLEUSDT","pair":"PEOPLEUSDT","contractType":"PERPETUAL","deliveryDate":4133404800000,"onboardDate":1640242800000,"status":"TRADING","maintMarginPercent":"2.5000","requiredMarginPercent":"5.0000","baseAsset":"PEOPLE","quoteAsset":"USDT","marginAsset":"USDT","pricePrecision":5,"quantityPrecision":0,"baseAssetPrecision":8,"quotePrecision":8,"underlyingType":"COIN","underlyingSubType":["DAO"],"settlePlan":0,"triggerProtect":"0.1000","liquidationFee":"0.010000","marketTakeBound":"0.10","filters":[{"minPrice":"0.00010","maxPrice":"100000","filterType":"PRICE_FILTER","tickSize":"0.00001"},{"stepSize":"1","filterType":"LOT_SIZE","maxQty":"10000000","minQty":"1"},{"stepSize":"1","filterType":"MARKET_LOT_SIZE","maxQty":"100000","minQty":"1"},{"limit":200,"filterType":"MAX_NUM_ORDERS"},{"limit":10,"filterType":"MAX_NUM_ALGO_ORDERS"},{"notional":"5","filterType":"MIN_NOTIONAL"},{"multiplierDown":"0.9000","multiplierUp":"1.1000","multiplierDecimal":"4","filterType":"PERCENT_PRICE"}],"orderTypes":["LIMIT","MARKET","STOP","STOP_MARKET","TAKE_PROFIT","TAKE_PROFIT_MARKET","TRAILING_STOP_MARKET"],"timeInForce":["GTC","IOC","FOK","GTX"]},{"symbol":"ANTUSDT","pair":"ANTUSDT","contractType":"PERPETUAL","deliveryDate":4133404800000,"onboardDate":1640329200000,"status":"TRADING","maintMarginPercent":"2.5000","requiredMarginPercent":"5.0000","baseAsset":"ANT","quoteAsset":"USDT","marginAsset":"USDT","pricePrecision":3,"quantityPrecision":1,"baseAssetPrecision":8,"quotePrecision":8,"underlyingType":"COIN","underlyingSubType":["DAO"],"settlePlan":0,"triggerProtect":"0.0500","liquidationFee":"0.020000","marketTakeBound":"0.05","filters":[{"minPrice":"0.010","maxPrice":"100000","filterType":"PRICE_FILTER","tickSize":"0.001"},{"stepSize":"0.1","filterType":"LOT_SIZE","maxQty":"1000000","minQty":"0.1"},{"stepSize":"0.1","filterType":"MARKET_LOT_SIZE","maxQty":"1000","minQty":"0.1"},{"limit":200,"filterType":"MAX_NUM_ORDERS"},{"limit":10,"filterType":"MAX_NUM_ALGO_ORDERS"},{"notional":"5","filterType":"MIN_NOTIONAL"},{"multiplierDown":"0.9500","multiplierUp":"1.0500","multiplierDecimal":"4","filterType":"PERCENT_PRICE"}],"orderTypes":["LIMIT","MARKET","STOP","STOP_MARKET","TAKE_PROFIT","TAKE_PROFIT_MARKET","TRAILING_STOP_MARKET"],"timeInForce":["GTC","IOC","FOK","GTX"]},{"symbol":"ROSEUSDT","pair":"ROSEUSDT","contractType":"PERPETUAL","deliveryDate":4133404800000,"onboardDate":1640934000000,"status":"TRADING","maintMarginPercent":"2.5000","requiredMarginPercent":"5.0000","baseAsset":"ROSE","quoteAsset":"USDT","marginAsset":"USDT","pricePrecision":5,"quantityPrecision":0,"baseAssetPrecision":8,"quotePrecision":8,"underlyingType":"COIN","underlyingSubType":["DeFi"],"settlePlan":0,"triggerProtect":"0.1000","liquidationFee":"0.010000","marketTakeBound":"0.10","filters":[{"minPrice":"0.00010","maxPrice":"10000","filterType":"PRICE_FILTER","tickSize":"0.00001"},{"stepSize":"1","filterType":"LOT_SIZE","maxQty":"10000000","minQty":"1"},{"stepSize":"1","filterType":"MARKET_LOT_SIZE","maxQty":"100000","minQty":"1"},{"limit":200,"filterType":"MAX_NUM_ORDERS"},{"limit":10,"filterType":"MAX_NUM_ALGO_ORDERS"},{"notional":"5","filterType":"MIN_NOTIONAL"},{"multiplierDown":"0.9000","multiplierUp":"1.1000","multiplierDecimal":"4","filterType":"PERCENT_PRICE"}],"orderTypes":["LIMIT","MARKET","STOP","STOP_MARKET","TAKE_PROFIT","TAKE_PROFIT_MARKET","TRAILING_STOP_MARKET"],"timeInForce":["GTC","IOC","FOK","GTX"]},{"symbol":"DUSKUSDT","pair":"DUSKUSDT","contractType":"PERPETUAL","deliveryDate":4133404800000,"onboardDate":1641452400000,"status":"TRADING","maintMarginPercent":"2.5000","requiredMarginPercent":"5.0000","baseAsset":"DUSK","quoteAsset":"USDT","marginAsset":"USDT","pricePrecision":5,"quantityPrecision":0,"baseAssetPrecision":8,"quotePrecision":8,"underlyingType":"COIN","underlyingSubType":["Privacy"],"settlePlan":0,"triggerProtect":"0.1000","liquidationFee":"0.015000","marketTakeBound":"0.10","filters":[{"minPrice":"0.00010","maxPrice":"10000","filterType":"PRICE_FILTER","tickSize":"0.00001"},{"stepSize":"1","filterType":"LOT_SIZE","maxQty":"10000000","minQty":"1"},{"stepSize":"1","filterType":"MARKET_LOT_SIZE","maxQty":"20000","minQty":"1"},{"limit":200,"filterType":"MAX_NUM_ORDERS"},{"limit":10,"filterType":"MAX_NUM_ALGO_ORDERS"},{"notional":"5","filterType":"MIN_NOTIONAL"},{"multiplierDown":"0.9000","multiplierUp":"1.1000","multiplierDecimal":"4","filterType":"PERCENT_PRICE"}],"orderTypes":["LIMIT","MARKET","STOP","STOP_MARKET","TAKE_PROFIT","TAKE_PROFIT_MARKET","TRAILING_STOP_MARKET"],"timeInForce":["GTC","IOC","FOK","GTX"]},{"symbol":"FLOWUSDT","pair":"FLOWUSDT","contractType":"PERPETUAL","deliveryDate":4133404800000,"onboardDate":1644390000000,"status":"TRADING","maintMarginPercent":"2.5000","requiredMarginPercent":"5.0000","baseAsset":"FLOW","quoteAsset":"USDT","marginAsset":"USDT","pricePrecision":3,"quantityPrecision":1,"baseAssetPrecision":8,"quotePrecision":8,"underlyingType":"COIN","underlyingSubType":["NFT"],"settlePlan":0,"triggerProtect":"0.1000","liquidationFee":"0.010000","marketTakeBound":"0.10","filters":[{"minPrice":"0.010","maxPrice":"100000","filterType":"PRICE_FILTER","tickSize":"0.001"},{"stepSize":"0.1","filterType":"LOT_SIZE","maxQty":"1000000","minQty":"0.1"},{"stepSize":"0.1","filterType":"MARKET_LOT_SIZE","maxQty":"6000","minQty":"0.1"},{"limit":200,"filterType":"MAX_NUM_ORDERS"},{"limit":10,"filterType":"MAX_NUM_ALGO_ORDERS"},{"notional":"5","filterType":"MIN_NOTIONAL"},{"multiplierDown":"0.9000","multiplierUp":"1.1000","multiplierDecimal":"4","filterType":"PERCENT_PRICE"}],"orderTypes":["LIMIT","MARKET","STOP","STOP_MARKET","TAKE_PROFIT","TAKE_PROFIT_MARKET","TRAILING_STOP_MARKET"],"timeInForce":["GTC","IOC","FOK","GTX"]},{"symbol":"IMXUSDT","pair":"IMXUSDT","contractType":"PERPETUAL","deliveryDate":4133404800000,"onboardDate":1644476400000,"status":"TRADING","maintMarginPercent":"2.5000","requiredMarginPercent":"5.0000","baseAsset":"IMX","quoteAsset":"USDT","marginAsset":"USDT","pricePrecision":4,"quantityPrecision":0,"baseAssetPrecision":8,"quotePrecision":8,"underlyingType":"COIN","underlyingSubType":["NFT"],"settlePlan":0,"triggerProtect":"0.1000","liquidationFee":"0.010000","marketTakeBound":"0.10","filters":[{"minPrice":"0.0010","maxPrice":"100000","filterType":"PRICE_FILTER","tickSize":"0.0001"},{"stepSize":"1","filterType":"LOT_SIZE","maxQty":"1000000","minQty":"1"},{"stepSize":"1","filterType":"MARKET_LOT_SIZE","maxQty":"20000","minQty":"1"},{"limit":200,"filterType":"MAX_NUM_ORDERS"},{"limit":10,"filterType":"MAX_NUM_ALGO_ORDERS"},{"notional":"5","filterType":"MIN_NOTIONAL"},{"multiplierDown":"0.9000","multiplierUp":"1.1000","multiplierDecimal":"4","filterType":"PERCENT_PRICE"}],"orderTypes":["LIMIT","MARKET","STOP","STOP_MARKET","TAKE_PROFIT","TAKE_PROFIT_MARKET","TRAILING_STOP_MARKET"],"timeInForce":["GTC","IOC","FOK","GTX"]},{"symbol":"API3USDT","pair":"API3USDT","contractType":"PERPETUAL","deliveryDate":4133404800000,"onboardDate":1645426800000,"status":"TRADING","maintMarginPercent":"2.5000","requiredMarginPercent":"5.0000","baseAsset":"API3","quoteAsset":"USDT","marginAsset":"USDT","pricePrecision":4,"quantityPrecision":1,"baseAssetPrecision":8,"quotePrecision":8,"underlyingType":"COIN","underlyingSubType":["DeFi"],"settlePlan":0,"triggerProtect":"0.1000","liquidationFee":"0.015000","marketTakeBound":"0.10","filters":[{"minPrice":"0.0010","maxPrice":"10000","filterType":"PRICE_FILTER","tickSize":"0.0010"},{"stepSize":"0.1","filterType":"LOT_SIZE","maxQty":"1000000","minQty":"0.1"},{"stepSize":"0.1","filterType":"MARKET_LOT_SIZE","maxQty":"100000","minQty":"0.1"},{"limit":200,"filterType":"MAX_NUM_ORDERS"},{"limit":10,"filterType":"MAX_NUM_ALGO_ORDERS"},{"notional":"5","filterType":"MIN_NOTIONAL"},{"multiplierDown":"0.9000","multiplierUp":"1.1000","multiplierDecimal":"4","filterType":"PERCENT_PRICE"}],"orderTypes":["LIMIT","MARKET","STOP","STOP_MARKET","TAKE_PROFIT","TAKE_PROFIT_MARKET","TRAILING_STOP_MARKET"],"timeInForce":["GTC","IOC","FOK","GTX"]},{"symbol":"GMTUSDT","pair":"GMTUSDT","contractType":"PERPETUAL","deliveryDate":4133404800000,"onboardDate":1647241200000,"status":"TRADING","maintMarginPercent":"2.5000","requiredMarginPercent":"5.0000","baseAsset":"GMT","quoteAsset":"USDT","marginAsset":"USDT","pricePrecision":5,"quantityPrecision":0,"baseAssetPrecision":8,"quotePrecision":8,"underlyingType":"COIN","underlyingSubType":["Metaverse"],"settlePlan":0,"triggerProtect":"0.1000","liquidationFee":"0.010000","marketTakeBound":"0.10","filters":[{"minPrice":"0.00010","maxPrice":"10000","filterType":"PRICE_FILTER","tickSize":"0.00010"},{"stepSize":"1","filterType":"LOT_SIZE","maxQty":"10000000","minQty":"1"},{"stepSize":"1","filterType":"MARKET_LOT_SIZE","maxQty":"150000","minQty":"1"},{"limit":200,"filterType":"MAX_NUM_ORDERS"},{"limit":10,"filterType":"MAX_NUM_ALGO_ORDERS"},{"notional":"5","filterType":"MIN_NOTIONAL"},{"multiplierDown":"0.9000","multiplierUp":"1.1000","multiplierDecimal":"4","filterType":"PERCENT_PRICE"}],"orderTypes":["LIMIT","MARKET","STOP","STOP_MARKET","TAKE_PROFIT","TAKE_PROFIT_MARKET","TRAILING_STOP_MARKET"],"timeInForce":["GTC","IOC","FOK","GTX"]},{"symbol":"APEUSDT","pair":"APEUSDT","contractType":"PERPETUAL","deliveryDate":4133404800000,"onboardDate":1647500400000,"status":"TRADING","maintMarginPercent":"2.5000","requiredMarginPercent":"5.0000","baseAsset":"APE","quoteAsset":"USDT","marginAsset":"USDT","pricePrecision":4,"quantityPrecision":0,"baseAssetPrecision":8,"quotePrecision":8,"underlyingType":"COIN","underlyingSubType":["NFT"],"settlePlan":0,"triggerProtect":"0.1000","liquidationFee":"0.015000","marketTakeBound":"0.10","filters":[{"minPrice":"0.0010","maxPrice":"100000","filterType":"PRICE_FILTER","tickSize":"0.0010"},{"stepSize":"1","filterType":"LOT_SIZE","maxQty":"1000000","minQty":"1"},{"stepSize":"1","filterType":"MARKET_LOT_SIZE","maxQty":"6000","minQty":"1"},{"limit":200,"filterType":"MAX_NUM_ORDERS"},{"limit":10,"filterType":"MAX_NUM_ALGO_ORDERS"},{"notional":"5","filterType":"MIN_NOTIONAL"},{"multiplierDown":"0.9000","multiplierUp":"1.1000","multiplierDecimal":"4","filterType":"PERCENT_PRICE"}],"orderTypes":["LIMIT","MARKET","STOP","STOP_MARKET","TAKE_PROFIT","TAKE_PROFIT_MARKET","TRAILING_STOP_MARKET"],"timeInForce":["GTC","IOC","FOK","GTX"]},{"symbol":"BNXUSDT","pair":"BNXUSDT","contractType":"PERPETUAL","deliveryDate":4133404800000,"onboardDate":1648710000000,"status":"TRADING","maintMarginPercent":"2.5000","requiredMarginPercent":"5.0000","baseAsset":"BNX","quoteAsset":"USDT","marginAsset":"USDT","pricePrecision":4,"quantityPrecision":1,"baseAssetPrecision":8,"quotePrecision":8,"underlyingType":"COIN","underlyingSubType":["NFT"],"settlePlan":0,"triggerProtect":"0.0500","liquidationFee":"0.025000","marketTakeBound":"0.05","filters":[{"minPrice":"0.0100","maxPrice":"100000","filterType":"PRICE_FILTER","tickSize":"0.0100"},{"stepSize":"0.1","filterType":"LOT_SIZE","maxQty":"1000000","minQty":"0.1"},{"stepSize":"0.1","filterType":"MARKET_LOT_SIZE","maxQty":"85","minQty":"0.1"},{"limit":200,"filterType":"MAX_NUM_ORDERS"},{"limit":10,"filterType":"MAX_NUM_ALGO_ORDERS"},{"notional":"5","filterType":"MIN_NOTIONAL"},{"multiplierDown":"0.9500","multiplierUp":"1.0500","multiplierDecimal":"4","filterType":"PERCENT_PRICE"}],"orderTypes":["LIMIT","MARKET","STOP","STOP_MARKET","TAKE_PROFIT","TAKE_PROFIT_MARKET","TRAILING_STOP_MARKET"],"timeInForce":["GTC","IOC","FOK","GTX"]},{"symbol":"WOOUSDT","pair":"WOOUSDT","contractType":"PERPETUAL","deliveryDate":4133404800000,"onboardDate":1649314800000,"status":"TRADING","maintMarginPercent":"2.5000","requiredMarginPercent":"5.0000","baseAsset":"WOO","quoteAsset":"USDT","marginAsset":"USDT","pricePrecision":5,"quantityPrecision":0,"baseAssetPrecision":8,"quotePrecision":8,"underlyingType":"COIN","underlyingSubType":["CEX"],"settlePlan":0,"triggerProtect":"0.1000","liquidationFee":"0.010000","marketTakeBound":"0.10","filters":[{"minPrice":"0.00010","maxPrice":"10000","filterType":"PRICE_FILTER","tickSize":"0.00001"},{"stepSize":"1","filterType":"LOT_SIZE","maxQty":"10000000","minQty":"1"},{"stepSize":"1","filterType":"MARKET_LOT_SIZE","maxQty":"100000","minQty":"1"},{"limit":200,"filterType":"MAX_NUM_ORDERS"},{"limit":10,"filterType":"MAX_NUM_ALGO_ORDERS"},{"notional":"5","filterType":"MIN_NOTIONAL"},{"multiplierDown":"0.9000","multiplierUp":"1.1000","multiplierDecimal":"4","filterType":"PERCENT_PRICE"}],"orderTypes":["LIMIT","MARKET","STOP","STOP_MARKET","TAKE_PROFIT","TAKE_PROFIT_MARKET","TRAILING_STOP_MARKET"],"timeInForce":["GTC","IOC","FOK","GTX"]},{"symbol":"FTTUSDT","pair":"FTTUSDT","contractType":"PERPETUAL","deliveryDate":1668398400000,"onboardDate":1649919600000,"status":"SETTLING","maintMarginPercent":"2.5000","requiredMarginPercent":"5.0000","baseAsset":"FTT","quoteAsset":"USDT","marginAsset":"USDT","pricePrecision":4,"quantityPrecision":1,"baseAssetPrecision":8,"quotePrecision":8,"underlyingType":"COIN","underlyingSubType":["CEX"],"settlePlan":0,"triggerProtect":"0.0500","liquidationFee":"0.025000","marketTakeBound":"0.05","filters":[{"minPrice":"0.0010","maxPrice":"100000","filterType":"PRICE_FILTER","tickSize":"0.0001"},{"stepSize":"0.1","filterType":"LOT_SIZE","maxQty":"1000000","minQty":"0.1"},{"stepSize":"0.1","filterType":"MARKET_LOT_SIZE","maxQty":"1000","minQty":"0.1"},{"limit":200,"filterType":"MAX_NUM_ORDERS"},{"limit":10,"filterType":"MAX_NUM_ALGO_ORDERS"},{"notional":"5","filterType":"MIN_NOTIONAL"},{"multiplierDown":"0.9500","multiplierUp":"1.0500","multiplierDecimal":"4","filterType":"PERCENT_PRICE"}],"orderTypes":["LIMIT","MARKET","STOP","STOP_MARKET","TAKE_PROFIT","TAKE_PROFIT_MARKET","TRAILING_STOP_MARKET"],"timeInForce":["GTC","IOC","FOK","GTX"]},{"symbol":"JASMYUSDT","pair":"JASMYUSDT","contractType":"PERPETUAL","deliveryDate":4133404800000,"onboardDate":1650351600000,"status":"TRADING","maintMarginPercent":"2.5000","requiredMarginPercent":"5.0000","baseAsset":"JASMY","quoteAsset":"USDT","marginAsset":"USDT","pricePrecision":6,"quantityPrecision":0,"baseAssetPrecision":8,"quotePrecision":8,"underlyingType":"COIN","underlyingSubType":["Infrastructure"],"settlePlan":0,"triggerProtect":"0.1000","liquidationFee":"0.010000","marketTakeBound":"0.10","filters":[{"minPrice":"0.000010","maxPrice":"10000","filterType":"PRICE_FILTER","tickSize":"0.000001"},{"stepSize":"1","filterType":"LOT_SIZE","maxQty":"10000000","minQty":"1"},{"stepSize":"1","filterType":"MARKET_LOT_SIZE","maxQty":"2500000","minQty":"1"},{"limit":200,"filterType":"MAX_NUM_ORDERS"},{"limit":10,"filterType":"MAX_NUM_ALGO_ORDERS"},{"notional":"5","filterType":"MIN_NOTIONAL"},{"multiplierDown":"0.9000","multiplierUp":"1.1000","multiplierDecimal":"4","filterType":"PERCENT_PRICE"}],"orderTypes":["LIMIT","MARKET","STOP","STOP_MARKET","TAKE_PROFIT","TAKE_PROFIT_MARKET","TRAILING_STOP_MARKET"],"timeInForce":["GTC","IOC","FOK","GTX"]},{"symbol":"DARUSDT","pair":"DARUSDT","contractType":"PERPETUAL","deliveryDate":4133404800000,"onboardDate":1651129200000,"status":"TRADING","maintMarginPercent":"2.5000","requiredMarginPercent":"5.0000","baseAsset":"DAR","quoteAsset":"USDT","marginAsset":"USDT","pricePrecision":4,"quantityPrecision":1,"baseAssetPrecision":8,"quotePrecision":8,"underlyingType":"COIN","underlyingSubType":["NFT"],"settlePlan":0,"triggerProtect":"0.1000","liquidationFee":"0.010000","marketTakeBound":"0.10","filters":[{"minPrice":"0.0010","maxPrice":"100000","filterType":"PRICE_FILTER","tickSize":"0.0001"},{"stepSize":"0.1","filterType":"LOT_SIZE","maxQty":"1000000","minQty":"0.1"},{"stepSize":"0.1","filterType":"MARKET_LOT_SIZE","maxQty":"30000","minQty":"0.1"},{"limit":200,"filterType":"MAX_NUM_ORDERS"},{"limit":10,"filterType":"MAX_NUM_ALGO_ORDERS"},{"notional":"5","filterType":"MIN_NOTIONAL"},{"multiplierDown":"0.9000","multiplierUp":"1.1000","multiplierDecimal":"4","filterType":"PERCENT_PRICE"}],"orderTypes":["LIMIT","MARKET","STOP","STOP_MARKET","TAKE_PROFIT","TAKE_PROFIT_MARKET","TRAILING_STOP_MARKET"],"timeInForce":["GTC","IOC","FOK","GTX"]},{"symbol":"GALUSDT","pair":"GALUSDT","contractType":"PERPETUAL","deliveryDate":4133404800000,"onboardDate":1651734000000,"status":"TRADING","maintMarginPercent":"2.5000","requiredMarginPercent":"5.0000","baseAsset":"GAL","quoteAsset":"USDT","marginAsset":"USDT","pricePrecision":5,"quantityPrecision":0,"baseAssetPrecision":8,"quotePrecision":8,"underlyingType":"COIN","underlyingSubType":["Infrastructure"],"settlePlan":0,"triggerProtect":"0.1000","liquidationFee":"0.010000","marketTakeBound":"0.10","filters":[{"minPrice":"0.00010","maxPrice":"10000","filterType":"PRICE_FILTER","tickSize":"0.00010"},{"stepSize":"1","filterType":"LOT_SIZE","maxQty":"10000000","minQty":"1"},{"stepSize":"1","filterType":"MARKET_LOT_SIZE","maxQty":"4000","minQty":"1"},{"limit":200,"filterType":"MAX_NUM_ORDERS"},{"limit":10,"filterType":"MAX_NUM_ALGO_ORDERS"},{"notional":"5","filterType":"MIN_NOTIONAL"},{"multiplierDown":"0.9000","multiplierUp":"1.1000","multiplierDecimal":"4","filterType":"PERCENT_PRICE"}],"orderTypes":["LIMIT","MARKET","STOP","STOP_MARKET","TAKE_PROFIT","TAKE_PROFIT_MARKET","TRAILING_STOP_MARKET"],"timeInForce":["GTC","IOC","FOK","GTX"]},{"symbol":"AVAXBUSD","pair":"AVAXBUSD","contractType":"PERPETUAL","deliveryDate":4133404800000,"onboardDate":1652425200000,"status":"TRADING","maintMarginPercent":"2.5000","requiredMarginPercent":"5.0000","baseAsset":"AVAX","quoteAsset":"BUSD","marginAsset":"BUSD","pricePrecision":6,"quantityPrecision":1,"baseAssetPrecision":8,"quotePrecision":8,"underlyingType":"COIN","underlyingSubType":["BUSD"],"settlePlan":0,"triggerProtect":"0.0500","liquidationFee":"0.012500","marketTakeBound":"0.05","filters":[{"minPrice":"0.001000","maxPrice":"100000","filterType":"PRICE_FILTER","tickSize":"0.001000"},{"stepSize":"0.1","filterType":"LOT_SIZE","maxQty":"1000000","minQty":"0.1"},{"stepSize":"0.1","filterType":"MARKET_LOT_SIZE","maxQty":"500","minQty":"0.1"},{"limit":200,"filterType":"MAX_NUM_ORDERS"},{"limit":10,"filterType":"MAX_NUM_ALGO_ORDERS"},{"notional":"5","filterType":"MIN_NOTIONAL"},{"multiplierDown":"0.9500","multiplierUp":"1.0500","multiplierDecimal":"4","filterType":"PERCENT_PRICE"}],"orderTypes":["LIMIT","MARKET","STOP","STOP_MARKET","TAKE_PROFIT","TAKE_PROFIT_MARKET","TRAILING_STOP_MARKET"],"timeInForce":["GTC","IOC","FOK","GTX"]},{"symbol":"NEARBUSD","pair":"NEARBUSD","contractType":"PERPETUAL","deliveryDate":4133404800000,"onboardDate":1652684400000,"status":"TRADING","maintMarginPercent":"2.5000","requiredMarginPercent":"5.0000","baseAsset":"NEAR","quoteAsset":"BUSD","marginAsset":"BUSD","pricePrecision":7,"quantityPrecision":1,"baseAssetPrecision":8,"quotePrecision":8,"underlyingType":"COIN","underlyingSubType":["BUSD"],"settlePlan":0,"triggerProtect":"0.0500","liquidationFee":"0.010000","marketTakeBound":"0.05","filters":[{"minPrice":"0.0010000","maxPrice":"100000","filterType":"PRICE_FILTER","tickSize":"0.0010000"},{"stepSize":"0.1","filterType":"LOT_SIZE","maxQty":"1000000","minQty":"0.1"},{"stepSize":"0.1","filterType":"MARKET_LOT_SIZE","maxQty":"9000","minQty":"0.1"},{"limit":200,"filterType":"MAX_NUM_ORDERS"},{"limit":10,"filterType":"MAX_NUM_ALGO_ORDERS"},{"notional":"5","filterType":"MIN_NOTIONAL"},{"multiplierDown":"0.9500","multiplierUp":"1.0500","multiplierDecimal":"4","filterType":"PERCENT_PRICE"}],"orderTypes":["LIMIT","MARKET","STOP","STOP_MARKET","TAKE_PROFIT","TAKE_PROFIT_MARKET","TRAILING_STOP_MARKET"],"timeInForce":["GTC","IOC","FOK","GTX"]},{"symbol":"GMTBUSD","pair":"GMTBUSD","contractType":"PERPETUAL","deliveryDate":4133404800000,"onboardDate":1652684400000,"status":"TRADING","maintMarginPercent":"2.5000","requiredMarginPercent":"5.0000","baseAsset":"GMT","quoteAsset":"BUSD","marginAsset":"BUSD","pricePrecision":7,"quantityPrecision":1,"baseAssetPrecision":8,"quotePrecision":8,"underlyingType":"COIN","underlyingSubType":["BUSD"],"settlePlan":0,"triggerProtect":"0.1000","liquidationFee":"0.010000","marketTakeBound":"0.10","filters":[{"minPrice":"0.0001000","maxPrice":"100000","filterType":"PRICE_FILTER","tickSize":"0.0001000"},{"stepSize":"0.1","filterType":"LOT_SIZE","maxQty":"1000000","minQty":"0.1"},{"stepSize":"0.1","filterType":"MARKET_LOT_SIZE","maxQty":"32000","minQty":"0.1"},{"limit":200,"filterType":"MAX_NUM_ORDERS"},{"limit":10,"filterType":"MAX_NUM_ALGO_ORDERS"},{"notional":"5","filterType":"MIN_NOTIONAL"},{"multiplierDown":"0.9000","multiplierUp":"1.1000","multiplierDecimal":"4","filterType":"PERCENT_PRICE"}],"orderTypes":["LIMIT","MARKET","STOP","STOP_MARKET","TAKE_PROFIT","TAKE_PROFIT_MARKET","TRAILING_STOP_MARKET"],"timeInForce":["GTC","IOC","FOK","GTX"]},{"symbol":"APEBUSD","pair":"APEBUSD","contractType":"PERPETUAL","deliveryDate":4133404800000,"onboardDate":1652684400000,"status":"TRADING","maintMarginPercent":"2.5000","requiredMarginPercent":"5.0000","baseAsset":"APE","quoteAsset":"BUSD","marginAsset":"BUSD","pricePrecision":7,"quantityPrecision":1,"baseAssetPrecision":8,"quotePrecision":8,"underlyingType":"COIN","underlyingSubType":["BUSD"],"settlePlan":0,"triggerProtect":"0.1000","liquidationFee":"0.010000","marketTakeBound":"0.10","filters":[{"minPrice":"0.0010000","maxPrice":"100000","filterType":"PRICE_FILTER","tickSize":"0.0010000"},{"stepSize":"0.1","filterType":"LOT_SIZE","maxQty":"1000000","minQty":"0.1"},{"stepSize":"0.1","filterType":"MARKET_LOT_SIZE","maxQty":"8000","minQty":"0.1"},{"limit":200,"filterType":"MAX_NUM_ORDERS"},{"limit":10,"filterType":"MAX_NUM_ALGO_ORDERS"},{"notional":"5","filterType":"MIN_NOTIONAL"},{"multiplierDown":"0.9000","multiplierUp":"1.1000","multiplierDecimal":"4","filterType":"PERCENT_PRICE"}],"orderTypes":["LIMIT","MARKET","STOP","STOP_MARKET","TAKE_PROFIT","TAKE_PROFIT_MARKET","TRAILING_STOP_MARKET"],"timeInForce":["GTC","IOC","FOK","GTX"]},{"symbol":"GALBUSD","pair":"GALBUSD","contractType":"PERPETUAL","deliveryDate":4133404800000,"onboardDate":1653289200000,"status":"TRADING","maintMarginPercent":"2.5000","requiredMarginPercent":"5.0000","baseAsset":"GAL","quoteAsset":"BUSD","marginAsset":"BUSD","pricePrecision":7,"quantityPrecision":0,"baseAssetPrecision":8,"quotePrecision":8,"underlyingType":"COIN","underlyingSubType":["BUSD"],"settlePlan":0,"triggerProtect":"0.0500","liquidationFee":"0.010000","marketTakeBound":"0.05","filters":[{"minPrice":"0.0010000","maxPrice":"100000","filterType":"PRICE_FILTER","tickSize":"0.0010000"},{"stepSize":"1","filterType":"LOT_SIZE","maxQty":"1000000","minQty":"1"},{"stepSize":"1","filterType":"MARKET_LOT_SIZE","maxQty":"10000","minQty":"1"},{"limit":200,"filterType":"MAX_NUM_ORDERS"},{"limit":10,"filterType":"MAX_NUM_ALGO_ORDERS"},{"notional":"5","filterType":"MIN_NOTIONAL"},{"multiplierDown":"0.9500","multiplierUp":"1.0500","multiplierDecimal":"4","filterType":"PERCENT_PRICE"}],"orderTypes":["LIMIT","MARKET","STOP","STOP_MARKET","TAKE_PROFIT","TAKE_PROFIT_MARKET","TRAILING_STOP_MARKET"],"timeInForce":["GTC","IOC","FOK","GTX"]},{"symbol":"FTMBUSD","pair":"FTMBUSD","contractType":"PERPETUAL","deliveryDate":4133404800000,"onboardDate":1653289200000,"status":"TRADING","maintMarginPercent":"2.5000","requiredMarginPercent":"5.0000","baseAsset":"FTM","quoteAsset":"BUSD","marginAsset":"BUSD","pricePrecision":7,"quantityPrecision":0,"baseAssetPrecision":8,"quotePrecision":8,"underlyingType":"COIN","underlyingSubType":["BUSD"],"settlePlan":0,"triggerProtect":"0.0500","liquidationFee":"0.010000","marketTakeBound":"0.05","filters":[{"minPrice":"0.0001000","maxPrice":"100000","filterType":"PRICE_FILTER","tickSize":"0.0001000"},{"stepSize":"1","filterType":"LOT_SIZE","maxQty":"10000000","minQty":"1"},{"stepSize":"1","filterType":"MARKET_LOT_SIZE","maxQty":"120000","minQty":"1"},{"limit":200,"filterType":"MAX_NUM_ORDERS"},{"limit":10,"filterType":"MAX_NUM_ALGO_ORDERS"},{"notional":"5","filterType":"MIN_NOTIONAL"},{"multiplierDown":"0.9500","multiplierUp":"1.0500","multiplierDecimal":"4","filterType":"PERCENT_PRICE"}],"orderTypes":["LIMIT","MARKET","STOP","STOP_MARKET","TAKE_PROFIT","TAKE_PROFIT_MARKET","TRAILING_STOP_MARKET"],"timeInForce":["GTC","IOC","FOK","GTX"]},{"symbol":"DODOBUSD","pair":"DODOBUSD","contractType":"PERPETUAL","deliveryDate":4133404800000,"onboardDate":1653375600000,"status":"TRADING","maintMarginPercent":"2.5000","requiredMarginPercent":"5.0000","baseAsset":"DODO","quoteAsset":"BUSD","marginAsset":"BUSD","pricePrecision":7,"quantityPrecision":0,"baseAssetPrecision":8,"quotePrecision":8,"underlyingType":"COIN","underlyingSubType":["BUSD"],"settlePlan":0,"triggerProtect":"0.0500","liquidationFee":"0.010000","marketTakeBound":"0.05","filters":[{"minPrice":"0.0001000","maxPrice":"100000","filterType":"PRICE_FILTER","tickSize":"0.0001000"},{"stepSize":"1","filterType":"LOT_SIZE","maxQty":"10000000","minQty":"1"},{"stepSize":"1","filterType":"MARKET_LOT_SIZE","maxQty":"300000","minQty":"1"},{"limit":200,"filterType":"MAX_NUM_ORDERS"},{"limit":10,"filterType":"MAX_NUM_ALGO_ORDERS"},{"notional":"5","filterType":"MIN_NOTIONAL"},{"multiplierDown":"0.9500","multiplierUp":"1.0500","multiplierDecimal":"4","filterType":"PERCENT_PRICE"}],"orderTypes":["LIMIT","MARKET","STOP","STOP_MARKET","TAKE_PROFIT","TAKE_PROFIT_MARKET","TRAILING_STOP_MARKET"],"timeInForce":["GTC","IOC","FOK","GTX"]},{"symbol":"ANCBUSD","pair":"ANCBUSD","contractType":"PERPETUAL","deliveryDate":1671094800000,"onboardDate":1653375600000,"status":"SETTLING","maintMarginPercent":"2.5000","requiredMarginPercent":"5.0000","baseAsset":"ANC","quoteAsset":"BUSD","marginAsset":"BUSD","pricePrecision":7,"quantityPrecision":0,"baseAssetPrecision":8,"quotePrecision":8,"underlyingType":"COIN","underlyingSubType":["BUSD"],"settlePlan":1671094800000,"triggerProtect":"0.1000","liquidationFee":"0.025000","marketTakeBound":"0.10","filters":[{"minPrice":"0.0001000","maxPrice":"100000","filterType":"PRICE_FILTER","tickSize":"0.0000100"},{"stepSize":"1","filterType":"LOT_SIZE","maxQty":"10000000","minQty":"1"},{"stepSize":"1","filterType":"MARKET_LOT_SIZE","maxQty":"150000","minQty":"1"},{"limit":200,"filterType":"MAX_NUM_ORDERS"},{"limit":10,"filterType":"MAX_NUM_ALGO_ORDERS"},{"notional":"5","filterType":"MIN_NOTIONAL"},{"multiplierDown":"0.9000","multiplierUp":"1.1000","multiplierDecimal":"4","filterType":"PERCENT_PRICE"}],"orderTypes":["LIMIT","MARKET","STOP","STOP_MARKET","TAKE_PROFIT","TAKE_PROFIT_MARKET","TRAILING_STOP_MARKET"],"timeInForce":["GTC","IOC","FOK","GTX"]},{"symbol":"GALABUSD","pair":"GALABUSD","contractType":"PERPETUAL","deliveryDate":4133404800000,"onboardDate":1653462000000,"status":"TRADING","maintMarginPercent":"2.5000","requiredMarginPercent":"5.0000","baseAsset":"GALA","quoteAsset":"BUSD","marginAsset":"BUSD","pricePrecision":7,"quantityPrecision":0,"baseAssetPrecision":8,"quotePrecision":8,"underlyingType":"COIN","underlyingSubType":["BUSD"],"settlePlan":0,"triggerProtect":"0.1000","liquidationFee":"0.010000","marketTakeBound":"0.10","filters":[{"minPrice":"0.0000010","maxPrice":"100000","filterType":"PRICE_FILTER","tickSize":"0.0000010"},{"stepSize":"1","filterType":"LOT_SIZE","maxQty":"100000000","minQty":"1"},{"stepSize":"1","filterType":"MARKET_LOT_SIZE","maxQty":"600000","minQty":"1"},{"limit":200,"filterType":"MAX_NUM_ORDERS"},{"limit":10,"filterType":"MAX_NUM_ALGO_ORDERS"},{"notional":"5","filterType":"MIN_NOTIONAL"},{"multiplierDown":"0.9000","multiplierUp":"1.1000","multiplierDecimal":"4","filterType":"PERCENT_PRICE"}],"orderTypes":["LIMIT","MARKET","STOP","STOP_MARKET","TAKE_PROFIT","TAKE_PROFIT_MARKET","TRAILING_STOP_MARKET"],"timeInForce":["GTC","IOC","FOK","GTX"]},{"symbol":"TRXBUSD","pair":"TRXBUSD","contractType":"PERPETUAL","deliveryDate":4133404800000,"onboardDate":1653462000000,"status":"TRADING","maintMarginPercent":"2.5000","requiredMarginPercent":"5.0000","baseAsset":"TRX","quoteAsset":"BUSD","marginAsset":"BUSD","pricePrecision":7,"quantityPrecision":0,"baseAssetPrecision":8,"quotePrecision":8,"underlyingType":"COIN","underlyingSubType":["BUSD"],"settlePlan":0,"triggerProtect":"0.0500","liquidationFee":"0.010000","marketTakeBound":"0.05","filters":[{"minPrice":"0.0000010","maxPrice":"10000","filterType":"PRICE_FILTER","tickSize":"0.0000010"},{"stepSize":"1","filterType":"LOT_SIZE","maxQty":"100000000","minQty":"1"},{"stepSize":"1","filterType":"MARKET_LOT_SIZE","maxQty":"600000","minQty":"1"},{"limit":200,"filterType":"MAX_NUM_ORDERS"},{"limit":10,"filterType":"MAX_NUM_ALGO_ORDERS"},{"notional":"5","filterType":"MIN_NOTIONAL"},{"multiplierDown":"0.9500","multiplierUp":"1.0500","multiplierDecimal":"4","filterType":"PERCENT_PRICE"}],"orderTypes":["LIMIT","MARKET","STOP","STOP_MARKET","TAKE_PROFIT","TAKE_PROFIT_MARKET","TRAILING_STOP_MARKET"],"timeInForce":["GTC","IOC","FOK","GTX"]},{"symbol":"1000LUNCBUSD","pair":"1000LUNCBUSD","contractType":"PERPETUAL","deliveryDate":4133404800000,"onboardDate":1653894000000,"status":"TRADING","maintMarginPercent":"2.5000","requiredMarginPercent":"5.0000","baseAsset":"1000LUNC","quoteAsset":"BUSD","marginAsset":"BUSD","pricePrecision":7,"quantityPrecision":0,"baseAssetPrecision":8,"quotePrecision":8,"underlyingType":"COIN","underlyingSubType":["BUSD"],"settlePlan":0,"triggerProtect":"0.1000","liquidationFee":"0.015000","marketTakeBound":"0.10","filters":[{"minPrice":"0.0001000","maxPrice":"100000","filterType":"PRICE_FILTER","tickSize":"0.0001000"},{"stepSize":"1","filterType":"LOT_SIZE","maxQty":"10000000","minQty":"1"},{"stepSize":"1","filterType":"MARKET_LOT_SIZE","maxQty":"2000000","minQty":"1"},{"limit":200,"filterType":"MAX_NUM_ORDERS"},{"limit":10,"filterType":"MAX_NUM_ALGO_ORDERS"},{"notional":"5","filterType":"MIN_NOTIONAL"},{"multiplierDown":"0.9000","multiplierUp":"1.1000","multiplierDecimal":"4","filterType":"PERCENT_PRICE"}],"orderTypes":["LIMIT","MARKET","STOP","STOP_MARKET","TAKE_PROFIT","TAKE_PROFIT_MARKET","TRAILING_STOP_MARKET"],"timeInForce":["GTC","IOC","FOK","GTX"]},{"symbol":"LUNA2BUSD","pair":"LUNA2BUSD","contractType":"PERPETUAL","deliveryDate":4133404800000,"onboardDate":1653980400000,"status":"TRADING","maintMarginPercent":"2.5000","requiredMarginPercent":"5.0000","baseAsset":"LUNA2","quoteAsset":"BUSD","marginAsset":"BUSD","pricePrecision":7,"quantityPrecision":0,"baseAssetPrecision":8,"quotePrecision":8,"underlyingType":"COIN","underlyingSubType":["BUSD"],"settlePlan":0,"triggerProtect":"0.1000","liquidationFee":"0.020000","marketTakeBound":"0.10","filters":[{"minPrice":"0.0001000","maxPrice":"100000","filterType":"PRICE_FILTER","tickSize":"0.0001000"},{"stepSize":"1","filterType":"LOT_SIZE","maxQty":"100000000","minQty":"1"},{"stepSize":"1","filterType":"MARKET_LOT_SIZE","maxQty":"50000","minQty":"1"},{"limit":200,"filterType":"MAX_NUM_ORDERS"},{"limit":10,"filterType":"MAX_NUM_ALGO_ORDERS"},{"notional":"5","filterType":"MIN_NOTIONAL"},{"multiplierDown":"0.9000","multiplierUp":"1.1000","multiplierDecimal":"4","filterType":"PERCENT_PRICE"}],"orderTypes":["LIMIT","MARKET","STOP","STOP_MARKET","TAKE_PROFIT","TAKE_PROFIT_MARKET","TRAILING_STOP_MARKET"],"timeInForce":["GTC","IOC","FOK","GTX"]},{"symbol":"OPUSDT","pair":"OPUSDT","contractType":"PERPETUAL","deliveryDate":4133404800000,"onboardDate":1654066800000,"status":"TRADING","maintMarginPercent":"2.5000","requiredMarginPercent":"5.0000","baseAsset":"OP","quoteAsset":"USDT","marginAsset":"USDT","pricePrecision":7,"quantityPrecision":1,"baseAssetPrecision":8,"quotePrecision":8,"underlyingType":"COIN","underlyingSubType":["Layer-2"],"settlePlan":0,"triggerProtect":"0.1000","liquidationFee":"0.010000","marketTakeBound":"0.10","filters":[{"minPrice":"0.0001000","maxPrice":"100000","filterType":"PRICE_FILTER","tickSize":"0.0001000"},{"stepSize":"0.1","filterType":"LOT_SIZE","maxQty":"10000000","minQty":"0.1"},{"stepSize":"0.1","filterType":"MARKET_LOT_SIZE","maxQty":"50000","minQty":"0.1"},{"limit":200,"filterType":"MAX_NUM_ORDERS"},{"limit":10,"filterType":"MAX_NUM_ALGO_ORDERS"},{"notional":"5","filterType":"MIN_NOTIONAL"},{"multiplierDown":"0.9000","multiplierUp":"1.1000","multiplierDecimal":"4","filterType":"PERCENT_PRICE"}],"orderTypes":["LIMIT","MARKET","STOP","STOP_MARKET","TAKE_PROFIT","TAKE_PROFIT_MARKET","TRAILING_STOP_MARKET"],"timeInForce":["GTC","IOC","FOK","GTX"]},{"symbol":"DOTBUSD","pair":"DOTBUSD","contractType":"PERPETUAL","deliveryDate":4133404800000,"onboardDate":1654585200000,"status":"TRADING","maintMarginPercent":"2.5000","requiredMarginPercent":"5.0000","baseAsset":"DOT","quoteAsset":"BUSD","marginAsset":"BUSD","pricePrecision":7,"quantityPrecision":1,"baseAssetPrecision":8,"quotePrecision":8,"underlyingType":"COIN","underlyingSubType":["BUSD"],"settlePlan":0,"triggerProtect":"0.0500","liquidationFee":"0.010000","marketTakeBound":"0.05","filters":[{"minPrice":"0.0010000","maxPrice":"100000","filterType":"PRICE_FILTER","tickSize":"0.0010000"},{"stepSize":"0.1","filterType":"LOT_SIZE","maxQty":"10000000","minQty":"0.1"},{"stepSize":"0.1","filterType":"MARKET_LOT_SIZE","maxQty":"6000","minQty":"0.1"},{"limit":200,"filterType":"MAX_NUM_ORDERS"},{"limit":10,"filterType":"MAX_NUM_ALGO_ORDERS"},{"notional":"5","filterType":"MIN_NOTIONAL"},{"multiplierDown":"0.9500","multiplierUp":"1.0500","multiplierDecimal":"4","filterType":"PERCENT_PRICE"}],"orderTypes":["LIMIT","MARKET","STOP","STOP_MARKET","TAKE_PROFIT","TAKE_PROFIT_MARKET","TRAILING_STOP_MARKET"],"timeInForce":["GTC","IOC","FOK","GTX"]},{"symbol":"TLMBUSD","pair":"TLMBUSD","contractType":"PERPETUAL","deliveryDate":4133404800000,"onboardDate":1654585200000,"status":"TRADING","maintMarginPercent":"2.5000","requiredMarginPercent":"5.0000","baseAsset":"TLM","quoteAsset":"BUSD","marginAsset":"BUSD","pricePrecision":7,"quantityPrecision":0,"baseAssetPrecision":8,"quotePrecision":8,"underlyingType":"COIN","underlyingSubType":["BUSD"],"settlePlan":0,"triggerProtect":"0.0500","liquidationFee":"0.010000","marketTakeBound":"0.05","filters":[{"minPrice":"0.0000100","maxPrice":"100000","filterType":"PRICE_FILTER","tickSize":"0.0000100"},{"stepSize":"1","filterType":"LOT_SIZE","maxQty":"100000000","minQty":"1"},{"stepSize":"1","filterType":"MARKET_LOT_SIZE","maxQty":"1600000","minQty":"1"},{"limit":200,"filterType":"MAX_NUM_ORDERS"},{"limit":10,"filterType":"MAX_NUM_ALGO_ORDERS"},{"notional":"5","filterType":"MIN_NOTIONAL"},{"multiplierDown":"0.9500","multiplierUp":"1.0500","multiplierDecimal":"4","filterType":"PERCENT_PRICE"}],"orderTypes":["LIMIT","MARKET","STOP","STOP_MARKET","TAKE_PROFIT","TAKE_PROFIT_MARKET","TRAILING_STOP_MARKET"],"timeInForce":["GTC","IOC","FOK","GTX"]},{"symbol":"ICPBUSD","pair":"ICPBUSD","contractType":"PERPETUAL","deliveryDate":4133404800000,"onboardDate":1654585200000,"status":"TRADING","maintMarginPercent":"2.5000","requiredMarginPercent":"5.0000","baseAsset":"ICP","quoteAsset":"BUSD","marginAsset":"BUSD","pricePrecision":7,"quantityPrecision":1,"baseAssetPrecision":8,"quotePrecision":8,"underlyingType":"COIN","underlyingSubType":["BUSD"],"settlePlan":0,"triggerProtect":"0.0500","liquidationFee":"0.010000","marketTakeBound":"0.05","filters":[{"minPrice":"0.0010000","maxPrice":"100000","filterType":"PRICE_FILTER","tickSize":"0.0010000"},{"stepSize":"0.1","filterType":"LOT_SIZE","maxQty":"10000000","minQty":"0.1"},{"stepSize":"0.1","filterType":"MARKET_LOT_SIZE","maxQty":"8000","minQty":"0.1"},{"limit":200,"filterType":"MAX_NUM_ORDERS"},{"limit":10,"filterType":"MAX_NUM_ALGO_ORDERS"},{"notional":"5","filterType":"MIN_NOTIONAL"},{"multiplierDown":"0.9500","multiplierUp":"1.0500","multiplierDecimal":"4","filterType":"PERCENT_PRICE"}],"orderTypes":["LIMIT","MARKET","STOP","STOP_MARKET","TAKE_PROFIT","TAKE_PROFIT_MARKET","TRAILING_STOP_MARKET"],"timeInForce":["GTC","IOC","FOK","GTX"]},{"symbol":"WAVESBUSD","pair":"WAVESBUSD","contractType":"PERPETUAL","deliveryDate":4133404800000,"onboardDate":1656313200000,"status":"TRADING","maintMarginPercent":"2.5000","requiredMarginPercent":"5.0000","baseAsset":"WAVES","quoteAsset":"BUSD","marginAsset":"BUSD","pricePrecision":7,"quantityPrecision":1,"baseAssetPrecision":8,"quotePrecision":8,"underlyingType":"COIN","underlyingSubType":["BUSD"],"settlePlan":0,"triggerProtect":"0.0500","liquidationFee":"0.015000","marketTakeBound":"0.05","filters":[{"minPrice":"0.0010000","maxPrice":"100000","filterType":"PRICE_FILTER","tickSize":"0.0001000"},{"stepSize":"0.1","filterType":"LOT_SIZE","maxQty":"10000000","minQty":"0.1"},{"stepSize":"0.1","filterType":"MARKET_LOT_SIZE","maxQty":"9500","minQty":"0.1"},{"limit":200,"filterType":"MAX_NUM_ORDERS"},{"limit":10,"filterType":"MAX_NUM_ALGO_ORDERS"},{"notional":"5","filterType":"MIN_NOTIONAL"},{"multiplierDown":"0.9500","multiplierUp":"1.0500","multiplierDecimal":"4","filterType":"PERCENT_PRICE"}],"orderTypes":["LIMIT","MARKET","STOP","STOP_MARKET","TAKE_PROFIT","TAKE_PROFIT_MARKET","TRAILING_STOP_MARKET"],"timeInForce":["GTC","IOC","FOK","GTX"]},{"symbol":"LINKBUSD","pair":"LINKBUSD","contractType":"PERPETUAL","deliveryDate":4133404800000,"onboardDate":1656313200000,"status":"TRADING","maintMarginPercent":"2.5000","requiredMarginPercent":"5.0000","baseAsset":"LINK","quoteAsset":"BUSD","marginAsset":"BUSD","pricePrecision":7,"quantityPrecision":1,"baseAssetPrecision":8,"quotePrecision":8,"underlyingType":"COIN","underlyingSubType":["BUSD"],"settlePlan":0,"triggerProtect":"0.0500","liquidationFee":"0.010000","marketTakeBound":"0.05","filters":[{"minPrice":"0.0010000","maxPrice":"100000","filterType":"PRICE_FILTER","tickSize":"0.0010000"},{"stepSize":"0.1","filterType":"LOT_SIZE","maxQty":"10000000","minQty":"0.1"},{"stepSize":"0.1","filterType":"MARKET_LOT_SIZE","maxQty":"10000","minQty":"0.1"},{"limit":200,"filterType":"MAX_NUM_ORDERS"},{"limit":10,"filterType":"MAX_NUM_ALGO_ORDERS"},{"notional":"5","filterType":"MIN_NOTIONAL"},{"multiplierDown":"0.9500","multiplierUp":"1.0500","multiplierDecimal":"4","filterType":"PERCENT_PRICE"}],"orderTypes":["LIMIT","MARKET","STOP","STOP_MARKET","TAKE_PROFIT","TAKE_PROFIT_MARKET","TRAILING_STOP_MARKET"],"timeInForce":["GTC","IOC","FOK","GTX"]},{"symbol":"SANDBUSD","pair":"SANDBUSD","contractType":"PERPETUAL","deliveryDate":4133404800000,"onboardDate":1657090800000,"status":"TRADING","maintMarginPercent":"2.5000","requiredMarginPercent":"5.0000","baseAsset":"SAND","quoteAsset":"BUSD","marginAsset":"BUSD","pricePrecision":7,"quantityPrecision":1,"baseAssetPrecision":8,"quotePrecision":8,"underlyingType":"COIN","underlyingSubType":["BUSD"],"settlePlan":0,"triggerProtect":"0.0500","liquidationFee":"0.015000","marketTakeBound":"0.05","filters":[{"minPrice":"0.0010000","maxPrice":"100000","filterType":"PRICE_FILTER","tickSize":"0.0001000"},{"stepSize":"0.1","filterType":"LOT_SIZE","maxQty":"1000000","minQty":"0.1"},{"stepSize":"0.1","filterType":"MARKET_LOT_SIZE","maxQty":"45000","minQty":"0.1"},{"limit":200,"filterType":"MAX_NUM_ORDERS"},{"limit":10,"filterType":"MAX_NUM_ALGO_ORDERS"},{"notional":"5","filterType":"MIN_NOTIONAL"},{"multiplierDown":"0.9500","multiplierUp":"1.0500","multiplierDecimal":"4","filterType":"PERCENT_PRICE"}],"orderTypes":["LIMIT","MARKET","STOP","STOP_MARKET","TAKE_PROFIT","TAKE_PROFIT_MARKET","TRAILING_STOP_MARKET"],"timeInForce":["GTC","IOC","FOK","GTX"]},{"symbol":"LTCBUSD","pair":"LTCBUSD","contractType":"PERPETUAL","deliveryDate":4133404800000,"onboardDate":1657090800000,"status":"TRADING","maintMarginPercent":"2.5000","requiredMarginPercent":"5.0000","baseAsset":"LTC","quoteAsset":"BUSD","marginAsset":"BUSD","pricePrecision":6,"quantityPrecision":2,"baseAssetPrecision":8,"quotePrecision":8,"underlyingType":"COIN","underlyingSubType":["BUSD"],"settlePlan":0,"triggerProtect":"0.0500","liquidationFee":"0.020000","marketTakeBound":"0.05","filters":[{"minPrice":"0.010000","maxPrice":"100000","filterType":"PRICE_FILTER","tickSize":"0.010000"},{"stepSize":"0.01","filterType":"LOT_SIZE","maxQty":"1000000","minQty":"0.01"},{"stepSize":"0.01","filterType":"MARKET_LOT_SIZE","maxQty":"800","minQty":"0.01"},{"limit":200,"filterType":"MAX_NUM_ORDERS"},{"limit":10,"filterType":"MAX_NUM_ALGO_ORDERS"},{"notional":"5","filterType":"MIN_NOTIONAL"},{"multiplierDown":"0.9500","multiplierUp":"1.0500","multiplierDecimal":"4","filterType":"PERCENT_PRICE"}],"orderTypes":["LIMIT","MARKET","STOP","STOP_MARKET","TAKE_PROFIT","TAKE_PROFIT_MARKET","TRAILING_STOP_MARKET"],"timeInForce":["GTC","IOC","FOK","GTX"]},{"symbol":"MATICBUSD","pair":"MATICBUSD","contractType":"PERPETUAL","deliveryDate":4133404800000,"onboardDate":1657609200000,"status":"TRADING","maintMarginPercent":"2.5000","requiredMarginPercent":"5.0000","baseAsset":"MATIC","quoteAsset":"BUSD","marginAsset":"BUSD","pricePrecision":7,"quantityPrecision":0,"baseAssetPrecision":8,"quotePrecision":8,"underlyingType":"COIN","underlyingSubType":["BUSD"],"settlePlan":0,"triggerProtect":"0.0500","liquidationFee":"0.015000","marketTakeBound":"0.05","filters":[{"minPrice":"0.0001000","maxPrice":"10000","filterType":"PRICE_FILTER","tickSize":"0.0001000"},{"stepSize":"1","filterType":"LOT_SIZE","maxQty":"10000000","minQty":"1"},{"stepSize":"1","filterType":"MARKET_LOT_SIZE","maxQty":"100000","minQty":"1"},{"limit":200,"filterType":"MAX_NUM_ORDERS"},{"limit":10,"filterType":"MAX_NUM_ALGO_ORDERS"},{"notional":"5","filterType":"MIN_NOTIONAL"},{"multiplierDown":"0.9500","multiplierUp":"1.0500","multiplierDecimal":"4","filterType":"PERCENT_PRICE"}],"orderTypes":["LIMIT","MARKET","STOP","STOP_MARKET","TAKE_PROFIT","TAKE_PROFIT_MARKET","TRAILING_STOP_MARKET"],"timeInForce":["GTC","IOC","FOK","GTX"]},{"symbol":"CVXBUSD","pair":"CVXBUSD","contractType":"PERPETUAL","deliveryDate":4133404800000,"onboardDate":1657609200000,"status":"TRADING","maintMarginPercent":"2.5000","requiredMarginPercent":"5.0000","baseAsset":"CVX","quoteAsset":"BUSD","marginAsset":"BUSD","pricePrecision":7,"quantityPrecision":1,"baseAssetPrecision":8,"quotePrecision":8,"underlyingType":"COIN","underlyingSubType":["BUSD"],"settlePlan":0,"triggerProtect":"0.0500","liquidationFee":"0.020000","marketTakeBound":"0.05","filters":[{"minPrice":"0.0010000","maxPrice":"100000","filterType":"PRICE_FILTER","tickSize":"0.0010000"},{"stepSize":"0.1","filterType":"LOT_SIZE","maxQty":"1000000","minQty":"0.1"},{"stepSize":"0.1","filterType":"MARKET_LOT_SIZE","maxQty":"1000","minQty":"0.1"},{"limit":200,"filterType":"MAX_NUM_ORDERS"},{"limit":10,"filterType":"MAX_NUM_ALGO_ORDERS"},{"notional":"5","filterType":"MIN_NOTIONAL"},{"multiplierDown":"0.9500","multiplierUp":"1.0500","multiplierDecimal":"4","filterType":"PERCENT_PRICE"}],"orderTypes":["LIMIT","MARKET","STOP","STOP_MARKET","TAKE_PROFIT","TAKE_PROFIT_MARKET","TRAILING_STOP_MARKET"],"timeInForce":["GTC","IOC","FOK","GTX"]},{"symbol":"FILBUSD","pair":"FILBUSD","contractType":"PERPETUAL","deliveryDate":4133404800000,"onboardDate":1658214000000,"status":"TRADING","maintMarginPercent":"2.5000","requiredMarginPercent":"5.0000","baseAsset":"FIL","quoteAsset":"BUSD","marginAsset":"BUSD","pricePrecision":7,"quantityPrecision":1,"baseAssetPrecision":8,"quotePrecision":8,"underlyingType":"COIN","underlyingSubType":["BUSD"],"settlePlan":0,"triggerProtect":"0.1000","liquidationFee":"0.010000","marketTakeBound":"0.10","filters":[{"minPrice":"0.0010000","maxPrice":"100000","filterType":"PRICE_FILTER","tickSize":"0.0010000"},{"stepSize":"0.1","filterType":"LOT_SIZE","maxQty":"1000000","minQty":"0.1"},{"stepSize":"0.1","filterType":"MARKET_LOT_SIZE","maxQty":"10000","minQty":"0.1"},{"limit":200,"filterType":"MAX_NUM_ORDERS"},{"limit":10,"filterType":"MAX_NUM_ALGO_ORDERS"},{"notional":"5","filterType":"MIN_NOTIONAL"},{"multiplierDown":"0.9000","multiplierUp":"1.1000","multiplierDecimal":"4","filterType":"PERCENT_PRICE"}],"orderTypes":["LIMIT","MARKET","STOP","STOP_MARKET","TAKE_PROFIT","TAKE_PROFIT_MARKET","TRAILING_STOP_MARKET"],"timeInForce":["GTC","IOC","FOK","GTX"]},{"symbol":"1000SHIBBUSD","pair":"1000SHIBBUSD","contractType":"PERPETUAL","deliveryDate":4133404800000,"onboardDate":1658214000000,"status":"TRADING","maintMarginPercent":"2.5000","requiredMarginPercent":"5.0000","baseAsset":"1000SHIB","quoteAsset":"BUSD","marginAsset":"BUSD","pricePrecision":7,"quantityPrecision":0,"baseAssetPrecision":8,"quotePrecision":8,"underlyingType":"COIN","underlyingSubType":["BUSD"],"settlePlan":0,"triggerProtect":"0.1000","liquidationFee":"0.010000","marketTakeBound":"0.10","filters":[{"minPrice":"0.0000010","maxPrice":"10000","filterType":"PRICE_FILTER","tickSize":"0.0000010"},{"stepSize":"1","filterType":"LOT_SIZE","maxQty":"10000000","minQty":"1"},{"stepSize":"1","filterType":"MARKET_LOT_SIZE","maxQty":"4000000","minQty":"1"},{"limit":200,"filterType":"MAX_NUM_ORDERS"},{"limit":10,"filterType":"MAX_NUM_ALGO_ORDERS"},{"notional":"5","filterType":"MIN_NOTIONAL"},{"multiplierDown":"0.9000","multiplierUp":"1.1000","multiplierDecimal":"4","filterType":"PERCENT_PRICE"}],"orderTypes":["LIMIT","MARKET","STOP","STOP_MARKET","TAKE_PROFIT","TAKE_PROFIT_MARKET","TRAILING_STOP_MARKET"],"timeInForce":["GTC","IOC","FOK","GTX"]},{"symbol":"LEVERBUSD","pair":"LEVERBUSD","contractType":"PERPETUAL","deliveryDate":4133404800000,"onboardDate":1658214000000,"status":"TRADING","maintMarginPercent":"2.5000","requiredMarginPercent":"5.0000","baseAsset":"LEVER","quoteAsset":"BUSD","marginAsset":"BUSD","pricePrecision":7,"quantityPrecision":0,"baseAssetPrecision":8,"quotePrecision":8,"underlyingType":"COIN","underlyingSubType":["BUSD"],"settlePlan":0,"triggerProtect":"0.1000","liquidationFee":"0.010000","marketTakeBound":"0.10","filters":[{"minPrice":"0.0000010","maxPrice":"10000","filterType":"PRICE_FILTER","tickSize":"0.0000001"},{"stepSize":"1","filterType":"LOT_SIZE","maxQty":"80000000","minQty":"1"},{"stepSize":"1","filterType":"MARKET_LOT_SIZE","maxQty":"10000000","minQty":"1"},{"limit":200,"filterType":"MAX_NUM_ORDERS"},{"limit":10,"filterType":"MAX_NUM_ALGO_ORDERS"},{"notional":"5","filterType":"MIN_NOTIONAL"},{"multiplierDown":"0.9000","multiplierUp":"1.1000","multiplierDecimal":"4","filterType":"PERCENT_PRICE"}],"orderTypes":["LIMIT","MARKET","STOP","STOP_MARKET","TAKE_PROFIT","TAKE_PROFIT_MARKET","TRAILING_STOP_MARKET"],"timeInForce":["GTC","IOC","FOK","GTX"]},{"symbol":"ETCBUSD","pair":"ETCBUSD","contractType":"PERPETUAL","deliveryDate":4133404800000,"onboardDate":1658905200000,"status":"TRADING","maintMarginPercent":"2.5000","requiredMarginPercent":"5.0000","baseAsset":"ETC","quoteAsset":"BUSD","marginAsset":"BUSD","pricePrecision":6,"quantityPrecision":1,"baseAssetPrecision":8,"quotePrecision":8,"underlyingType":"COIN","underlyingSubType":["BUSD"],"settlePlan":0,"triggerProtect":"0.1000","liquidationFee":"0.012500","marketTakeBound":"0.10","filters":[{"minPrice":"0.001000","maxPrice":"100000","filterType":"PRICE_FILTER","tickSize":"0.001000"},{"stepSize":"0.1","filterType":"LOT_SIZE","maxQty":"1000000","minQty":"0.1"},{"stepSize":"0.1","filterType":"MARKET_LOT_SIZE","maxQty":"1500","minQty":"0.1"},{"limit":200,"filterType":"MAX_NUM_ORDERS"},{"limit":10,"filterType":"MAX_NUM_ALGO_ORDERS"},{"notional":"5","filterType":"MIN_NOTIONAL"},{"multiplierDown":"0.9000","multiplierUp":"1.1000","multiplierDecimal":"4","filterType":"PERCENT_PRICE"}],"orderTypes":["LIMIT","MARKET","STOP","STOP_MARKET","TAKE_PROFIT","TAKE_PROFIT_MARKET","TRAILING_STOP_MARKET"],"timeInForce":["GTC","IOC","FOK","GTX"]},{"symbol":"LDOBUSD","pair":"LDOBUSD","contractType":"PERPETUAL","deliveryDate":4133404800000,"onboardDate":1658905200000,"status":"TRADING","maintMarginPercent":"2.5000","requiredMarginPercent":"5.0000","baseAsset":"LDO","quoteAsset":"BUSD","marginAsset":"BUSD","pricePrecision":6,"quantityPrecision":1,"baseAssetPrecision":8,"quotePrecision":8,"underlyingType":"COIN","underlyingSubType":["BUSD"],"settlePlan":0,"triggerProtect":"0.1000","liquidationFee":"0.015000","marketTakeBound":"0.10","filters":[{"minPrice":"0.001000","maxPrice":"100000","filterType":"PRICE_FILTER","tickSize":"0.000100"},{"stepSize":"0.1","filterType":"LOT_SIZE","maxQty":"1000000","minQty":"0.1"},{"stepSize":"0.1","filterType":"MARKET_LOT_SIZE","maxQty":"25000","minQty":"0.1"},{"limit":200,"filterType":"MAX_NUM_ORDERS"},{"limit":10,"filterType":"MAX_NUM_ALGO_ORDERS"},{"notional":"5","filterType":"MIN_NOTIONAL"},{"multiplierDown":"0.9000","multiplierUp":"1.1000","multiplierDecimal":"4","filterType":"PERCENT_PRICE"}],"orderTypes":["LIMIT","MARKET","STOP","STOP_MARKET","TAKE_PROFIT","TAKE_PROFIT_MARKET","TRAILING_STOP_MARKET"],"timeInForce":["GTC","IOC","FOK","GTX"]},{"symbol":"UNIBUSD","pair":"UNIBUSD","contractType":"PERPETUAL","deliveryDate":4133404800000,"onboardDate":1659596400000,"status":"TRADING","maintMarginPercent":"2.5000","requiredMarginPercent":"5.0000","baseAsset":"UNI","quoteAsset":"BUSD","marginAsset":"BUSD","pricePrecision":6,"quantityPrecision":1,"baseAssetPrecision":8,"quotePrecision":8,"underlyingType":"COIN","underlyingSubType":["BUSD"],"settlePlan":0,"triggerProtect":"0.0500","liquidationFee":"0.010000","marketTakeBound":"0.05","filters":[{"minPrice":"0.001000","maxPrice":"100000","filterType":"PRICE_FILTER","tickSize":"0.001000"},{"stepSize":"0.1","filterType":"LOT_SIZE","maxQty":"1000000","minQty":"0.1"},{"stepSize":"0.1","filterType":"MARKET_LOT_SIZE","maxQty":"6000","minQty":"0.1"},{"limit":200,"filterType":"MAX_NUM_ORDERS"},{"limit":10,"filterType":"MAX_NUM_ALGO_ORDERS"},{"notional":"5","filterType":"MIN_NOTIONAL"},{"multiplierDown":"0.9500","multiplierUp":"1.0500","multiplierDecimal":"4","filterType":"PERCENT_PRICE"}],"orderTypes":["LIMIT","MARKET","STOP","STOP_MARKET","TAKE_PROFIT","TAKE_PROFIT_MARKET","TRAILING_STOP_MARKET"],"timeInForce":["GTC","IOC","FOK","GTX"]},{"symbol":"AUCTIONBUSD","pair":"AUCTIONBUSD","contractType":"PERPETUAL","deliveryDate":1671094800000,"onboardDate":1659596400000,"status":"SETTLING","maintMarginPercent":"2.5000","requiredMarginPercent":"5.0000","baseAsset":"AUCTION","quoteAsset":"BUSD","marginAsset":"BUSD","pricePrecision":7,"quantityPrecision":1,"baseAssetPrecision":8,"quotePrecision":8,"underlyingType":"COIN","underlyingSubType":["BUSD"],"settlePlan":1671094800000,"triggerProtect":"0.1000","liquidationFee":"0.010000","marketTakeBound":"0.10","filters":[{"minPrice":"0.0010000","maxPrice":"100000","filterType":"PRICE_FILTER","tickSize":"0.0010000"},{"stepSize":"0.1","filterType":"LOT_SIZE","maxQty":"1000000","minQty":"0.1"},{"stepSize":"0.1","filterType":"MARKET_LOT_SIZE","maxQty":"6000","minQty":"0.1"},{"limit":200,"filterType":"MAX_NUM_ORDERS"},{"limit":10,"filterType":"MAX_NUM_ALGO_ORDERS"},{"notional":"5","filterType":"MIN_NOTIONAL"},{"multiplierDown":"0.9000","multiplierUp":"1.1000","multiplierDecimal":"4","filterType":"PERCENT_PRICE"}],"orderTypes":["LIMIT","MARKET","STOP","STOP_MARKET","TAKE_PROFIT","TAKE_PROFIT_MARKET","TRAILING_STOP_MARKET"],"timeInForce":["GTC","IOC","FOK","GTX"]},{"symbol":"INJUSDT","pair":"INJUSDT","contractType":"PERPETUAL","deliveryDate":4133404800000,"onboardDate":1660633200000,"status":"TRADING","maintMarginPercent":"2.5000","requiredMarginPercent":"5.0000","baseAsset":"INJ","quoteAsset":"USDT","marginAsset":"USDT","pricePrecision":6,"quantityPrecision":1,"baseAssetPrecision":8,"quotePrecision":8,"underlyingType":"COIN","underlyingSubType":["DeFi"],"settlePlan":0,"triggerProtect":"0.1000","liquidationFee":"0.010000","marketTakeBound":"0.10","filters":[{"minPrice":"0.001000","maxPrice":"100000","filterType":"PRICE_FILTER","tickSize":"0.001000"},{"stepSize":"0.1","filterType":"LOT_SIZE","maxQty":"1000000","minQty":"0.1"},{"stepSize":"0.1","filterType":"MARKET_LOT_SIZE","maxQty":"25000","minQty":"0.1"},{"limit":200,"filterType":"MAX_NUM_ORDERS"},{"limit":10,"filterType":"MAX_NUM_ALGO_ORDERS"},{"notional":"5","filterType":"MIN_NOTIONAL"},{"multiplierDown":"0.9000","multiplierUp":"1.1000","multiplierDecimal":"4","filterType":"PERCENT_PRICE"}],"orderTypes":["LIMIT","MARKET","STOP","STOP_MARKET","TAKE_PROFIT","TAKE_PROFIT_MARKET","TRAILING_STOP_MARKET"],"timeInForce":["GTC","IOC","FOK","GTX"]},{"symbol":"STGUSDT","pair":"STGUSDT","contractType":"PERPETUAL","deliveryDate":4133404800000,"onboardDate":1661324400000,"status":"TRADING","maintMarginPercent":"2.5000","requiredMarginPercent":"5.0000","baseAsset":"STG","quoteAsset":"USDT","marginAsset":"USDT","pricePrecision":7,"quantityPrecision":0,"baseAssetPrecision":8,"quotePrecision":8,"underlyingType":"COIN","underlyingSubType":[],"settlePlan":0,"triggerProtect":"0.1000","liquidationFee":"0.015000","marketTakeBound":"0.10","filters":[{"minPrice":"0.0001000","maxPrice":"5000","filterType":"PRICE_FILTER","tickSize":"0.0001000"},{"stepSize":"1","filterType":"LOT_SIZE","maxQty":"10000000","minQty":"1"},{"stepSize":"1","filterType":"MARKET_LOT_SIZE","maxQty":"56000","minQty":"1"},{"limit":200,"filterType":"MAX_NUM_ORDERS"},{"limit":10,"filterType":"MAX_NUM_ALGO_ORDERS"},{"notional":"5","filterType":"MIN_NOTIONAL"},{"multiplierDown":"0.9000","multiplierUp":"1.1000","multiplierDecimal":"4","filterType":"PERCENT_PRICE"}],"orderTypes":["LIMIT","MARKET","STOP","STOP_MARKET","TAKE_PROFIT","TAKE_PROFIT_MARKET","TRAILING_STOP_MARKET"],"timeInForce":["GTC","IOC","FOK","GTX"]},{"symbol":"FOOTBALLUSDT","pair":"FOOTBALLUSDT","contractType":"PERPETUAL","deliveryDate":4133404800000,"onboardDate":1661929200000,"status":"TRADING","maintMarginPercent":"2.5000","requiredMarginPercent":"5.0000","baseAsset":"FOOTBALL","quoteAsset":"USDT","marginAsset":"USDT","pricePrecision":5,"quantityPrecision":2,"baseAssetPrecision":8,"quotePrecision":8,"underlyingType":"INDEX","underlyingSubType":["Index"],"settlePlan":0,"triggerProtect":"0.1500","liquidationFee":"0.015000","marketTakeBound":"0.15","filters":[{"minPrice":"0.01000","maxPrice":"100000","filterType":"PRICE_FILTER","tickSize":"0.01000"},{"stepSize":"0.01","filterType":"LOT_SIZE","maxQty":"10000","minQty":"0.01"},{"stepSize":"0.01","filterType":"MARKET_LOT_SIZE","maxQty":"200","minQty":"0.01"},{"limit":200,"filterType":"MAX_NUM_ORDERS"},{"limit":10,"filterType":"MAX_NUM_ALGO_ORDERS"},{"notional":"5","filterType":"MIN_NOTIONAL"},{"multiplierDown":"0.8500","multiplierUp":"1.1500","multiplierDecimal":"4","filterType":"PERCENT_PRICE"}],"orderTypes":["LIMIT","MARKET","STOP","STOP_MARKET","TAKE_PROFIT","TAKE_PROFIT_MARKET","TRAILING_STOP_MARKET"],"timeInForce":["GTC","IOC","FOK","GTX"]},{"symbol":"SPELLUSDT","pair":"SPELLUSDT","contractType":"PERPETUAL","deliveryDate":4133404800000,"onboardDate":1662361200000,"status":"TRADING","maintMarginPercent":"2.5000","requiredMarginPercent":"5.0000","baseAsset":"SPELL","quoteAsset":"USDT","marginAsset":"USDT","pricePrecision":7,"quantityPrecision":0,"baseAssetPrecision":8,"quotePrecision":8,"underlyingType":"COIN","underlyingSubType":["DeFi"],"settlePlan":0,"triggerProtect":"0.1000","liquidationFee":"0.010000","marketTakeBound":"0.10","filters":[{"minPrice":"0.0000010","maxPrice":"1000","filterType":"PRICE_FILTER","tickSize":"0.0000001"},{"stepSize":"1","filterType":"LOT_SIZE","maxQty":"80000000","minQty":"1"},{"stepSize":"1","filterType":"MARKET_LOT_SIZE","maxQty":"8000000","minQty":"1"},{"limit":200,"filterType":"MAX_NUM_ORDERS"},{"limit":10,"filterType":"MAX_NUM_ALGO_ORDERS"},{"notional":"5","filterType":"MIN_NOTIONAL"},{"multiplierDown":"0.9000","multiplierUp":"1.1000","multiplierDecimal":"4","filterType":"PERCENT_PRICE"}],"orderTypes":["LIMIT","MARKET","STOP","STOP_MARKET","TAKE_PROFIT","TAKE_PROFIT_MARKET","TRAILING_STOP_MARKET"],"timeInForce":["GTC","IOC","FOK","GTX"]},{"symbol":"1000LUNCUSDT","pair":"1000LUNCUSDT","contractType":"PERPETUAL","deliveryDate":4133404800000,"onboardDate":1662620400000,"status":"TRADING","maintMarginPercent":"2.5000","requiredMarginPercent":"5.0000","baseAsset":"1000LUNC","quoteAsset":"USDT","marginAsset":"USDT","pricePrecision":7,"quantityPrecision":0,"baseAssetPrecision":8,"quotePrecision":8,"underlyingType":"COIN","underlyingSubType":["Layer-1"],"settlePlan":0,"triggerProtect":"0.1000","liquidationFee":"0.015000","marketTakeBound":"0.10","filters":[{"minPrice":"0.0001000","maxPrice":"100000","filterType":"PRICE_FILTER","tickSize":"0.0001000"},{"stepSize":"1","filterType":"LOT_SIZE","maxQty":"10000000","minQty":"1"},{"stepSize":"1","filterType":"MARKET_LOT_SIZE","maxQty":"2000000","minQty":"1"},{"limit":200,"filterType":"MAX_NUM_ORDERS"},{"limit":10,"filterType":"MAX_NUM_ALGO_ORDERS"},{"notional":"5","filterType":"MIN_NOTIONAL"},{"multiplierDown":"0.9000","multiplierUp":"1.1000","multiplierDecimal":"4","filterType":"PERCENT_PRICE"}],"orderTypes":["LIMIT","MARKET","STOP","STOP_MARKET","TAKE_PROFIT","TAKE_PROFIT_MARKET","TRAILING_STOP_MARKET"],"timeInForce":["GTC","IOC","FOK","GTX"]},{"symbol":"LUNA2USDT","pair":"LUNA2USDT","contractType":"PERPETUAL","deliveryDate":4133404800000,"onboardDate":1662706800000,"status":"TRADING","maintMarginPercent":"2.5000","requiredMarginPercent":"5.0000","baseAsset":"LUNA2","quoteAsset":"USDT","marginAsset":"USDT","pricePrecision":7,"quantityPrecision":0,"baseAssetPrecision":8,"quotePrecision":8,"underlyingType":"COIN","underlyingSubType":["Layer-1"],"settlePlan":0,"triggerProtect":"0.1000","liquidationFee":"0.020000","marketTakeBound":"0.10","filters":[{"minPrice":"0.0001000","maxPrice":"10000","filterType":"PRICE_FILTER","tickSize":"0.0001000"},{"stepSize":"1","filterType":"LOT_SIZE","maxQty":"1000000","minQty":"1"},{"stepSize":"1","filterType":"MARKET_LOT_SIZE","maxQty":"60000","minQty":"1"},{"limit":200,"filterType":"MAX_NUM_ORDERS"},{"limit":10,"filterType":"MAX_NUM_ALGO_ORDERS"},{"notional":"5","filterType":"MIN_NOTIONAL"},{"multiplierDown":"0.9000","multiplierUp":"1.1000","multiplierDecimal":"4","filterType":"PERCENT_PRICE"}],"orderTypes":["LIMIT","MARKET","STOP","STOP_MARKET","TAKE_PROFIT","TAKE_PROFIT_MARKET","TRAILING_STOP_MARKET"],"timeInForce":["GTC","IOC","FOK","GTX"]},{"symbol":"AMBBUSD","pair":"AMBBUSD","contractType":"PERPETUAL","deliveryDate":4133404800000,"onboardDate":1663052400000,"status":"TRADING","maintMarginPercent":"2.5000","requiredMarginPercent":"5.0000","baseAsset":"AMB","quoteAsset":"BUSD","marginAsset":"BUSD","pricePrecision":7,"quantityPrecision":0,"baseAssetPrecision":8,"quotePrecision":8,"underlyingType":"COIN","underlyingSubType":["Infrastructure"],"settlePlan":0,"triggerProtect":"0.1000","liquidationFee":"0.010000","marketTakeBound":"0.10","filters":[{"minPrice":"0.0000100","maxPrice":"10000","filterType":"PRICE_FILTER","tickSize":"0.0000010"},{"stepSize":"1","filterType":"LOT_SIZE","maxQty":"50000000","minQty":"1"},{"stepSize":"1","filterType":"MARKET_LOT_SIZE","maxQty":"3000000","minQty":"1"},{"limit":200,"filterType":"MAX_NUM_ORDERS"},{"limit":10,"filterType":"MAX_NUM_ALGO_ORDERS"},{"notional":"5","filterType":"MIN_NOTIONAL"},{"multiplierDown":"0.9000","multiplierUp":"1.1000","multiplierDecimal":"4","filterType":"PERCENT_PRICE"}],"orderTypes":["LIMIT","MARKET","STOP","STOP_MARKET","TAKE_PROFIT","TAKE_PROFIT_MARKET","TRAILING_STOP_MARKET"],"timeInForce":["GTC","IOC","FOK","GTX"]},{"symbol":"PHBBUSD","pair":"PHBBUSD","contractType":"PERPETUAL","deliveryDate":4133404800000,"onboardDate":1663225200000,"status":"TRADING","maintMarginPercent":"2.5000","requiredMarginPercent":"5.0000","baseAsset":"PHB","quoteAsset":"BUSD","marginAsset":"BUSD","pricePrecision":7,"quantityPrecision":0,"baseAssetPrecision":8,"quotePrecision":8,"underlyingType":"COIN","underlyingSubType":["Infrastructure"],"settlePlan":0,"triggerProtect":"0.1000","liquidationFee":"0.020000","marketTakeBound":"0.10","filters":[{"minPrice":"0.0001000","maxPrice":"10000","filterType":"PRICE_FILTER","tickSize":"0.0001000"},{"stepSize":"1","filterType":"LOT_SIZE","maxQty":"1000000","minQty":"1"},{"stepSize":"1","filterType":"MARKET_LOT_SIZE","maxQty":"100000","minQty":"1"},{"limit":200,"filterType":"MAX_NUM_ORDERS"},{"limit":10,"filterType":"MAX_NUM_ALGO_ORDERS"},{"notional":"5","filterType":"MIN_NOTIONAL"},{"multiplierDown":"0.9000","multiplierUp":"1.1000","multiplierDecimal":"4","filterType":"PERCENT_PRICE"}],"orderTypes":["LIMIT","MARKET","STOP","STOP_MARKET","TAKE_PROFIT","TAKE_PROFIT_MARKET","TRAILING_STOP_MARKET"],"timeInForce":["GTC","IOC","FOK","GTX"]},{"symbol":"LDOUSDT","pair":"LDOUSDT","contractType":"PERPETUAL","deliveryDate":4133404800000,"onboardDate":1663743600000,"status":"TRADING","maintMarginPercent":"2.5000","requiredMarginPercent":"5.0000","baseAsset":"LDO","quoteAsset":"USDT","marginAsset":"USDT","pricePrecision":6,"quantityPrecision":0,"baseAssetPrecision":8,"quotePrecision":8,"underlyingType":"COIN","underlyingSubType":["Layer-2"],"settlePlan":0,"triggerProtect":"0.1000","liquidationFee":"0.020000","marketTakeBound":"0.10","filters":[{"minPrice":"0.001000","maxPrice":"10000","filterType":"PRICE_FILTER","tickSize":"0.000100"},{"stepSize":"1","filterType":"LOT_SIZE","maxQty":"100000","minQty":"1"},{"stepSize":"1","filterType":"MARKET_LOT_SIZE","maxQty":"30000","minQty":"1"},{"limit":200,"filterType":"MAX_NUM_ORDERS"},{"limit":10,"filterType":"MAX_NUM_ALGO_ORDERS"},{"notional":"5","filterType":"MIN_NOTIONAL"},{"multiplierDown":"0.9000","multiplierUp":"1.1000","multiplierDecimal":"4","filterType":"PERCENT_PRICE"}],"orderTypes":["LIMIT","MARKET","STOP","STOP_MARKET","TAKE_PROFIT","TAKE_PROFIT_MARKET","TRAILING_STOP_MARKET"],"timeInForce":["GTC","IOC","FOK","GTX"]},{"symbol":"CVXUSDT","pair":"CVXUSDT","contractType":"PERPETUAL","deliveryDate":4133404800000,"onboardDate":1663743600000,"status":"TRADING","maintMarginPercent":"2.5000","requiredMarginPercent":"5.0000","baseAsset":"CVX","quoteAsset":"USDT","marginAsset":"USDT","pricePrecision":6,"quantityPrecision":0,"baseAssetPrecision":8,"quotePrecision":8,"underlyingType":"COIN","underlyingSubType":["DeFi"],"settlePlan":0,"triggerProtect":"0.1200","liquidationFee":"0.020000","marketTakeBound":"0.12","filters":[{"minPrice":"0.001000","maxPrice":"10000","filterType":"PRICE_FILTER","tickSize":"0.001000"},{"stepSize":"1","filterType":"LOT_SIZE","maxQty":"100000","minQty":"1"},{"stepSize":"1","filterType":"MARKET_LOT_SIZE","maxQty":"1000","minQty":"1"},{"limit":200,"filterType":"MAX_NUM_ORDERS"},{"limit":10,"filterType":"MAX_NUM_ALGO_ORDERS"},{"notional":"5","filterType":"MIN_NOTIONAL"},{"multiplierDown":"0.8800","multiplierUp":"1.1200","multiplierDecimal":"4","filterType":"PERCENT_PRICE"}],"orderTypes":["LIMIT","MARKET","STOP","STOP_MARKET","TAKE_PROFIT","TAKE_PROFIT_MARKET","TRAILING_STOP_MARKET"],"timeInForce":["GTC","IOC","FOK","GTX"]},{"symbol":"ICPUSDT","pair":"ICPUSDT","contractType":"PERPETUAL","deliveryDate":4133404800000,"onboardDate":1627628400000,"status":"TRADING","maintMarginPercent":"2.5000","requiredMarginPercent":"5.0000","baseAsset":"ICP","quoteAsset":"USDT","marginAsset":"USDT","pricePrecision":6,"quantityPrecision":0,"baseAssetPrecision":8,"quotePrecision":8,"underlyingType":"COIN","underlyingSubType":["Layer-1"],"settlePlan":0,"triggerProtect":"0.1500","liquidationFee":"0.015000","marketTakeBound":"0.30","filters":[{"minPrice":"0.001000","maxPrice":"10000","filterType":"PRICE_FILTER","tickSize":"0.001000"},{"stepSize":"1","filterType":"LOT_SIZE","maxQty":"100000","minQty":"1"},{"stepSize":"1","filterType":"MARKET_LOT_SIZE","maxQty":"10000","minQty":"1"},{"limit":200,"filterType":"MAX_NUM_ORDERS"},{"limit":10,"filterType":"MAX_NUM_ALGO_ORDERS"},{"notional":"5","filterType":"MIN_NOTIONAL"},{"multiplierDown":"0.9000","multiplierUp":"1.1000","multiplierDecimal":"4","filterType":"PERCENT_PRICE"}],"orderTypes":["LIMIT","MARKET","STOP","STOP_MARKET","TAKE_PROFIT","TAKE_PROFIT_MARKET","TRAILING_STOP_MARKET"],"timeInForce":["GTC","IOC","FOK","GTX"]},{"symbol":"APTUSDT","pair":"APTUSDT","contractType":"PERPETUAL","deliveryDate":4133404800000,"onboardDate":1666076400000,"status":"TRADING","maintMarginPercent":"2.5000","requiredMarginPercent":"5.0000","baseAsset":"APT","quoteAsset":"USDT","marginAsset":"USDT","pricePrecision":5,"quantityPrecision":1,"baseAssetPrecision":8,"quotePrecision":8,"underlyingType":"COIN","underlyingSubType":["Layer-1"],"settlePlan":0,"triggerProtect":"0.1000","liquidationFee":"0.025000","marketTakeBound":"0.10","filters":[{"minPrice":"0.00100","maxPrice":"100000","filterType":"PRICE_FILTER","tickSize":"0.00010"},{"stepSize":"0.1","filterType":"LOT_SIZE","maxQty":"1000000","minQty":"0.1"},{"stepSize":"0.1","filterType":"MARKET_LOT_SIZE","maxQty":"8000","minQty":"0.1"},{"limit":200,"filterType":"MAX_NUM_ORDERS"},{"limit":10,"filterType":"MAX_NUM_ALGO_ORDERS"},{"notional":"5","filterType":"MIN_NOTIONAL"},{"multiplierDown":"0.9000","multiplierUp":"1.1000","multiplierDecimal":"4","filterType":"PERCENT_PRICE"}],"orderTypes":["LIMIT","MARKET","STOP","STOP_MARKET","TAKE_PROFIT","TAKE_PROFIT_MARKET","TRAILING_STOP_MARKET"],"timeInForce":["GTC","IOC","FOK","GTX"]},{"symbol":"QNTUSDT","pair":"QNTUSDT","contractType":"PERPETUAL","deliveryDate":4133404800000,"onboardDate":1666076400000,"status":"TRADING","maintMarginPercent":"2.5000","requiredMarginPercent":"5.0000","baseAsset":"QNT","quoteAsset":"USDT","marginAsset":"USDT","pricePrecision":6,"quantityPrecision":1,"baseAssetPrecision":8,"quotePrecision":8,"underlyingType":"COIN","underlyingSubType":["Infrastructure"],"settlePlan":0,"triggerProtect":"0.1000","liquidationFee":"0.012500","marketTakeBound":"0.10","filters":[{"minPrice":"0.010000","maxPrice":"100000","filterType":"PRICE_FILTER","tickSize":"0.010000"},{"stepSize":"0.1","filterType":"LOT_SIZE","maxQty":"1000000","minQty":"0.1"},{"stepSize":"0.1","filterType":"MARKET_LOT_SIZE","maxQty":"300","minQty":"0.1"},{"limit":200,"filterType":"MAX_NUM_ORDERS"},{"limit":10,"filterType":"MAX_NUM_ALGO_ORDERS"},{"notional":"5","filterType":"MIN_NOTIONAL"},{"multiplierDown":"0.9000","multiplierUp":"1.1000","multiplierDecimal":"4","filterType":"PERCENT_PRICE"}],"orderTypes":["LIMIT","MARKET","STOP","STOP_MARKET","TAKE_PROFIT","TAKE_PROFIT_MARKET","TRAILING_STOP_MARKET"],"timeInForce":["GTC","IOC","FOK","GTX"]},{"symbol":"APTBUSD","pair":"APTBUSD","contractType":"PERPETUAL","deliveryDate":4133404800000,"onboardDate":1666594800000,"status":"TRADING","maintMarginPercent":"2.5000","requiredMarginPercent":"5.0000","baseAsset":"APT","quoteAsset":"BUSD","marginAsset":"BUSD","pricePrecision":5,"quantityPrecision":1,"baseAssetPrecision":8,"quotePrecision":8,"underlyingType":"COIN","underlyingSubType":["Layer-1"],"settlePlan":0,"triggerProtect":"0.1000","liquidationFee":"0.015000","marketTakeBound":"0.10","filters":[{"minPrice":"0.00100","maxPrice":"100000","filterType":"PRICE_FILTER","tickSize":"0.00010"},{"stepSize":"0.1","filterType":"LOT_SIZE","maxQty":"1000000","minQty":"0.1"},{"stepSize":"0.1","filterType":"MARKET_LOT_SIZE","maxQty":"5000","minQty":"0.1"},{"limit":200,"filterType":"MAX_NUM_ORDERS"},{"limit":10,"filterType":"MAX_NUM_ALGO_ORDERS"},{"notional":"5","filterType":"MIN_NOTIONAL"},{"multiplierDown":"0.9000","multiplierUp":"1.1000","multiplierDecimal":"4","filterType":"PERCENT_PRICE"}],"orderTypes":["LIMIT","MARKET","STOP","STOP_MARKET","TAKE_PROFIT","TAKE_PROFIT_MARKET","TRAILING_STOP_MARKET"],"timeInForce":["GTC","IOC","FOK","GTX"]},{"symbol":"BLUEBIRDUSDT","pair":"BLUEBIRDUSDT","contractType":"PERPETUAL","deliveryDate":4133404800000,"onboardDate":1667286000000,"status":"TRADING","maintMarginPercent":"2.5000","requiredMarginPercent":"5.0000","baseAsset":"BLUEBIRD","quoteAsset":"USDT","marginAsset":"USDT","pricePrecision":5,"quantityPrecision":1,"baseAssetPrecision":8,"quotePrecision":8,"underlyingType":"INDEX","underlyingSubType":["Index"],"settlePlan":0,"triggerProtect":"0.1000","liquidationFee":"0.015000","marketTakeBound":"0.10","filters":[{"minPrice":"0.00100","maxPrice":"100000","filterType":"PRICE_FILTER","tickSize":"0.00100"},{"stepSize":"0.1","filterType":"LOT_SIZE","maxQty":"1000000","minQty":"0.1"},{"stepSize":"0.1","filterType":"MARKET_LOT_SIZE","maxQty":"5000","minQty":"0.1"},{"limit":200,"filterType":"MAX_NUM_ORDERS"},{"limit":10,"filterType":"MAX_NUM_ALGO_ORDERS"},{"notional":"5","filterType":"MIN_NOTIONAL"},{"multiplierDown":"0.9000","multiplierUp":"1.1000","multiplierDecimal":"4","filterType":"PERCENT_PRICE"}],"orderTypes":["LIMIT","MARKET","STOP","STOP_MARKET","TAKE_PROFIT","TAKE_PROFIT_MARKET","TRAILING_STOP_MARKET"],"timeInForce":["GTC","IOC","FOK","GTX"]},{"symbol":"ETHUSDT_230331","pair":"ETHUSDT","contractType":"CURRENT_QUARTER","deliveryDate":1680249600000,"onboardDate":1671696000000,"status":"TRADING","maintMarginPercent":"2.5000","requiredMarginPercent":"5.0000","baseAsset":"ETH","quoteAsset":"USDT","marginAsset":"USDT","pricePrecision":2,"quantityPrecision":3,"baseAssetPrecision":8,"quotePrecision":8,"underlyingType":"COIN","underlyingSubType":[],"settlePlan":0,"triggerProtect":"0.0500","liquidationFee":"0.010000","marketTakeBound":"0.05","filters":[{"minPrice":"41.10","maxPrice":"100000","filterType":"PRICE_FILTER","tickSize":"0.01"},{"stepSize":"0.001","filterType":"LOT_SIZE","maxQty":"10000","minQty":"0.001"},{"stepSize":"0.001","filterType":"MARKET_LOT_SIZE","maxQty":"10","minQty":"0.001"},{"limit":200,"filterType":"MAX_NUM_ORDERS"},{"limit":10,"filterType":"MAX_NUM_ALGO_ORDERS"},{"notional":"5","filterType":"MIN_NOTIONAL"},{"multiplierDown":"0.9500","multiplierUp":"1.0500","multiplierDecimal":"4","filterType":"PERCENT_PRICE"}],"orderTypes":["LIMIT","MARKET","STOP","STOP_MARKET","TAKE_PROFIT","TAKE_PROFIT_MARKET","TRAILING_STOP_MARKET"],"timeInForce":["GTC","IOC","FOK","GTX"]},{"symbol":"BTCUSDT_230331","pair":"BTCUSDT","contractType":"CURRENT_QUARTER","deliveryDate":1680249600000,"onboardDate":1671696000000,"status":"TRADING","maintMarginPercent":"2.5000","requiredMarginPercent":"5.0000","baseAsset":"BTC","quoteAsset":"USDT","marginAsset":"USDT","pricePrecision":1,"quantityPrecision":3,"baseAssetPrecision":8,"quotePrecision":8,"underlyingType":"COIN","underlyingSubType":[],"settlePlan":0,"triggerProtect":"0.0500","liquidationFee":"0.010000","marketTakeBound":"0.05","filters":[{"minPrice":"576.3","maxPrice":"1000000","filterType":"PRICE_FILTER","tickSize":"0.1"},{"stepSize":"0.001","filterType":"LOT_SIZE","maxQty":"500","minQty":"0.001"},{"stepSize":"0.001","filterType":"MARKET_LOT_SIZE","maxQty":"3","minQty":"0.001"},{"limit":200,"filterType":"MAX_NUM_ORDERS"},{"limit":10,"filterType":"MAX_NUM_ALGO_ORDERS"},{"notional":"5","filterType":"MIN_NOTIONAL"},{"multiplierDown":"0.9500","multiplierUp":"1.0500","multiplierDecimal":"4","filterType":"PERCENT_PRICE"}],"orderTypes":["LIMIT","MARKET","STOP","STOP_MARKET","TAKE_PROFIT","TAKE_PROFIT_MARKET","TRAILING_STOP_MARKET"],"timeInForce":["GTC","IOC","FOK","GTX"]},{"symbol":"FETUSDT","pair":"FETUSDT","contractType":"PERPETUAL","deliveryDate":4133404800000,"onboardDate":1673766000000,"status":"TRADING","maintMarginPercent":"2.5000","requiredMarginPercent":"5.0000","baseAsset":"FET","quoteAsset":"USDT","marginAsset":"USDT","pricePrecision":7,"quantityPrecision":0,"baseAssetPrecision":8,"quotePrecision":8,"underlyingType":"COIN","underlyingSubType":["AI"],"settlePlan":0,"triggerProtect":"0.1000","liquidationFee":"0.015000","marketTakeBound":"0.10","filters":[{"minPrice":"0.0001000","maxPrice":"1000","filterType":"PRICE_FILTER","tickSize":"0.0001000"},{"stepSize":"1","filterType":"LOT_SIZE","maxQty":"10000000","minQty":"1"},{"stepSize":"1","filterType":"MARKET_LOT_SIZE","maxQty":"250000","minQty":"1"},{"limit":200,"filterType":"MAX_NUM_ORDERS"},{"limit":10,"filterType":"MAX_NUM_ALGO_ORDERS"},{"notional":"5","filterType":"MIN_NOTIONAL"},{"multiplierDown":"0.9000","multiplierUp":"1.1000","multiplierDecimal":"4","filterType":"PERCENT_PRICE"}],"orderTypes":["LIMIT","MARKET","STOP","STOP_MARKET","TAKE_PROFIT","TAKE_PROFIT_MARKET","TRAILING_STOP_MARKET"],"timeInForce":["GTC","IOC","FOK","GTX"]},{"symbol":"AGIXBUSD","pair":"AGIXBUSD","contractType":"PERPETUAL","deliveryDate":4133404800000,"onboardDate":1673938800000,"status":"TRADING","maintMarginPercent":"2.5000","requiredMarginPercent":"5.0000","baseAsset":"AGIX","quoteAsset":"BUSD","marginAsset":"BUSD","pricePrecision":7,"quantityPrecision":0,"baseAssetPrecision":8,"quotePrecision":8,"underlyingType":"COIN","underlyingSubType":["AI"],"settlePlan":0,"triggerProtect":"0.1000","liquidationFee":"0.015000","marketTakeBound":"0.10","filters":[{"minPrice":"0.0001000","maxPrice":"1000","filterType":"PRICE_FILTER","tickSize":"0.0001000"},{"stepSize":"1","filterType":"LOT_SIZE","maxQty":"10000000","minQty":"1"},{"stepSize":"1","filterType":"MARKET_LOT_SIZE","maxQty":"270000","minQty":"1"},{"limit":200,"filterType":"MAX_NUM_ORDERS"},{"limit":10,"filterType":"MAX_NUM_ALGO_ORDERS"},{"notional":"5","filterType":"MIN_NOTIONAL"},{"multiplierDown":"0.9000","multiplierUp":"1.1000","multiplierDecimal":"4","filterType":"PERCENT_PRICE"}],"orderTypes":["LIMIT","MARKET","STOP","STOP_MARKET","TAKE_PROFIT","TAKE_PROFIT_MARKET","TRAILING_STOP_MARKET"],"timeInForce":["GTC","IOC","FOK","GTX"]},{"symbol":"FXSUSDT","pair":"FXSUSDT","contractType":"PERPETUAL","deliveryDate":4133404800000,"onboardDate":1674111600000,"status":"TRADING","maintMarginPercent":"2.5000","requiredMarginPercent":"5.0000","baseAsset":"FXS","quoteAsset":"USDT","marginAsset":"USDT","pricePrecision":6,"quantityPrecision":1,"baseAssetPrecision":8,"quotePrecision":8,"underlyingType":"COIN","underlyingSubType":[],"settlePlan":0,"triggerProtect":"0.1500","liquidationFee":"0.015000","marketTakeBound":"0.15","filters":[{"minPrice":"0.001000","maxPrice":"100000","filterType":"PRICE_FILTER","tickSize":"0.001000"},{"stepSize":"0.1","filterType":"LOT_SIZE","maxQty":"1000000","minQty":"0.1"},{"stepSize":"0.1","filterType":"MARKET_LOT_SIZE","maxQty":"5000","minQty":"0.1"},{"limit":200,"filterType":"MAX_NUM_ORDERS"},{"limit":10,"filterType":"MAX_NUM_ALGO_ORDERS"},{"notional":"5","filterType":"MIN_NOTIONAL"},{"multiplierDown":"0.8500","multiplierUp":"1.1500","multiplierDecimal":"4","filterType":"PERCENT_PRICE"}],"orderTypes":["LIMIT","MARKET","STOP","STOP_MARKET","TAKE_PROFIT","TAKE_PROFIT_MARKET","TRAILING_STOP_MARKET"],"timeInForce":["GTC","IOC","FOK","GTX"]},{"symbol":"HOOKUSDT","pair":"HOOKUSDT","contractType":"PERPETUAL","deliveryDate":4133404800000,"onboardDate":1674111600000,"status":"TRADING","maintMarginPercent":"2.5000","requiredMarginPercent":"5.0000","baseAsset":"HOOK","quoteAsset":"USDT","marginAsset":"USDT","pricePrecision":6,"quantityPrecision":1,"baseAssetPrecision":8,"quotePrecision":8,"underlyingType":"COIN","underlyingSubType":[],"settlePlan":0,"triggerProtect":"0.1500","liquidationFee":"0.015000","marketTakeBound":"0.15","filters":[{"minPrice":"0.001000","maxPrice":"100000","filterType":"PRICE_FILTER","tickSize":"0.001000"},{"stepSize":"0.1","filterType":"LOT_SIZE","maxQty":"1000000","minQty":"0.1"},{"stepSize":"0.1","filterType":"MARKET_LOT_SIZE","maxQty":"20000","minQty":"0.1"},{"limit":200,"filterType":"MAX_NUM_ORDERS"},{"limit":10,"filterType":"MAX_NUM_ALGO_ORDERS"},{"notional":"5","filterType":"MIN_NOTIONAL"},{"multiplierDown":"0.8500","multiplierUp":"1.1500","multiplierDecimal":"4","filterType":"PERCENT_PRICE"}],"orderTypes":["LIMIT","MARKET","STOP","STOP_MARKET","TAKE_PROFIT","TAKE_PROFIT_MARKET","TRAILING_STOP_MARKET"],"timeInForce":["GTC","IOC","FOK","GTX"]}]}"""
# info = json.loads(perpetual_info)

In [3]:
perpetual_ticker_df = pd.DataFrame(perpetual_ticker)
perpetual_ticker_df["quoteVolume"] = perpetual_ticker_df["quoteVolume"].astype(float)
perpetual_ticker_df = perpetual_ticker_df.sort_values(by="quoteVolume")
perpetual_ticker_df.set_index("symbol", inplace=True)

In [4]:
from datetime import datetime
volumeThreshold = 50_000_000
liveDaysThreshold = 90
today = datetime.utcnow()

In [5]:
def get_tag(tags):
    if len(tags) > 1:
        raise ValueError()
    elif len(tags) == 1:
        return tags[0]
    else:
        return "None"
perpetual_symbol_info = pd.DataFrame(perpetual_info["symbols"])
perpetual_symbol_info.set_index("symbol", verify_integrity=True, inplace=True)
perpetual_symbol_info["onboardDate"] = pd.to_datetime(perpetual_symbol_info["onboardDate"], unit="ms")
perpetual_symbol_info["deliveryDate"] = pd.to_datetime(perpetual_symbol_info["deliveryDate"], unit="ms")
perpetual_symbol_info["liveDays"] = (today - perpetual_symbol_info["onboardDate"]).dt.days
perpetual_symbol_info["quoteVolume"] = perpetual_ticker_df["quoteVolume"]
perpetual_symbol_info["tag"] = perpetual_symbol_info["underlyingSubType"].apply(get_tag)

In [6]:
perpetual_symbol_info_selected = perpetual_symbol_info.query("status=='TRADING' and contractType=='PERPETUAL'")
perpetual_symbol_info_selected = perpetual_symbol_info_selected.query("quoteVolume>=@volumeThreshold")
perpetual_symbol_info_selected = perpetual_symbol_info_selected.query("liveDays>=@liveDaysThreshold")
perpetual_symbol_info_selected = perpetual_symbol_info_selected.query("tag!='None'")
risky_coins = ["FTTUSDT", "LUNAUSDT", "FTTBUSD", "LUNABUSD"]
perpetual_symbol_info_selected = perpetual_symbol_info_selected.query("symbol not in @risky_coins")

In [7]:
perpetual_symbol_info_selected.tag.value_counts()

Layer-1           19
BUSD               9
Layer-2            7
NFT                6
DeFi               5
Metaverse          5
DEX                4
PoW                3
Oracle             3
Infrastructure     3
Meme               2
Storage            2
Payment            1
CEX                1
Privacy            1
DAO                1
Name: tag, dtype: int64

In [8]:
perpetual_symbol_info_selected["CreatedDateByAuthor"] = today
perpetual_symbol_info_selected.to_csv("/Users/bytedance/binance_data/perpetual_symbol_info_selected.csv", index=True)

In [9]:
# spot_symbol_info = pd.DataFrame(spot_info["symbols"])
# spot_symbol_info.set_index("symbol", verify_integrity=True, inplace=True)

In [10]:
csv_path = "/Users/bytedance/MLTrader-deploy/research/data_binance/final_csv.csv"
df = pd.read_csv(csv_path, index_col=0)
df

,symbol,volume,turnover,open,high,low,close,closed_datetime
0,ETHBTC,1.005964e+05,7643.073074,0.075020,0.077084,0.074927,0.076153,2022-11-07 15:59:59.999
1,ETHBTC,3.001515e+05,22254.036472,0.076158,0.077140,0.071195,0.071965,2022-11-08 15:59:59.999
2,ETHBTC,3.588011e+05,24905.428504,0.071965,0.072218,0.066000,0.069253,2022-11-09 15:59:59.999
3,ETHBTC,2.894290e+05,21021.622849,0.069269,0.075638,0.068824,0.073828,2022-11-10 15:59:59.999
4,ETHBTC,1.908479e+05,14128.237467,0.073828,0.076133,0.072133,0.075563,2022-11-11 15:59:59.999
...,...,...,...,...,...,...,...,...
1,HOOKBNB,2.247054e+06,19682.234724,0.008700,0.010296,0.007812,0.008443,2022-12-02 15:59:59.999
2,HOOKBNB,1.627284e+06,12949.144115,0.008403,0.009040,0.007316,0.007560,2022-12-03 15:59:59.999
3,HOOKBNB,1.624098e+06,13075.388816,0.007560,0.008768,0.007343,0.007688,2022-12-04 15:59:59.999
4,HOOKBNB,9.356433e+05,7220.692024,0.007719,0.008748,0.007450,0.007849,2022-12-05 15:59:59.999


In [11]:
min_turnover = 3_000_000
min_turnover2 = 3_000_000

In [12]:
sum_counts = df.groupby(by=df["symbol"])["turnover"].count()

In [13]:
sum_counts.value_counts()

30    2110
21       8
6        4
19       3
22       1
8        1
27       1
Name: turnover, dtype: int64

In [14]:
stable_coins = ["USDT", "BUSD", "TUSD", "USDC", "USDP"]
delete = ["UST", "FTT", "ETHBEAR", "NPXS", "ETHBULL", "EUR", 
          "BCHABC", "BCC", "BEAR", "GXS", 'BULL', "EPS", 
          "BZRX", "EOSBULL", "LEND", "HNT", 
          "SRM", "KEEP", "HOOK"]
usdt_symbols = [symbol for symbol in np.unique(df["symbol"]) if symbol.endswith("USDT") and symbol[:4] not in stable_coins and symbol[:-4] not in delete]
busd_symbols = [symbol for symbol in np.unique(df["symbol"]) if symbol.endswith("BUSD") and symbol[:4] not in stable_coins and symbol[:-4] not in delete]
avg_turnover = df.groupby(by=df["symbol"])["turnover"].agg(["mean", "count", "median"])

In [15]:
sub_df = avg_turnover.loc[usdt_symbols].loc[(avg_turnover["mean"] >= min_turnover2) & (avg_turnover["median"] >= min_turnover) & (avg_turnover["count"] == 30)]
sub_df.sort_values(by="median", ascending=False)

,mean,count,median
symbol,,,
BTCUSDT,4.764826e+09,30,3.898110e+09
ETHUSDT,8.929443e+08,30,6.316753e+08
DOGEUSDT,2.132911e+08,30,1.837455e+08
XRPUSDT,1.527341e+08,30,1.350818e+08
BNBUSDT,1.915259e+08,30,1.327555e+08
...,...,...,...
CAKEUSDT,5.905444e+06,30,3.354292e+06
CELOUSDT,5.338510e+06,30,3.268743e+06
LDOUSDT,5.617680e+06,30,3.095883e+06


In [16]:
import requests
def symbol_get_most_recent_depth_data(symbol, depth):
    response = requests.get(
        f"https://api.binance.com/api/v3/depth?symbol={symbol}&limit={depth}"
    ).json()
    if len(response["bids"]) == 0:
        raise ValueError(f"{symbol} got zero depth: {response}")
    d = {"symbol": symbol}
    d["bids_price"], d["bids_qty"] = zip(*response["bids"])
    d["asks_price"], d["asks_qty"] = zip(*response["asks"])
    d["bids_price"] = list(map(float, d["bids_price"]))
    d["bids_qty"] = list(map(float, d["bids_qty"]))
    d["asks_price"] = list(map(float, d["asks_price"]))
    d["asks_qty"] = list(map(float, d["asks_qty"]))
    return d

In [17]:
valid_symbols = []
for symbol in sub_df.index:
    try:
        depth = symbol_get_most_recent_depth_data(symbol, 2)
        valid_symbols.append(symbol)
    except Exception as e:
        print(symbol, "failed")


ANYUSDT failed
BTTUSDT failed
EOSBEARUSDT failed
ERDUSDT failed
NUUSDT failed
PAXUSDT failed
POLYUSDT failed
VENUSDT failed
XRPBULLUSDT failed
XTZUPUSDT failed


In [20]:
len(valid_symbols)

89

In [22]:
valid_symbols = list(perpetual_symbol_info_selected.index)

In [23]:
valid_symbols

['BTCUSDT',
 'ETHUSDT',
 'BCHUSDT',
 'XRPUSDT',
 'EOSUSDT',
 'LTCUSDT',
 'TRXUSDT',
 'ETCUSDT',
 'LINKUSDT',
 'XLMUSDT',
 'ADAUSDT',
 'BNBUSDT',
 'ATOMUSDT',
 'THETAUSDT',
 'ALGOUSDT',
 'ZILUSDT',
 'DOGEUSDT',
 'KAVAUSDT',
 'RLCUSDT',
 'WAVESUSDT',
 'DOTUSDT',
 'YFIUSDT',
 'CRVUSDT',
 'SUSHIUSDT',
 'SOLUSDT',
 'ICXUSDT',
 'UNIUSDT',
 'AVAXUSDT',
 'FTMUSDT',
 'ENJUSDT',
 'NEARUSDT',
 'AAVEUSDT',
 'FILUSDT',
 'MATICUSDT',
 'OCEANUSDT',
 'AXSUSDT',
 'ALPHAUSDT',
 'GRTUSDT',
 '1INCHUSDT',
 'BTCBUSD',
 'CHZUSDT',
 'SANDUSDT',
 'ANKRUSDT',
 'UNFIUSDT',
 'MANAUSDT',
 'ONEUSDT',
 'CELRUSDT',
 '1000SHIBUSDT',
 'ETHBUSD',
 'BNBBUSD',
 'XRPBUSD',
 'DOGEBUSD',
 'AUDIOUSDT',
 'MASKUSDT',
 'SOLBUSD',
 'DYDXUSDT',
 'GALAUSDT',
 'CELOUSDT',
 'PEOPLEUSDT',
 'FLOWUSDT',
 'GMTUSDT',
 'APEUSDT',
 'JASMYUSDT',
 'GALUSDT',
 'GMTBUSD',
 'GALABUSD',
 'OPUSDT',
 'LDOBUSD',
 'LUNA2USDT',
 'LDOUSDT',
 'APTUSDT',
 'APTBUSD']

In [42]:
# sub_df = avg_turnover.loc[busd_symbols].loc[(avg_turnover["mean"] >= min_turnover) & (avg_turnover["count"] == 30)]
# sub_df.sort_values(by="mean", ascending=False)

In [24]:
import pandas as pd
import os
os.chdir("/Users/bytedance/MLTrader-deploy/")
from utility import read_symbol

/opt/miniconda3/envs/vnpy3.9/lib/python3.9/site-packages/backtesting/_plotting.py:50: UserWarning: Jupyter Notebook detected. Setting Bokeh output to notebook. This may not work in Jupyter clients without JavaScript support (e.g. PyCharm, Spyder IDE). Reset with `backtesting.set_bokeh_output(notebook=False)`.
  warnings.warn('Jupyter Notebook detected. '


Loading BokehJS ...

In [28]:
new_dfs = {}
bar_window = 60
DATA_ROOT = "/Users/bytedance/binance_data/csv_data"
for asset_name in valid_symbols:
    try:
        sub_df = read_symbol(
            asset_name,
            bar_window,
            root=DATA_ROOT,
            fake=False,
        )
        sub_df.index = sub_df.index.tz_localize("UTC")
        if sub_df.size == 0:
            raise ValueError()

        new_dfs[asset_name] = sub_df
    except Exception as e:
        print(e)

result_df = pd.concat(new_dfs, axis=0, names=["Asset_ID"])
global_dv = {}
for feature in result_df.columns:
    global_dv[feature] = result_df.pivot(
        index="timestamp", columns="Asset_ID", values=feature
    )
    global_dv[feature] = global_dv[feature].fillna(method="ffill")

h5_path = os.path.join(DATA_ROOT, f"dataview_{bar_window}m.h5")
for key in global_dv:
    global_dv[key].to_hdf(h5_path, key, format="table")

/Users/bytedance/MLTrader-deploy/research/utility.py:432: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  df: pd.DataFrame = df.reindex(pd.Int64Index(np.arange(start, end + 60, 60)))


load BTCUSDT data succeed. 2017-08-17 05:00:00 to 2023-01-23 00:00:00; bar window=60


/Users/bytedance/MLTrader-deploy/research/utility.py:432: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  df: pd.DataFrame = df.reindex(pd.Int64Index(np.arange(start, end + 60, 60)))


load ETHUSDT data succeed. 2017-08-17 05:00:00 to 2023-01-23 00:00:00; bar window=60


/Users/bytedance/MLTrader-deploy/research/utility.py:432: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  df: pd.DataFrame = df.reindex(pd.Int64Index(np.arange(start, end + 60, 60)))


...

load GALABUSD data succeed. 2021-09-13 07:00:00 to 2023-01-23 00:00:00; bar window=60


/Users/bytedance/MLTrader-deploy/research/utility.py:432: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  df: pd.DataFrame = df.reindex(pd.Int64Index(np.arange(start, end + 60, 60)))


load OPUSDT data succeed. 2022-06-01 09:00:00 to 2022-12-07 00:00:00; bar window=60
[Errno 2] No such file or directory: '/Users/bytedance/binance_data/csv_data/LDOBUSD-1m.csv'
[Errno 2] No such file or directory: '/Users/bytedance/binance_data/csv_data/LUNA2USDT-1m.csv'


/Users/bytedance/MLTrader-deploy/research/utility.py:432: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  df: pd.DataFrame = df.reindex(pd.Int64Index(np.arange(start, end + 60, 60)))
/Users/bytedance/MLTrader-deploy/research/utility.py:432: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  df: pd.DataFrame = df.reindex(pd.Int64Index(np.arange(start, end + 60, 60)))


load LDOUSDT data succeed. 2022-05-09 12:00:00 to 2022-12-07 00:00:00; bar window=60
load APTUSDT data succeed. 2022-10-19 02:00:00 to 2022-12-07 00:00:00; bar window=60
[Errno 2] No such file or directory: '/Users/bytedance/binance_data/csv_data/APTBUSD-1m.csv'


In [5]:
global_dv.keys()

dict_keys(['Close', 'High', 'Low', 'Open', 'Volume', 'Count', 'timestamp', 'Asset_ID'])

In [155]:
from jaqs_fxdayu.research import SignalDigger